![purple-divider](https://user-images.githubusercontent.com/7065401/52071927-c1cd7100-2562-11e9-908a-dde91ba14e59.png)
<!-- ![green-divider](https://user-images.githubusercontent.com/7065401/52071924-c003ad80-2562-11e9-8297-1c6595f8a7ff.png) -->
### Import our libraries and reading the dataset

<!-- ![green-divider](https://user-images.githubusercontent.com/7065401/52071924-c003ad80-2562-11e9-8297-1c6595f8a7ff.png) -->


In [1]:
# import our data and libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re

# sp_data is a pandas dataframe of our data
sp_data = pd.read_json(
    '/media/rafael/D/2020 RAFAEL/Faculdade/ICs/IC Direito/data/sp_shinx.json', lines=True)

new_sp_data = sp_data.copy()

![green-divider](https://user-images.githubusercontent.com/7065401/52071924-c003ad80-2562-11e9-8297-1c6595f8a7ff.png)
### Couting what we need. 
Count the separator ' - ' in `Conteúdo` of sp_data

In [2]:
count_separator = [] # this is a vector which contains the frequency of ' - ' in each Document

count_Counteudo_length = [] # this is a vector which contains the lenght of "Conteúdo" in each Document

# for each Document in data, count the frequency of ' - ' and the len of "Conteúdo"
for i, Document in enumerate(sp_data['Conteúdo']):
    count_separator.append(Document.count(' - '))

data = np.array(count_separator) # vector which contains the amount of ' - ' in each document

![green-divider](https://user-images.githubusercontent.com/7065401/52071924-c003ad80-2562-11e9-8297-1c6595f8a7ff.png)
### Filtering the dataset
Filter the `Conteúdo` of all documents which has 4 or more ' - '

In [3]:
Doc_series = pd.Series(sp_data['Conteúdo'])

ind_range = (data > 4)

data_in_6_and_9 = sp_data['Conteúdo'][ind_range]

positions_all = []

processos_all = []

classes_all = []

assuntos_all = []

partes_all = []

conteudo_all = []

advs_all = []

for document in data_in_6_and_9:

    # ----------------------------------------------------------
    # Filtering the processes
    # ----------------------------------------------------------

    position = re.search(r' - ', document)
    processos_all.append(document[0:position.start()])

    doc_no_processos = document[position.end():]

    # ----------------------------------------------------------
    # Filtering the classes
    # ----------------------------------------------------------
    position = re.search(r' - ', doc_no_processos)
    classes_all.append(doc_no_processos[0:position.start()])

    doc_no_classes_to = doc_no_processos[position.end():]

    # ----------------------------------------------------------
    # Filtering the assuntos
    # ----------------------------------------------------------
    position = re.search(r' - ', doc_no_classes_to)
    assuntos_all.append(doc_no_classes_to[0:position.start()])

    doc_no_assuntos_to = doc_no_classes_to[position.end():]

    # ----------------------------------------------------------
    # Filtering the parts
    # ----------------------------------------------------------
    position = re.search(r' - ', doc_no_assuntos_to)
    partes_all.append(doc_no_assuntos_to[0:position.start()])

    doc_no_parts_to = doc_no_assuntos_to[position.end():]

    # ----------------------------------------------------------
    # Filtering the content
    # ----------------------------------------------------------

    # ----------------------------------------------------------
    # Filtering the ADV
    # ----------------------------------------------------------
    position = re.search(r"(?i)- ADV", doc_no_parts_to)
    if position:
        advs_all.append(doc_no_parts_to[position.start():])

        conteudo_all.append(doc_no_parts_to[0: position.start()])
    else:
        conteudo_all.append(doc_no_parts_to)

![green-divider](https://user-images.githubusercontent.com/7065401/52071924-c003ad80-2562-11e9-8297-1c6595f8a7ff.png)
### Taking the right number of process

In [4]:
# conteudo_all[]
# processos_all[1],classes_all[1],assuntos_all[1],partes_all[1],sp_data['Conteúdo'][1]

regex_processo = r"\d{6,7}.\d{2,3}.\d{4,5}.\d{1,2}.\d{2}\.\d{4,5}" # return the pattern 

processos = sp_data['Conteúdo'].str.findall(regex_processo)

In [5]:
# take only the first time which that pattern was saw
teste = processos.copy()
teste = processos.str[0]

# add new process to "Processo" collumn
sp_data['Processo'] = teste

sp_data['Processo'][teste.isnull()] = new_sp_data['Processo'][teste.isnull()]

/home/rafael/.local/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


![green-divider](https://user-images.githubusercontent.com/7065401/52071924-c003ad80-2562-11e9-8297-1c6595f8a7ff.png)
### Seeying the true Assuntos of files

In [6]:
import time
import multiprocessing 

from tqdm import tqdm

assuntos = ["Periculosidade","Contribuição / Taxa Assistencial","Contribuição Confederativa","Eleição","Registro de Entidade Sindical","Salário In Natura","Assédio Moral","Assédio Sexual","Cesta Básica","Contribuição Sindical","Comissão","Complemento Temporário Variável de Ajuste ao Piso de Mercado","Culpa Recíproca","Descontos Salariais - Devolução","Despedida / Dispensa Imotivada","Justa Causa / Falta Grave","Diárias","Dirigente Sindical","Empregado Público","Portador de HIV","Contrato Suspenso","Estabilidade decorrente de Norma Coletiva","Gestante","Membro da Cipa","Multa de 40% do FGTS","Fruição / Gozo","Indenização / Terço Constitucional","Supressão de Horas Extras Habituais - Indenização","Multa do Artigo 467 da CLT","Multa do Artigo 477 da CLT","Multa Prevista em Norma Coletiva","Plano de Demissão Voluntária","PIS - Indenização","Piso Salarial da Categoria / Salário Mínimo Profissional","Prêmio","Contribuição de Previdência Privada - Resgate","Plano de Saúde","Rescisão Indireta","Reajuste Salarial","Gorjeta","Salário Vencido / Retido","Salário Família","Salário Paternidade","Salário Por Fora - Integração","Salário Por Safra","Seguro de Vida","Indenização","Liberação das Guias","Tarefa","Tíquete Alimentação","Vale Transporte","Contrato de Aprendizagem","Cooperativa de Trabalho","Contrato de Estágio","Acidente de Trabalho","Abono","Adicional de Transferência","Ajuda de Custo","Complementação de Aposentadoria / Pensão","AVISO PRÉVIO","Anistia","Estabilidade Acidentária","Abono Pecuniário","Décimo Terceiro Salário","Salário por Equiparação / Isonomia","Tomador de Serviços / Terceirização","Homicídio Simples","Homicídio Privilegiado","Homicídio Qualificado","Induzimento, Instigação ou Auxílio a Suicídio","Infanticídio","Leve","Seguida de Morte","Perigo de contágio de moléstia grave","Perigo para a vida ou saúde de outrem","Abandono de incapaz","Omissão de socorro","Rixa","Calúnia","Difamação","Injúria","Constrangimento ilegal","Ameaça","Seqüestro e cárcere privado","Redução a condição análoga à de escravo","Violação de domicílio","Violação de correspondência","Sonegação ou destruição de correspondência","Violação de comunicação telegráfica, radioelétrica ou telefônica","Violação de Correspondência comercial","Divulgação de segredo","Violação do segredo profissional","Furto","Furto Qualificado","Furto de coisa comum","Roubo","Extorsão","Extorsão mediante seqüestro","Extorsão indireta","Alteração de limites","Usurpação de águas","Esbulho possessório","Dano","Introdução ou abandono de animais em propriedade alheia","Dano em coisa de valor artístico, arqueológico ou histórico","Alteração de local especialmente protegido","Apropriação indébita Previdenciária","Estelionato","Estelionato Majorado","Outras fraudes","Fraude à execução","Receptação","Apropriação indébita","Apropriação de Coisa Havida por Erro, Caso Fortuito ou Força da Natureza","Violação de direito autoral","Atentado contra a liberdade de trabalho","Atentado contra a liberdade de contrato de trabalho e boicotagem violenta","Atentado contra a liberdade de associação","Paralisação de trabalho, seguida de violência ou perturbação da ordem","Paralisação de trabalho de interesse coletivo","Invasão de estabelecimento industrial, comercial ou agrícola. Sabotagem","Frustração de direitos assegurados por lei trabalhista","Exercício de atividade com infração de decisão administrativa","Aliciamento para fins de emigração","Aliciamento de trabalhadores de um local para outro do território nacional","Frustração de lei sobre a nacionalização do trabalho","Destruição / Subração / Ocultação de Cadáver","Impedimento ou Perturbação de Cerimônia Funerária","Vilipêndio a Cadáver","Violação a Sepultura","Ultraje / Impedimento ou Perturbação de Culto Religioso","Estupro","Atentado Violento ao Pudor","Atentado ao Pudor Mediante Fraude","Corrupção de Menores","Ultraje Público ao Pudor (Ato/Escrito Obsceno)","Mediação para Servir a Lascívia de Outrem","Abandono Material","Abandono Intelectual","Entrega de Filho Menor a Pessoa Inidônea","Bigamia","Conhecimento Prévio de Impedimento","Induzimento a erro essencial e ocultação de impedimento","Simulação de Casamento","Simulação de Autoridade para Celebração de Casamento","Parto Suposto","Registro de Nascimento Inexistente","Sonegação do Estado de Filiação","Induzimento a Fuga, Entrega Arbitrária ou Sonegação de Incapazes","Subtração de Incapazes","Incêndio","Explosão","Uso de gás tóxico ou asfixiante","Inundação / Perigo de Inundação","Desabamento ou desmoronamento","Subtração, ocultação ou inutilização de material de salvamento","Difusão de doença ou praga","Desastre ferroviário / Perigo de Desastre Ferroviário","Atentado contra a segurança de transporte público","Arremesso de projétil","Atentado contra a segurança de serviços de utilidade pública","Interrupção /perturbação de serviços telegráficos/telefônicos","Epidemia","Emprego de Processo Proibido/Substância não permitida","Invólucro ou recipiente com falsa indicação","Falsificação / Corrupção / Adulteração / Alteração de produto destinado a fins terapêuticos ou medicinais","Exercício Ilegal da Medicina, Arte Dentária ou Farmacêutica","Charlatanismo","Corrupção ou Poluição de Água Potável","Curandeirismo","Envenenamento de Água Potável / Substância Alimentícia ou Medicinal","Falsificação / Corrupção / Adulteração / Alteração de Substância ou Produtos Alimentícios","Infração de Medida Sanitária Preventiva","Omissão de Notificação de Doença","Outras Substâncias Nocivas a Saúde Pública","Substância Destinada à Falsificação","Quadrilha ou Bando","Moeda Falsa / Assimilados","Petrechos para falsificação de moeda","Emissão de título ao portador sem permissão legal","Falsificação de papéis públicos","Falsificação do selo ou sinal público","Falsificação de documento público","Falsificação de documento particular","Falsidade ideológica","Falso reconhecimento de firma ou letra","Certidão ou atestado ideologicamente falso","Falsidade material de atestado ou certidão","Falsidade de atestado médico","Reprodução ou adulteração de selo ou peça filatélica","Uso de documento falso","Supressão de documento","Falsificação do sinal empregado no contraste de metal precioso ou na fiscalização alfandegária, ou para outros fins","Falsa identidade","Fraude de lei sobre estrangeiros","Petrechos de Falsificação de Papéis Públicos","Adulteração de Sinal Identificador de Veículo Automotor","Peculato","Peculato mediante erro de outrem","Extravio, sonegação ou inutilização de livro ou documento","Emprego irregular de verbas ou rendas públicas","Concussão","Excesso de exação","Corrupção passiva","Facilitação de contrabando ou descaminho","Prevaricação","Condescendência criminosa","Advocacia administrativa","Violência arbitrária","Abandono de função","Exercício funcional ilegalmente antecipado ou prolongado","Violação do sigilo funcional","Violação do sigilo de proposta de concorrência","Usurpação de função pública","Resistência","Tráfico de influência","Corrupção ativa","Impedimento, perturbação ou fraude de concorrência","Inutilização de edital ou de sinal","Subtração ou inutilização de livro ou documento","Desobediência","Desacato","Contrabando ou descaminho","Reingresso de estrangeiro expulso","Denunciação caluniosa","Comunicação falsa de crime ou de contravenção","Auto-acusação falsa","Falso testemunho ou falsa perícia","Coação no curso do processo","Exercício arbitrário das próprias razões","Fraude processual","Favorecimento pessoal","Favorecimento real","Exercício arbitrário ou abuso de poder","Fuga de pessoa presa ou submetida a medida de segurança","Evasão mediante violência contra a pessoa","Arrebatamento de preso","Motim de presos","Patrocínio infiel","Patrocínio simultâneo ou tergiversação","Sonegação de papel ou objeto de valor probatório","Exploração de prestígio","Violência ou fraude em arrematação judicial","Desobediência a decisão judicial sobre perda ou suspensão de direitos","Inserção de dados falsos em sistema de informações","Modificação ou alteração não autorizada de sistema de informações","Sonegação de contribuição previdenciária","Crimes de Responsabilidade","Crimes contra a Economia Popular","Crimes de Abuso de Autoridade","Tráfico de Drogas e Condutas Afins","Responsabilidade Criminal por Danos Nucleares","Crimes contra o Serviço Postal e o Serviço de Telegrama","Crimes contra a Segurança Nacional, a Ordem Política e Social","Crimes contra o Sistema Financeiro Nacional","Crimes Resultante de Preconceito de Raça ou de Cor","Crimes contra a Ordem Tributária","Crimes contra a Ordem Econômica","Crimes contra as Relações de Consumo","Representação caluniosa","Crimes contra a Fauna","Crimes contra a Flora","Da Poluição","Agrotóxicos","Atividades Nucleares","Caça","Liberação ou Descarte de OGM (Organismo Geneticamente Modificado)","Pesca","ou Ocultação de Bens, Direitos ou Valores","Crimes contra as Telecomunicações","Crimes Previstos na Lei de Estrangeiros","Crimes de Tortura","Crimes de Trânsito","Crimes do Sistema Nacional de Armas","Crimes de Imprensa","Crimes Previstos no Estatuto da criança e do adolescente","Crimes contra o mercado de capitais","Crimes de genocídio","Sigilo Telefônico","Crimes da Lei de licitações","Crimes da Lei de remoção de órgãos e tecidos","Crimes praticados contra os índios e a cultura indígena","Recusa, retardamento ou omissão de dados técnicos indispensáveis à propositura de Ação Civil Pública","Crimes Agrários","Crimes relacionados à Comissão Parlamentar de Inquérito","Crime de Quebra de Sigilo Financeiro","Crimes do Código Brasileiro de Telecomunicações","Crimes Previstos no Estatuto do Idoso","Parcelamento do solo urbano","Crimes Falimentares","Trancamento","Fiança","Prisão Preventiva","Recebimento","Rejeição","Aplicabilidade","Revisão","Anulação","Restituição / Indenização de Despesa","Representação Sindical","Direito Autoral","Patente","Desenho Industrial","Marca","Defeito, nulidade ou anulação","Evicção ou Vicio Redibitório","Cessão de Crédito","Câmbio","Parceria Agrícola e/ou pecuária","Representação comercial","Atraso de vôo","Cancelamento de vôo","Overbooking","Extravio de bagagem","Equivalência salarial","Quitação","Reajuste de Prestações","Transferência de Financiamento (contrato de gaveta)","Sustação/Alteração de Leilão","Seguro","Revisão do Saldo Devedor","Assembléia","Eleição","Extinção","Assembléia","Eleição","Extinção","Apuração de haveres","Constituição","Dissolução","Desconsideração da Personalidade Jurídica","Ingresso e Exclusão dos Sócios na Sociedade","Responsabilidade dos sócios e administradores","Transferência de cotas","Anulação","Requisitos","Cédula de Crédito Bancário","Cédula de Crédito à Exportação","Cédula de Crédito Comercial","Cédula de Crédito Industrial","Cédula de Crédito Rural","Cédula de Produto Rural","Cédula Hipotecária","Cheque","Debêntures","Duplicata","Letra de Câmbio","Nota de Crédito Comercial","Nota de Crédito Industrial","Nota de Crédito Rural","Nota Promissória","Warrant","Recuperação extrajudicial","Autofalência","Concurso de Credores","Liquidação","Revogação de atos praticados em prejuízo de credores e da massa","Bolsa de Valores","Engajamento e Profissionais Marítimos","Seguros Marítimos","Aluguel de Embarcações (Fretamento E Carta Partida)","Mínimo","Advogados","Aeronautas","Artistas","Atleta Profissional","Bancários","Cabineiros de Elevador","Doméstico","Engenheiro, Arquiteto e Engenheiro Agrônomo","Jornalistas","Marítimos","Digitadores / Mecanógrafos / Datilógrafos","Médicos","Mineiros de Subsolos","Músicos Profissionais","Operadores de Carga e Descarga (Estiva e Capatazia)","Operadores Cinematográficos","Professores","Químicos","Radialistas","Serviços de Telefonia ou Telegrafia","Trabalhadores em Petróleo","Vigia e Vigilantes","Grupo Econômico","Crime Tentado","Grave","Gravíssima","Privilegiada","Furto Privilegiado","Furto de Veículo Automotor a ser Transportado para outro Estado ou Exterior","Roubo Majorado","Latrocínio","Extorsão mediante Seqüestro Seguida de Morte","Extorsão mediante Sequestro Seguida de Lesão Corporal Grave","Supressão ou Alteração de Marca em Animais","Dano Qualificado","Apropriação de Tesouro","Apropriação de Coisa Achada","Abandono","Acidentes da Navegação","Registro / Cadastro do Armador","Arresto de Embarcação","Assistência / Salvamento","Avaria","Responsabilidade do Comandante ou Capitão","Inscrição / Registro da Embarcação","Hipoteca Marítima","Créditos / Privilégios Marítimos","Agenciamento","Corretagem de Embarcação","Praticagem","Prescrição e Decadência","Edição","Exoneração","Revisão","Busca e Apreensão de Menores","Guarda","Investigação de Paternidade","Regulamentação de Visitas","Nulidade / Anulação","Nulidade e Anulação de Testamento","Nulidade e Anulação de Partilha e Adjudicação de Herança","Sub-rogação de Vinculo","Exclusão de herdeiro ou legatário","Petição de Herança","Deserdação","Disposição de coisa alheia como própria","Alienação ou Oneração Fraudulenta de Coisa Própria","Defraudação de Penhor","Fraude na Entrega de Coisa","Fraude para Recebimento de Indenização ou Valor de Seguro","Fraude no Pagamento por Meio de Cheque","Duplicata Simulada","Abuso de Incapazes","Induzimento à Especulação","Fraude no Comércio","Fraudes e Abusos na Fundação ou Administração de Sociedade por Ações","Receptação Qualificada","Posse Sexual Mediante Fraude","Assédio Sexual","Favorecimento da Prostituição","Casa de Prostituição","Rufianismo","Tráfico Internacional de Pessoas","Tráfico Interno de Pessoas","Fabrico / Fornecimento / Aquisição / Posse ou Transporte de Explosivos ou Gás Tóxico ou Asfixiante","Crime Culposo","Venda de Produtos ou Substância nas Condições dos Artigos 274 e 275","Medicamento em Desacordo com Receita Médica","Incitação ao Crime","Apologia de Crime ou Criminoso","Corrupção Ativa em Transação Comercial Internacional","Tráfico de Influência em Transação Comercial Internacional","Crimes contra Patente de Invenção","Crimes contra os Desenhos Industriais","Crimes contra as Marcas","Crimes Cometidos por Meio de Marca, Título de Estabelecimento e Sinal de Propaganda","Crimes contra Indicações Geeográficas e Demais Indicações","Crimes de Concorrência Desleal","Crime Contra a Administração da Justiça - Lei da Ação de Alimentos","Fabricação de Objeto Destinado a Produção de Drogas e Condutas Afins","Indução, Instigação ou Auxílio ao Uso de Drogas","Oferecimento de Drogas para Consumo Conjunto","Associação para a Produção e Tráfico e Condutas Afins","Financiamento ou Custeio de Produção ou Tráfico de Drogas","Colaboração com Grupo, Organização ou Associação Destinados à Produção ou Tráfico de Drogas","Prescrição Culposa de Drogas","Condução de Embarcação ou Aeronave sob Efeito de Drogas","Contratação de Operação de Crédito","Inscrição de Despesas Não Empenhadas","Assunção de Obrigação no Último Ano do Mandato ou Legislatura","Ordenação de Despesa Não Autorizada","Prestação de Garantia Graciosa","Não Cancelamento de Restos a Pagar","Aumento de Despesa com Pessoal no Último Ano do Mandato ou Legislatura","Oferta Pública ou Colocação de Títulos no Mercado","Perigo de Contágio Venéreo","Isenção","Incidência sobre PDV","Incidência sobre Aposentadoria","Incidência sobre Aplicações Financeiras","Incidência sobre Licença-Prêmio/Abono/Indenização","Retido na fonte","Incidência sobre Férias Compensadas","Incidência sobre Proventos de Previdência Privada","Incidência sobre Participação nos Lucros","Incidência sobre Lucro Imobiliário","Incidência sobre 1/3 de férias (art. 7º, XVII da CF)","Incidência sobre 13° Salário","Incidência sobre Função Comissionada","Incidência sobre Hora Extra","Incidência decorrente de Desligamento de Plano de Previdência Privada","Incidência decorrente de Liquidação de Entidade de Previdência Privada","Microempresa","Demonstrações Financeiras (DCTF)","Incidência sobre Aplicações Financeiras","Retido na fonte","Incidência sobre Lucro","Cooperativa","Compensação de Prejuízos","Importação de bens usados","Drawback","IE/ Imposto sobre Exportação","ICMS/Importação","Incidência sobre Ações de Companhias Abertas","ITR/ Imposto Territorial Rural","ISS/ Imposto sobre Serviços","IPTU/ Imposto Predial e Territorial Urbano","IPVA - Imposto Sobre Propriedade de Veículos Automotores","ITBI - Imposto de Transmissão Intervivos de Bens Móveis e Imóveis","ITCD - Imposto de Transmissão Causa Mortis","Taxa Anual por Hectare","Taxa de Armazenamento","Taxa de Despacho Aduaneiro","Taxa de Exploração Mineral","Taxa de Fiscalização Ambiental","Taxa de Guia de Importação","Taxa de Licença de Importação","Taxa de Melhoramento de Portos","Taxa de Permanência","Taxa de Saúde Suplementar","Taxa de Fiscalização da Comissão de Valores Mobiliários","Taxa Judiciária","Taxa de Aferição de Equipamentos de Metrologia","Estaduais","Aquisição de veículos automotores","Aquisição de combustíveis","Aquisição de passagens e moedas - Viagem Exterior","Energia Elétrica","Responsabilidade Tributária do Sócio-Gerente / Diretor / Representante","Substituição Tributária","DIMOB/Declaração de Informações sobre Atividades Imobilíárias","CNPJ/Cadastro Nacional de Pessoas Jurídicas","CPF/Cadastro de Pessoas Físicas","Obrigação Acessória","Carta de fiança","Parcelamento","Compensação com Apólices da Dívida Pública","Compensação","TDA/Títulos da Dívida Agrária","Anistia","Denúncia espontânea","Lançamento","Expedição de CND","Expedição de Certidão Positiva de Débito com Efeito de Negativa","Certificado de Regularidade - FGTS","Incentivos fiscais","Anulação de Débito Fiscal","Juros/Correção Monetária","Prazo de Recolhimento","Repetição de indébito","Crédito Prêmio","Crédito Presumido","Fato Gerador/Incidência","Alíquota Zero","Alíquota Progressiva","Índice da Alíquota","Creditamento","Depósito Prévio ao Recurso Administrativo","Arrolamento de Bens","Utilização de Dados Relativos à CPMF para Fins de Fiscalização","Liberação de mercadorias","Levantamento de depósito","Cadastro de Inadimplentes - CADIN","Autorização para Impressão de Documentos Fiscais - AIDF","Liberação de Veículo Apreendido","Perdimento de Bens","Contribuições de Melhoria","Finsocial","Salário-Educação","Seguro Acidentes do Trabalho","Funrural","Contribuição INCRA","PASEP","Construção Civil","Contribuições para o SEBRAE, SESC, SENAC, SENAI e outros","Conselhos Regionais e Afins (Anuidade)","Contribuição Sindical","Servidores Ativos","Servidores Inativos","Mandato Eletivo/Lei 9.506/97","Aposentadoria/Retorno aoTrabalho","Cargo em Comissão","Tempo de Serviço Urbano/Contribuições não Recolhidas","Tempo de Serviço Rural/Contribuições não Recolhidas","Gratificação Natalina/13º Salário","Produção Agropecuária","Indenização Trabalhista","Reembolso auxílio-creche","Contribuição sobre a folha de salários","Devolução de contribuições previdenciárias pagas além do teto","1/3 de férias","Auxílio-Alimentação","Custeio de Assistência Médica","Hora Extra","Massa Falida- Recolhimento","Nota Fiscal ou Fatura","Salário-Maternidade","Contribuição sobre Nota Fiscal de Execução de Serviços","Contribuição de Autônomos, Empresários (Pró-labore) e Facultativos","FNT/Fundo Nacional de Telecomunicações","ATP/Adicional de Tarifa Portuária","AITP/Adicional de Indenização do Trabalhador Portuário Avulso","Adicional de Tarifa Aeroportuária","AFRMM/Adicional ao Frete para Renovação da Marinha Mercante","FUNDEF/Fundo de Manutenção e Desenvolvimento do Ensino Fundamental e de Valorização do Magistério","Seguro Apagão (Lei 10.438/02)","Contribuição sobre licença de uso ou transferência de tecnologia","Contribuição de Intervenção no Domínio Econômico (CIDE)","FUNDAF/Fundo Especial de Desenvolvimento e Aperfeiçoamento das Atividades de Fiscalização","FUSEX/FUNSA/FUSMA/Fundo de Saúde das Forças Armadas","Contribuição de Iluminação Pública","IUEE/Imposto Único sobre Energia Elétrica","FGTS/Fundo de Garantia Por Tempo de Serviço","Contribuição sobre Açúcar e Álcool","CPMF/Contribuição Provisória sobre Movimentação Financeira","Discriminação Tributária MERCOSUL","REFIS/Programa de Recuperação Fiscal","PAES/Parcelamento Especial","SIMPLES","Super SIMPLES","Aposentadoria por Invalidez","Urbana (Art. 48/51)","Rural (Art. 48/51)","Aposentadoria por Tempo de Serviço (Art. 52/4)","Aposentadoria Especial (Art. 57/8)","Auxílio-Doença Previdenciário","Salário-Família (Art. 65/70)","Salário-Maternidade (Art. 71/73)","Pensão por Morte (Art. 74/9)","Auxílio-Reclusão (Art. 80)","Pecúlios (Art. 81/5)","Incapacidade Laborativa Parcial","Incapacidade Laborativa Permanente","Incapacidade Laborativa Temporária","Movimentos Repetitivos/Tenossinovite/LER/DORT","Redução da Capacidade Auditiva","Abono de Permanência em Serviço (Art. 87)","Ferroviário","Ex-combatentes","Renda Mensal Vitalícia","Aposentadoria por Tempo de Contribuição (Art. 55/6)","RMI pelo art. 202 CF/88 (média dos 36 últimos salários-de-contribuição)","RMI pelo art. 1º da Lei 6.423/77 - índices de atualização dos 24 1ºs salários-de-contribuição, anteriores aos 12 últimos)","RMI pela equivalência entre Salário-de-Benefício e Salário-de-Contribuição","RMI sem incidência de Teto Limitador","RMI da pensão de dependente de ex-combatente","Benefício mínimo a partir da CF/88 (art. 201, § 2º CF/88)","Limitação do salário-de-benefício e da renda mensal inicial","RMI cuja salário-de-benefício supera menor valor teto","Escala de Salário-Base","Contribuição sobre vinte salários mínimos","Cálculo do Benefício de acordo com a Sistemática anterior à Lei 9.876/99","IRSM de Fevereiro de 1994(39,67%)","Alteração do coeficiente de cálculo de pensão","Alteração do coeficiente de cálculo do benefício","Cálculo do benefício de segurado especial de acordo com a Lei 9.876/99","Cálculo do fator previdenciário - Lei 9.876/99","Reajuste pela Súmula 260 do TFR","Art. 58 ADCT da CF/88","Utilização do PNS no Reajuste de Benefícios","Manutenção do Benefício pela equivalência salarial","Expurgos inflacionários sobre os benefícios","Salário Mínimo de Ncz$ 120,00 para junho/89","Reajuste de 147%","Reajustamento pelo IGP-DI","Aplicação de coeficiente de cálculo diverso do fixado na Lei n.º 8.213/91","Reajuste aplicado ao salário mínimo em setembro/94","Reajustamento pelo INPC","Art. 144 da Lei 8.213/91 e/ou diferenças decorrentes","Gratificação Natalina a partir da CF/88 (Art. 201, § 6º CF/88)","Abono da Lei 8.178/91","Índice de 4,02% da Lei 8.222/91","Desconto do DL 1.910/81","Descontos dos benefícios","Correção Monetária pela Súmula 71 TFR","Correção Monetária de Benefício pago com atraso","Reajuste conforme Portaria MPAS 714/1993","Habilitação e Reabilitação Profissional","Decadência/Prescrição","Declaração de Ausência","Atividade concomitante","Renúncia ao benefício","Demonstrativo das importâncias pagas","Períodos de Carência","Limite Mínimo de Idade","Perda da qualidade de segurado","Inclusão de Dependente","Data de Início de Benefício (DIB)","Cumulação","Parcelas de benefício não pagas","Concessão","Restabelecimento","Conversão","Averbação/Cômputo/Conversão de tempo de serviço especial","Averbação/Cômputo de tempo de serviço de segurado especial (regime de economia familiar)","Averbação/Cômputo de tempo de serviço  rural (empregado/empregador)","Averbação/Cômputo do tempo de serviço como aluno aprendiz","Averbação/Cômputo do tempo de serviço militar","Averbação/Cômputo de tempo de serviço de empregado doméstico","Averbação/Cômputo de tempo de serviço urbano","Certidão de Tempo de Serviço","Contagem Recíproca de Tempo de Serviço","Aquisição","Perda","Concessão de Naturalização","Cancelamento de Naturalização","Admissão / Entrada / Permanência / Saída","Asilo","Livre Trânsito MERCOSUL","Trabalhador Migrante ou Fronteiriço do MERCOSUL","Proteção Internacional a Direitos Humanos","Auxílio Direto","Mercosul","Tratado Internacional","Normas do Mercosul","Casamento e Divórcio","Sucessão de Bens de Estrangeiro","Prestação de Alimentos","Funcionamento de Empresa Estrangeira","Pessoa Jurídica Estrangeira","Contratos Internacionais","Inclusão Indevida em Cadastro de Inadimplentes","Oferta","Fixação","Turismo","Consórcio","Estabelecimentos de Ensino","Seguro","Assinatura Básica Mensal","Pulsos Excedentes","Empreitada","Representante Comercial Autônomo","Honorários Profissionais","Trabalhador Eventual","Trabalhador Avulso","Interdito Proibitório","Aeroviários","Rural","Aplicabilidade","Tutela e Curatela","Regime de Bens Entre os Cônjuges","Usufruto e Administração dos Bens de Filhos Menores","Bem de Família","Dissolução","Investigação de Maternidade","Adoção de Maior","União Homoafetiva","Administração de herança","Reconhecimento / Dissolução","Inventário e Partilha","Assunção de Dívida","Preferências e Privilégios Creditórios","Correção Monetária","Perdas e Danos","Cláusula Penal","Arras ou Sinal","Pagamento","Pagamento em Consignação","Pagamento com Sub-rogação","Imputação do Pagamento","Dação em Pagamento","Novação","Compensação","Confusão","Remissão das Dívidas","Promessa de Recompensa","Gestão de Negócios","Pagamento Indevido","Enriquecimento sem Causa","Registro de nascimento após prazo legal","Retificação de Nome","Cancelamento de Protesto","Acidente Aéreo","Auxílio-Doença Acidentário","Fornecimento de Energia Elétrica","Fornecimento de Água","Substituição do Produto","Rescisão do contrato e devolução do dinheiro","Abatimento proporcional do preço","Interpretação / Revisão de Contrato","Cartão de Crédito","Financiamento de Produto","Serviços Profissionais","Serviços Hospitalares","Indenização por Dano Material","Protesto Indevido de Título","Prestação Pecuniária","Perda de Bens e Valores","Prestação de Serviços à Comunidade","Interdição Temporária de Direitos","Limitação de Fim de Semana","Pena de Multa","Tratamento Ambulatorial","Internação","Quanto à Embarcação","Quanto à Carga","Clandestinos","Retificação de Data de Nascimento","Retificação de Sexo","Inscrição na Matrícula de Registro Torrens","Instituição de Bem de Família","Bloqueio de Matrícula","Por Remição","Por Terceiro Prejudicado","Registro de Óbito após prazo legal","Registro de Nascimento de Filho de Brasileiro Nascido no Exterior","Liberdade Provisória","Prisão em flagrante","Perempção","Inclusão/exclusão de Jurado","Deserção","Denegação","Falsidade","Direitos da Personalidade","Organizações Religiosas","Sucessão de Empregadores","Subempreitada","Sócio / Acionista","Acidente de Trabalho","Salário por Acúmulo de Cargo / Função","Salário Maternidade","Licenças e Folgas - Conversão em Pecúnia","Salário Substituição","Gratificação por Tempo de Serviço","Gratificação de Função","Décimo Terceiro Salário Proporcional","Férias Proporcionais","Indenização Adicional","Saldo de Salário","Indenização do artigo 478 da CLT","Imunidade de Jurisdição","Assistência Judiciária Gratuita","Intervenção de Terceiros","Litigância de Má-Fé","Litisconsórcio","Sucessão Processual","Procuração","Representação em Juízo","Ministério Público","Dos Auxiliares da Justiça","Prazo","Valor da Causa","Suspensão do Processo","Modificação ou Alteração do Pedido","Antecipação de Tutela / Tutela Específica","Erro de Procedimento","Revelia","Vícios Formais da Sentença","Efeitos","Preparo / Deserção","Desconfiguração de Justa Causa","Cabimento","Levantamento de Valor","Penhora / Depósito/ Avaliação","Multa de 10%","Inexequibilidade do Título / Inexigibilidade da Obrigação","Expropriação de Bens","Remição","Liminar","Honorários Periciais","Extinção da Execução","Concurso de Credores","Execução Previdenciária","Fraude à Execução","Imunidade de Execução","Sucessão","Capacidade Processual","Substituição da Parte","Causas Supervenientes à Sentença","Efeito Suspensivo / Impugnação / Embargos à Execução","Benefício de Ordem","Ato Atentatório à Dignidade da Justiça","Indenização do Prejuízo","Caução","Coligação","Transformação","Incorporação","Fusão","Cisão","Liquidação","Alteração de capital","Capacidade","Curadoria dos bens do ausente","Sucessão Provisória","Inclusão de associado","Exclusão de associado","Fiscalização","Exclusão de filiado","Fusão","Eleições - Diretórios","Ineficácia de atos  em relação à massa","Convolação de recuperação judicial em falência","Administração judicial","Classificação de créditos","Laudo Arbitral Internacional","Sustação de Protesto","Agêncie e Distribuição","Alienação Fiduciária","Arrendamento Rural","Arrendamento Mercantil","Cartão de Crédito","Comissão","Compra e Venda","Corretagem","Depósito","Doação","Empreitada","Fiança","Mandato","Troca ou Permuta","Prestação de Serviços","Seguro","Transação","Transporte de Coisas","Transporte de Pessoas","Estimatório","Comodato","Mútuo","Constituição de Renda","Jogo e Aposta","Compromisso","Contratos Bancários","Franquia","Locação de Móvel","Despejo para Uso Próprio","Despejo para Uso de Ascendentes e Descendentes","Despejo por Denúncia Vazia","Benfeitorias","Direito de Preferência","Conta de Participação","Simples","Nome Coletivo","Comandita Simples","Limitada","Anônima","Comandita por Ações","Cooperativa","Coligadas","Dependente de Autorização","Estrangeira","Homicídio Simples","Homicídio Privilegiado","Homicídio Qualificado","Induzimento, Instigação ou Auxílio a Suicídio","Infanticídio","Leve","Seguida de Morte","Grave","Gravíssima","Privilegiada","Perigo de contágio de moléstia grave","Perigo para a vida ou saúde de outrem (art. 132)","Abandono de incapaz (art. 133)","Omissão de socorro (art. 135)","Rixa (art. 137)","Perigo de Contágio Venéreo","Calúnia","Difamação","Injúria","Constrangimento ilegal (art. 146)","Ameaça (art. 147)","Seqüestro e cárcere privado (art. 148)","Redução a condição análoga à de escravo (art. 149)","Violação de domicílio (art. 150)","Violação de correspondência (art. 151, caput)","Sonegação ou destruição de correspondência (art. 151, § 1º, I)","Violação de comunicação telegráfica, radioelétrica ou telefônica (art. 151, § 1º, II, III e IV)","Violação de Correspondência comercial (art. 152)","Divulgação de segredo (art. 153)","Violação do segredo profissional (art. 154)","Furto (art. 155)","Furto Qualificado (Art. 155, § 4o.)","Furto de coisa comum (art. 156)","Roubo (art. 157)","Extorsão (art. 158)","Extorsão mediante seqüestro (art. 159)","Extorsão indireta (art. 160)","Alteração de limites (art. 161, caput)","Usurpação de águas (art. 161, § 1º, I)","Esbulho possessório (art. 161, § 1º, II)","Dano (art. 163)","Introdução ou abandono de animais em propriedade alheia (art. 164)","Dano em coisa de valor artístico, arqueológico ou histórico (art. 165)","Alteração de local especialmente protegido (art. 166)","Apropriação indébita Previdenciária (art. 168-A e Lei 8.212/91)","Estelionato","Estelionato Majorado (art. 171, § 3º)","Outras fraudes","Fraude à execução (art. 179)","Receptação","Apropriação indébita (art. 168, caput)","Apropriação de Coisa Havida por Erro, Caso Fortuito ou Força da Natureza (art.169)","Furto Privilegiado","Furto de Veículo Automotor a ser Transportado para outro Estado ou Exterior","Roubo Majorado","Latrocínio","Extorsão mediante Seqüestro Seguida de Morte","Extorsão mediante Sequestro Seguida de Lesão Corporal Grave","Supressão ou Alteração de Marca em Animais","Dano Qualificado","Apropriação de Tesouro","Apropriação de Coisa Achada","Disposição de coisa alheia como própria","Alienação ou Oneração Fraudulenta de Coisa Própria","Defraudação de Penhor","Fraude na Entrega de Coisa","Fraude para Recebimento de Indenização ou Valor de Seguro","Fraude no Pagamento por Meio de Cheque","Duplicata Simulada","Abuso de Incapazes","Induzimento à Especulação","Fraude no Comércio","Fraudes e Abusos na Fundação ou Administração de Sociedade por Ações","Receptação Qualificada","Violação de direito autoral","Atentado contra a liberdade de trabalho (art. 197)","Atentado contra a liberdade de contrato de trabalho e boicotagem violenta (art. 198)","Atentado contra a liberdade de associação (art. 199)","Paralisação de trabalho, seguida de violência ou perturbação da ordem (art. 200)","Paralisação de trabalho de interesse coletivo (art. 201)","Invasão de estabelecimento industrial, comercial ou agrícola. Sabotagem (art. 202)","Frustração de direitos assegurados por lei trabalhista (art. 203)","Exercício de atividade com infração de decisão administrativa (art. 205)","Aliciamento para fins de emigração (art. 206)","Aliciamento de trabalhadores de um local para outro do território nacional (art. 207)","Frustração de lei sobre a nacionalização do trabalho (art. 204)","Destruição / Subração / Ocultação de Cadáver","Impedimento ou Perturbação de Cerimônia Funerária (Art. 209)","Vilipêndio a Cadáver","Violação a Sepultura (Art. 210)","Ultraje / Impedimento ou Perturbação de Culto Religioso","Tráfico Internacional de Pessoa para Fim de Exploração Sexual","Estupro","Atentado Violento ao Pudor","Atentado ao Pudor Mediante Fraude","Corrupção de Menores","Ultraje Público ao Pudor (Ato/Escrito Obsceno)","Mediação para Servir a Lascívia de Outrem","Posse Sexual Mediante Fraude","Assédio Sexual","Favorecimento da Prostituição","Casa de Prostituição","Rufianismo","Tráfico Internacional de Pessoas","Tráfico Interno de Pessoas","Abandono Material","Abandono Intelectual","Entrega de Filho Menor a Pessoa Inidônea","Bigamia","Conhecimento Prévio de Impedimento","Induzimento a erro essencial e ocultação de impedimento","Simulação de Casamento","Simulação de Autoridade para Celebração de Casamento","Parto Suposto (Art. 242)","Registro de Nascimento Inexistente (Art. 241)","Sonegação do Estado de Filiação (Art. 243)","Induzimento a Fuga, Entrega Arbitrária ou Sonegação de Incapazes (Art. 248)","Subtração de Incapazes (Art. 249)","Incêndio","Explosão","Uso de gás tóxico ou asfixiante","Inundação / Perigo de Inundação","Desabamento ou desmoronamento (art. 256)","Subtração, ocultação ou inutilização de material de salvamento (art. 257)","Difusão de doença ou praga (art. 259)","Desastre ferroviário / Perigo de Desastre Ferroviário","Atentado contra a segurança de transporte público","Arremesso de projétil","Atentado contra a segurança de serviços de utilidade pública (art. 265)","Interrupção /perturbação de serviços telegráficos/telefônicos (art. 266)","Epidemia (art. 267)","Emprego de Processo Proibido/Substância não permitida","Invólucro ou recipiente com falsa indicação (art. 275)","Falsificação / Corrupção / Adulteração / Alteração de produto destinado a fins terapêuticos ou medicinais","Exercício Ilegal da Medicina, Arte Dentária ou Farmacêutica (art. 282)","Charlatanismo (Art. 283)","Corrupção ou Poluição de Água Potável (Art. 271)","Curandeirismo (Art. 284)","Envenenamento de Água Potável / Substância Alimentícia ou Medicinal (Art. 270)","Falsificação / Corrupção / Adulteração / Alteração de Substância ou Produtos Alimentícios","Infração de Medida Sanitária Preventiva (Art. 268)","Omissão de Notificação de Doença (Art. 269)","Outras Substâncias Nocivas a Saúde Pública (Art. 278)","Substância Destinada à Falsificação (Art. 277)","Fabrico / Fornecimento / Aquisição / Posse ou Transporte de Explosivos ou Gás Tóxico ou Asfixiante","Venda de Produtos ou Substância nas Condições dos Artigos 274 e 275","Medicamento em Desacordo com Receita Médica","Quadrilha ou Bando (art. 288)","Incitação ao Crime","Apologia de Crime ou Criminoso","Moeda Falsa / Assimilados","Petrechos para falsificação de moeda (art. 291)","Emissão de título ao portador sem permissão legal (art. 292)","Falsificação de papéis públicos (art. 293)","Falsificação do selo ou sinal público (art. 296)","Falsificação de documento público","Falsificação de documento particular (art. 298)","Falsidade ideológica (art. 299)","Falso reconhecimento de firma ou letra (art. 300)","Certidão ou atestado ideologicamente falso (art. 301, caput)","Falsidade material de atestado ou certidão (art. 301, § 1º)","Falsidade de atestado médico (art. 302)","Reprodução ou adulteração de selo ou peça filatélica (art. 303)","Uso de documento falso (art. 304)","Supressão de documento (art. 305)","Falsificação do sinal empregado no contraste de metal precioso ou na fiscalização alfandegária, ou para outros fins (art. 306)","Falsa identidade","Fraude de lei sobre estrangeiros","Petrechos de Falsificação de Papéis Públicos","Adulteração de Sinal Identificador de Veículo Automotor","Peculato (art. 312, caput e § 1º)","Peculato mediante erro de outrem (art. 313)","Extravio, sonegação ou inutilização de livro ou documento (art. 314)","Emprego irregular de verbas ou rendas públicas (art. 315)","Concussão (art. 316, caput)","Excesso de exação (art. 316, §§ 1º e 2º)","Corrupção passiva (art. 317)","Facilitação de contrabando ou descaminho (art. 318)","Prevaricação","Condescendência criminosa (art. 320)","Advocacia administrativa (art. 321)","Violência arbitrária (art. 322)","Abandono de função (art. 323)","Exercício funcional ilegalmente antecipado ou prolongado (art. 324)","Violação do sigilo funcional (art. 325)","Violação do sigilo de proposta de concorrência (art. 326)","Inserção de dados falsos em sistema de informações (art. 313-A)","Modificação ou alteração não autorizada de sistema de informações (art. 313-B)","Contra Patente de Invenção","Contra os Desenhos Industriais","Contra as Marcas","Cometidos por Meio de Marca, Título de Estabelecimento e Sinal de Propaganda","Contra Indicações Geográficas e Demais Indicações","De Concorrência Desleal","de Responsabilidade","Contra a Economia Popular","de Abuso de Autoridade","Tráfico de Drogas e Condutas Afins","Posse de Drogas para Consumo Pessoal","Fabricação de Objeto Destinado a Produção de Drogas e Condutas Afins","Indução, Instigação ou Auxílio ao Uso de Drogas","Oferecimento de Drogas para Consumo Conjunto","Associação para a Produção e Tráfico e Condutas Afins","Financiamento ou Custeio de Produção ou Tráfico de Drogas","Colaboração com Grupo, Organização ou Associação Destinados à Produção ou Tráfico de Drogas","Prescrição Culposa de Drogas","Condução de Embarcação ou Aeronave sob Efeito de Drogas","Responsabilidade Criminal por Danos Nucleares","Contra o Serviço Postal e o Serviço de Telegrama","Contra a Segurança Nacional, a Ordem Política e Social","Contra o Sistema Financeiro Nacional","Resultante de Preconceito de Raça ou de Cor","Contra a Ordem Tributária","Contra a Ordem Econômica","Contra as Relações de Consumo","Representação caluniosa","Contra a Fauna","Contra a Flora","Da Poluição","Agrotóxicos (Lei 7.802/89)","Atividades Nucleares (Lei 6.453/77)","Caça (Lei nº 5.197/67)","Genética (Lei nº 8.974/95)","Pesca (Lei nº 5.197/67, Lei nº 7.643/87, Lei 7.679/88, DL 221/67)","ou Ocultação de Bens, Direitos ou Valores","Contra as Telecomunicações","Previstos na Lei de Estrangeiros","De Tortura","De Trânsito","Do Sistema Nacional de Armas","De Imprensa","Estatuto da criança e do adolescente","Contra o mercado de capitais","De genocídio","Sigilo Telefônico","Da Lei de licitações","Da Lei de remoção de órgãos e tecidos","Praticados contra os índios e a cultura indígena","Recusa, retardamento ou omissão de dados técnicos indispensáveis à propositura de Ação Civil Pública","Agrários","Relacionados à Comissão Parlamentar de Inquérito","Eleitorais","De Quebra de Sigilo Financeiro","Do Código Brasileiro de Telecomunicações","Estatuto do Idoso","Parcelamento do solo urbano","Falimentares","Militares","Contra a Administração da Justiça - Lei da Ação de Alimentos","Contravenções Penais","Análogo a Crime Tentado","Análogo a Crime Culposo","Usurpação de função pública (art. 328)","Resistência (art. 329)","Tráfico de influência (art. 332)","Corrupção ativa (art. 333)","Impedimento, perturbação ou fraude de concorrência (art. 335)","Inutilização de edital ou de sinal (art. 336)","Subtração ou inutilização de livro ou documento (art. 337, caput)","Desobediência (art. 330)","Desacato (art. 331)","Contrabando ou descaminho (art. 334)","Sonegação de contribuição previdenciária (art. 337-A e Lei 8.212/91)","Corrupção Ativa em Transação Comercial Internacional","Tráfico de Influência em Transação Comercial Internacional","Reingresso de estrangeiro expulso","Denunciação caluniosa","Comunicação falsa de crime ou de contravenção","Auto-acusação falsa","Falso testemunho ou falsa perícia","Coação no curso do processo","Exercício arbitrário das próprias razões","Fraude processual","Favorecimento pessoal","Favorecimento real","Exercício arbitrário ou abuso de poder","Fuga de pessoa presa ou submetida a medida de segurança","Evasão mediante violência contra a pessoa","Arrebatamento de preso","Motim de presos","Patrocínio infiel","Patrocínio simultâneo ou tergiversação","Sonegação de papel ou objeto de valor probatório","Exploração de prestígio","Violência ou fraude em arrematação judicial","Desobediência a decisão judicial sobre perda ou suspensão de direitos","Contratação de Operação de Crédito","Inscrição de Despesas Não Empenhadas","Assunção de Obrigação no Último Ano do Mandato ou Legislatura","Ordenação de Despesa Não Autorizada","Prestação de Garantia Graciosa","Não Cancelamento de Restos a Pagar","Aumento de Despesa com Pessoal no Último Ano do Mandato ou Legislatura","Oferta Pública ou Colocação de Títulos no Mercado","Abandono Material","Abandono Intelectual","Maus Tratos","Abuso Sexual","Exploração do Trabalho Infantil","Pobreza","Uso ou Tráfico de Drogas","Adoção Internacional","Adoção Nacional","Adoção de Criança","Adoção de Adolescente","Entrada e Permanência de Menores","Viagem ao Exterior","Trabalho do adolescente","Em comum / De fato","Anistia Política","Direitos Indígenas","Intervenção em Estado / Município","Indenização por Dano Ambiental","Erro Médico","Acidente de Trânsito","Declaração de Bagagem","Declaração de Trânsito Aduaneiro","Comercialização sem Restrições de Produtos Industrializados","Comercialização sem Restrições de Gêneros Alimentícios","Comercialização e/ou Utilização sem Restrições de Medicamentos","Funcionamento de Comércio de Derivados de Petróleo","Registro de Empresa","Funcionamento de Estabelecimentos Empresariais","Registro / Porte de arma de fogo","Registro de Aeronave","Inquérito / Processo / Recurso Administrativo","Ato Lesivo ao Patrimônio Artístico, Estético, Histórico ou Turístico","Dano ao Erário","Enriquecimento ilícito","Violação aos Princípios Administrativos","Inspeção Fitossanitária","Exportação/Vedações","Inspeção Sanitária de Origem Animal","Segurança e Medicina do Trabalho","Registro de Trabalho Portuário/Órgão de Gestão de Mão-de-Obra","Registro de Marcas, Patentes ou Invenções","Multas e demais Sanções","Interdição","Apreensão","Registro de Direito Autoral","Registro de Programa de Computador","Matrícula","Financiamento Público da Educação e/ou Pesquisa","Colação de Grau","Exigência de Estágio Profissionalizante","Transferência de Estudante","Freqüência às Aulas","Penalidades Disciplinares","Colisão de Horário","Vestibular","Mensalidades","Currículo Escolar","Ingresso no Curso Superior","Jubilamento de Aluno","Criação e/ou Autorização para funcionamento de Curso Superior","Omissão de Entrega de Notas","Diplomas/Certificado de Conclusão do Curso","Financiamento Privado da Educação e/ou Pesquisa","Residência Médica","Provão - Avaliação da Educação Superior","Sistemas de cotas - Lei 10.558/2002","Pós-Graduação","Mensalidade","Matrícula","Material Didático","Exame Nacional de Ensino Médio / ENEM","Penalidades Disciplinares","Transferência","Ensino Especial","Exame Supletivo","Merenda","Transporte","Educação Pré-escolar","Educação Profissionalizante","Repasse de Verbas do SUS","Reajuste da Tabela do SUS","Convênio Médico com o SUS","Ressarcimento ao SUS","Genética / Células Tronco","Sistema Nacional do Desporto","Fornecimento de Gás","Energia Elétrica","Transporte Terrestre","Transporte Aéreo - Aeroporto","Radiodifusão","Loterias/Sorteio","Telefonia","Transporte Aquaviário","Serviço Postal","Jogos de Bingo e/ou Caça-níqueis","Água e/ou Esgoto","Recolhimento e Tratamento de Lixo","Defensoria Pública","Locação / Permissão / Concessão / Autorização / Cessão de Uso","Terreno de Marinha","Taxa de Ocupação","Foro / Laudêmio","Terras Devolutas","Águas Públicas","Bloqueio de Valores de Contas Públicas","Alienação","Ocupação","Reintegração de Posse","Taxa de Ocupação","Terreno Aldeado","Restituição de área - FUNAI","Demarcação","Recursos Minerais","Espaço Aéreo","Patrimônio Histórico / Tombamento","Ordenação da Cidade / Plano Diretor","Revogação/Concessão de Licença Ambiental","Revogação/Anulação de multa ambiental","Flora","Fauna","Transgênicos","Agrotóxicos","Unidade de Conservação da Natureza","Gestão de Florestas Públicas","Desapropriação","Desapropriação por Utilidade Pública / DL 3.365/1941","Desapropriação por Interesse Social Comum / L 4.132/1962","Desapropriação por Interesse Social para Reforma Agrária","Desapropriação Indireta","Retrocessão","Vistoria","Servidão Administrativa","Ocupação Temporária","Limitação Administrativa","Privatização","Restituição de área","Direito de Preempção","Desapropriação de Imóvel Urbano","Requisição de Bem Particular","Política de Preço Mínimo","Controle de Preços","Controle de Abastecimento","Desembaraço Aduaneiro","Regime ou Certificado de Origem MERCOSUL","Veículo de Turista Comunitário MERCOSUL","PROAGRO","Venda Casada","Preços Predatórios","Acordo de Exclusividade","Cartel","Atos de Concentração","Moeda Estrangeira","Cruzados Novos / Bloqueio","Poupança","Liberação de Conta","Juros Progressivos","Localização de Contas","Atualização de Conta","Liberação de Conta","Registro Profissional","Multas e demais Sanções","Questões Funcionais","Exame da Ordem OAB","Eleições","Anuidades OAB","Exercício Profissional","Liberação de Depósitos de Instituição Liquidanda","Normatizações","Remuneração de Ativos Retidos","Cooperativas de Crédito","Liquidação extrajudicial","Fundo de Participação dos Municípios","Seguro desemprego","Apólices da Dívida Pública","Títulos da Dívida Pública","Títulos da Dívida Agrária","Remuneração","Afastamento","Processo Disciplinar / Sindicância","Promoção","Remuneração","Afastamento","Processo Disciplinar / Sindicância","Promoção","Afastamento do Cargo","Remuneração","Recondução","Prestação de Contas","Indisponibilidade de Bens","Afastamento do Cargo","Remuneração","Recondução","Prestação de Contas","Indisponibilidade de Bens","Afastamento do Cargo","Recondução","Prestação de Contas","Indisponibilidade de Bens","Estabilidade","Enquadramento","Recondução","Acumulação de Cargos","Reintegração","Direito de Greve","Transferência","Remoção","Regime Previdenciário","Anistia Administrativa","Reversão","Redistribuição","Disponibilidade / Aproveitamento","Lotação","Promoção / Ascensão","Readaptação","Estágio Probatório","Posse e Exercício","Exoneração","Complementação de Benefício/Ferroviário","Assistência à Saúde","Assistência Pré-escolar","Auxílio-Natalidade","Auxílio-Reclusão","Auxílio-Funeral","Salário-Família","Provisória","Concessão","Restabelecimento","Invalidez Permanente","Compulsória","Voluntária","Dirigente Sindical","Licença Capacitação (Aperfeiçoamento Profissional)","Doença em Pessoa da Família","Tratamento da Própria Saúde","Gestante / Adotante / Paternidade","Interesse Particular","Serviço Militar","Atividade Política","Casamento","Luto","Amamentação","Doação de Sangue","Alistamento / Serviço Eleitoral","Acompanhamento de Cônjuge ou Companheiro","Exercício em Outro Município","Licença por Acidente em Serviço","Averbação / Contagem de Tempo Especial","Averbação / Contagem Recíproca","Demissão ou Exoneração","Advertência","Suspensão","Responsabilidade Civil do Servidor Público / Indenização ao Erário","Programa de Desligamento Voluntário (PDV)","Jornada de Trabalho","Gratificações da Lei 8.112/1990","Gratificação de Incentivo","Adicional de Periculosidade","Adicional de Fronteira","Abono Pecuniário (Art. 78 Lei 8.112/1990)","Gratificação Incorporada / Quintos e Décimos / VPNI","Descontos Indevidos","Teto Salarial","Diárias e Outras Indenizações","Plano de Classificação de Cargos","Data Base","Adicional de Horas Extras","Auxílio-Alimentação","Gratificações de Atividade","Auxílio-transporte","Revisão Geral Anual (Mora do Executivo - inciso X, art. 37, CF 1988)","Adicional de Serviço Noturno","Adicional de Produtividade","Gratificação Natalina/13º salário","Irredutibilidade de Vencimentos","Piso Salarial","Índice da URV fev/1989","Índice de 84,32% março/1990","Índice de 45% Lei 8.237/1991","Índice de 28,86% Lei 8.622/1993 e 8.627/1993","Reajuste da Lei 8.270/1991","Índice do IPC junho/1987","Índice de 10,87% Lei 10.192/2001","Índice de 47,94% Lei 8.676/1993","Inclusão de Dependente","Ingresso e Concurso","Curso de Formação","Reintegração","Estabilidade","Anistia Política","Adidos, Agregados e Adjuntos","Serviço Militar Obrigatório","Ex-combatentes","Promoção","Serviço Militar dos Profissionais da Saúde","Corpo Feminino","Gratificações e Adicionais","Férias","Indenizações Regulares","Índice de 28,86% LL 8.622/1993 e 8.627/1993","Índice da URV Lei 8.880/1994","Índice do IPC  junho/1987","Índice de 84,32% IPC março/1990","Índice da URP abril e maio/1988 DL 2.425/1988","Reforma","Licenças","Agregação","Reserva Remunerada","Transferência ex-officio para reserva","Tempo de Serviço","Transferência para reserva","Assistência Médico-Hospitalar","Licença Prêmio","Licenciamento","Concessão","Restabelecimento","Provisória","Advertência / Repreensão","Impedimento / Detenção / Prisão","Licenciamento / Exclusão","Suspensão","Reserva de Vagas para Deficientes","Inscrição / Documentação","Limite de Idade","Exigência de Prática Forense","Prova de Títulos","Exame de Saúde e/ou Aptidão Física","Curso de Formação","Exame Psicotécnico / Psiquiátrico","Anulação e Correção de Provas / Questões","Escolaridade","Classificação e/ou Preterição","Anulação","Prazo de Validade","Condições Especiais para Prestação de Prova","Modalidade / Limite / Dispensa / Inexigibilidade","Habilitação / Registro Cadastral / Julgamento / Homologação","Edital","Revogação","Sanções Administrativas","Recursos Administrativos","Convênio","Adjudicação","Ambiental","Sanitárias","Metrológica","Profissional","Segurança e/ou Medicina do Trabalho","Taxa de Ocupação / Laudêmio / Foro","Cessão de créditos não-tributários","Contribuição Sindical","Eleições Sindicais","Registro Sindical","Dilação e Extensão de Base Territorial","Filiação","Contrato Temporário de Mão de Obra L 8.745/1993","Admissão / Permanência / Despedida","Quebra de Sigilo Bancário / Fiscal / Telefônico","Criação / Instalação / Prosseguimento / Encerramento","Limites dos Poderes de Investigação","Limites do Objeto","CNH - Carteira Nacional de Habilitação","Liberação de Veículo Apreendido","Licenciamento de Veículo","Pagamento Atrasado / Correção Monetária","Anulação","Suspensão","Rescisão","Prorrogação","Termo Aditivo","Penalidades","Execução Contratual","Equilíbrio Financeiro","Erro Médico","Acidente de Trânsito","Lei de Imprensa","Direito de Imagem","Dano Ambiental","Erro Médico","Acidente de Trânsito","Lei de Imprensa","Direito de Imagem","Esbulho / Turbação / Ameaça","Imissão","Aquisição","Perda da Propriedade","Adjudicação Compulsória","Divisão e Demarcação","Reivindicação","Retificação de Área de Imóvel","Alienação Judicial","Acessão","Usucapião Especial (Constitucional)","Usucapião Extraordinária","Usucapião Ordinária","Usucapião Especial Coletiva","Direito de Vizinhança","Condomínio","Administração","Alteração de Coisa Comum","Assembléia","Despesas Condominiais","Direitos / Deveres do Condômino","Vaga de garagem","Incorporação Imobiliária","Propriedade Resolúvel","Propriedade Fiduciária","Enfiteuse","Servidão","Usufruto","Superfície","Uso","Habitação","Rural - Agrícola/Pecuário","Industrial / Mercantil","Direitos e Títulos de Crédito","Veículos","Legal","Hipoteca","Anticrese","Promessa de Compra e Venda","Subsídios","Programa de Computador","Usucapião da L 6.969/1981","Crédito Rural","Erro Médico","Acidente de Trânsito","Variação Cambial","Maus Tratos","Exposição ou Abandono de Recém Nascido","Responsabilidade Civil do Militar - Indenização ao Erário","Utilização de Embrião Humano em Desacordo com a Legislação","Prática de engenharia genética em célula germinal, zigoto ou embrião humanos","Clonagem Humana","Despenalização / Descriminalização","Livros / Jornais / Periódicos","Entidades Sem Fins Lucrativos","Partidos Políticos","Imunidade Recíproca","ICMS / Incidência Sobre o Ativo Fixo","Incidência sobre Auxílio-condução","Incidência sobre Auxílio-creche","Taxa de Limpeza Pública","Taxa de Iluminação Pública","Taxa de Coleta de Lixo","Taxa de Prevenção e Combate a Incêndio","Taxa de Licenciamento de Estabelecimento","Capacidade Tributária","Competência Tributária","Depósito Judicial","Constitucionalidade do artigo 4º da LC 118/05","Constitucionalidade do artigo 46 da Lei 8212/91","Decretação de Ofício","Arquivamento Administrativo - Crédito de Pequeno Valor","Despacho de Citação","Constitucionalidade do artigo 45 da Lei 8212/91","Exclusão - ICMS","Exclusão - IPI","Exclusão - Receitas Provenientes de Exportação","Quebra de Sigilo Bancário","PIS - Importação","COFINS - Importação","Compensação de Prejuízo","Contribuição Sindical Rural","Contribuição Sindical Rural","Aposentadoria por Invalidez Acidentária","Rural","FGTS","Acidente de Trabalho","Termo de Adesão da LC 110/2001","Rescisão / Resolução","Capitalização / Anatocismo","Limitação de Juros","Proteção da Intimidade e Sigilo de Dados","Vícios de Construção","Juros Compensatórios","Resgate de Contribuição","Inclusão de Dependente","Desconto em folha de pagamento","Remuneração Mínima","Multa","Auxílio-invalidez","Cobrança indevida de ligações","Quesitos","Impedimento","Suspeição","Condicional do processo","Parcelamento de crédito tributário","Competência do MP","Cerceamento de Defesa","Quebra do Sigilo Bancário","Quebra do Sigilo Fiscal","Quebra do Sigilo Telefônico","Trancamento","Abolitio Criminis","Princípio da Insignificância","Legítima Defesa","Estado de Necessidade","Estrito Cumprimento do Dever Legal","Inexigibilidade de Conduta Diversa","Prescrição","Perdão","Anistia","Indulto","Pagamento ou Parcelamento do Crédito Tributário","Substituição da Pena","Transação","Desaforamento","Prisão Temporária","Regime inicial","Suspensão condicional da pena","Livramento condicional","Remição","Acumulação de Proventos","Prisão Decorrente de Sentença Condenatória","Penas do Código Penal Militar","Perda da Função Pública","Perda de Bens e Valores","Inconstitucionalidade Material","Processo Legislativo","Competência dos Juizados Especiais","Competência da Justiça do Trabalho","Competência da Justiça Federal","Competência da Justiça Estadual","Honorários Advocatícios","Honorários Advocatícios em Execução Contra a Fazenda Pública","Honorários Advocatícios em FGTS","Custas","Suspeição","Impedimento","Incidência sobre Abono de Permanência","Abono de Permanência","Regularidade Formal","Fac-símile","Protocolo Integrado / Descentralizado","Obrigação de Fazer / Não Fazer","Requisição de Pequeno Valor - RPV","Quinto Constitucional","Imissão na Posse","Busca e Apreensão","Sequestro de Verbas Públicas","Fracionamento","Crédito Complementar","Aposentadoria / Pensão Especial","Fato Superveniente ao Término do Prazo para Impugnação","Juros","Correção Monetária","Multa Cominatória / Astreintes","Taxa SELIC","Correção da Tabela","IPMF - Imposto Provisório sobre Movimentação Financeira","Questão Prejudicial","Tribunal de Contas","Cargo em Comissão","Professor","Procuradores de Órgãos / Entidades Públicos","Serventuários da Justiça","Extensão de Vantagem aos Inativos","Contagem em Dobro","Pagamento em Pecúnia","Índice de 3,17%","Índice de 11,98%","Subteto Salarial","Gratificações Municipais Específicas","Gratificação de Encargos Especiais - GEE","Gratificação de Estímulo à Produção Individual - GEPI","Gratificação Estadual - AM","Adicional de Etapa Alimentar","Gratificação de Suporte às Atividades Escolares - GSAE","Gratificação por Trabalho Educacional - GTE","Gratificação de Suporte Administrativo - GASA","Gratificação-Prêmio","Gratificação Complementar de Vencimento","Gratificação de Desempenho de Atividade Técnico Administrativa - GDATA","Gratificação de Desempenho de Atividade Tributária - GDAT","Gratificação de Desempenho de Atividade de Fiscalização Agropecuária - GDAFA","Gratificação de Desempenho de Atividade Jurídica - GDAJ","Gratificação de Atividade - GATA","Gratificação de Incentivo à Docência - GID","Gratificação de Produção Suplementar - GPS","Gratificação Eleitoral","Gratificação de Inatividade","Gratificação Extraordinária - GE","Gratificação de Desempenho de Função - GADF","Adicional de Desempenho","Adicional de Sexta-Parte","Verba de Representação","Extensão de Vantagem aos Inativos","Nulidade - Não Observância da Reserva de Plenário","Ausência de Interesse Processual","Ausência de Pressupostos de Constituição e Desenvolvimento","Sobrestamento","Criação do Partido Político","Registro do Partido Político","Extinção do Partido Político","Filiação partidária","Convenção para escolha de candidatos","Inscrição fraudulenta","Calúnia","Difamação","Injúria","Captação ilícita de votos ou corrupção eleitoral","Aliciamento eleitoral","Falso eleitoral","Desobediência à determinações da Justiça Eleitoral","Boca de urna","Indução à inscrição indevida","Fidelidade partidária","Plebiscito","Referendo popular","Sistema Majoritário / Proporcional","fixação","aumento","redução","Domicílio eleitoral","Isenção do dever eleitoral","Justificação de ausência de voto","Cancelamento da inscrição de eleitor","Desincompatibilização do candidato","Inelegibilidade","Reeleição","Cassação do registro da candidatura","Anistia","Propaganda eleitoral","Direito de resposta","Prestação de contas","Pesquisa eleitoral","Abuso do poder econômico","Abuso do poder político (uso indevido dos meios de comunicação)","Uso de bem público","Sistema eletrônico de votação","Sigilo na votação","Apuração/recontagem de votos","Nulidade da votação","Quociente eleitoral","Transporte em dia de eleição","Boca de urna","Cassação do diploma","Sustação da diplomação","Prestação de contas","Declaração de inelegibilidade","Impugnação de mandato eletivo","Perda do mandato","Recondução","Suspensão de mandato","Convocação de suplente","Renúncia","Afastamento do cargo","Cassação de mandato","Por cumulação de cargos","Por condenação criminal","Por suspensão dos direitos políticos","Desordem eleitoral","Falsificação de documento público para fins eleitorais","Uso de documento falso","Obtenção de documento falso","Aliciamento de eleitores","Divulgação de pesquisa de fraudulenta","Impedimento à fiscalização das pesquisas","Publicação de pesquisa irregular","Uso de símbolos governamentais em propaganda eleitoral","Coação eleitoral","Aliciamento violento de eleitores","Aliciamento comercial de eleitores","Ocultação de urna","Motim","Pederastia ou outro ato de libidinagem","Estelionato","Incêndio","Peculato","Concussão","Corrupção ativa","Prevaricação","Deserção","Lesão levíssima","Difamação","Injúria","Homicídio simples","Homicídio qualificado","Leve","Grave","Lesões qualificadas pelo resultado","Roubo simples","Roubo qualificado","Latrocínio","Auditores Fiscais","Comissão de Permanência","Prescrição por Médico não vinculado ao SUS","Gratificação de Desempenho de Atividade de Seguridade Social e do Trabalho - GDASST","Gratificação de Desempenho de Atividade de Ciência e Tecnologia - GDACT","Alimentos","Depositário Infiel","Alienação Fiduciária","Proibição de Privilégio Fiscal às Empresas Públicas e Sociedades de Economia Mista","Ausência de Legitimidade para propositura de Ação Civil Pública","Crimes Hediondos","Cerceamento de Defesa","Ausência de Publicidade","Ausência de Fundamentação","Criação / Extinção / Reestruturação de Orgãos ou Cargos Públicos","Parcela Incontroversa","Cancelamento / Duplicidade de CPF","Não Cumulatividade","Nao Cumulatividade","Não Cumulatividade","Não Cumulatividade","Base de Cálculo","Policiais Civis","Limite de Renda Familiar","Cumprimento Provisório de Sentença","Nepotismo","Fruição / Gozo","Indenização / Terço Constitucional","Liquidação Parcelada","Nulidade do Decreto que autoriza a desapropriação","Ausência de Cobrança Administrativa Prévia","Inscrição Indevida no CADIN","Efeitos da Declaração de Inconstitucionalidade","Vício Formal do Julgamento","Habeas Corpus - Cabimento","Medicamento / Tratamento / Cirurgia de Eficácia não comprovada","Base de Cálculo","Abuso de Poder","Competência dos Juizados Especiais","Competência da Justiça Federal","Competência da Justiça Estadual","Competência por Prerrogativa de Função","Competência por Prerrogativa de Função","Excesso de prazo para instrução / julgamento","Incidência sobre desconto incondicional","Prisão Domiciliar / Especial","Passe livre em transporte","Regressão de Regime","Transferência de Preso","Execução Penal Provisória - Cabimento","Imunidade em razão de função, ofício ou profissão","Transferência de Unidade","Indisponibilidade / Seqüestro de Bens","Busca e Apreensão de Bens","Aborto provocado pela gestante ou com o seu consentimento","Aborto provocado por terceiro","Aborto qualificado","Restituição de Criança, Convenção de Haia 1980","Cobrança de Multa Moratória de Massa Falida","Cobrança de Juros Moratórios de Massa Falida","Depósito Elisivo","Prova Ilícita","Competência do Órgão Fiscalizador","Criação","Desmembramento","Desclassificação","Guarda com genitor ou responsável no exterior","Citação","Intimação / Notificação","Depoimento","Viagem Nacional","Expurgos Inflacionários / Planos Econômicos","Correção Monetária de Diferenças Pagas em Atraso","Princípio da Seletividade","Violência Doméstica Contra a Mulher","Violência Doméstica Contra a Mulher","Crime / Contravenção contra Criança / Adolescente","Crime / Contravenção contra Idoso","Fato Atípico","Criação de Dotação Orçamentária","Repasse de Duodécimos","Repasse de Verbas Públicas","Crédito Suplementar","Exclusão - Receitas Transferidas a outras Pessoas Jurídicas","Lavagem ou Ocultação de Bens, Direitos ou Valores Oriundos de Corrupção","Lavagem ou Ocultação de Bens, Direitos ou Valores Oriundos de Corrupção","Lavagem ou Ocultação de Bens, Direitos ou Valores Oriundos de Corrupção","Crimes contra a Ordem Tributária praticado por Funcionário Público","Crime contra a administração ambiental","Tráfico Ilícito de Drogas praticado por Funcionário Público","Má -gestão praticada por Prefeitos e Vereadores","Má-gestão praticada por Prefeitos e Vereadores","Corrupção Praticada por Prefeitos e Vereadores","Adulteração de Sinal Identificador de Veículo Automotor por Funcionário Público","Petrechos de Falsificação de Papéis Públicos por Funcionário Público","Petrechos para falsificação de moeda por Funcionário Público","Falsidade ideológica praticada por Funcionário Público","Falsificação de documento público praticada por Funcionário Público","Falsificação de documento Público praticada por Funcionário Público","Conversão em Agravo Retido","Cobrança de Aluguéis - Sem despejo","Consignação de Chaves","Falsificação do Selo ou Sinal Público Praticado por Funcionário Público","Falsificação de Papéis Públicos Praticado por Funcionário Público","Crime de Desvio e/ou Circulação de Moeda Não Autorizada","Crime de Moeda Falsa Praticado por Funcionário Público","Competência da Justiça Militar da União","Competência da Justiça Militar dos Estados","Menagem","Competência da Justiça Militar da União","Competência da Justiça Militar Estadual","Cassação de licença para direção de veículo motorizado","Confisco","Exílio local","Interdição de estabecimento  ou associação","Proibição de frequentar determinados locais","Suspensão do exercício do posto, graduação, cargo ou função","Representação do Corregedor","Requerimento da Parte","Irregularidade na composição do Conselho","Crime Culposo","Crime em tempo de guerra","Crime Tentado","Livramento Condicional","Suspensão Condicional da Pena","Abolitio criminis","Princípio da Insignificância","Exclusão das Forças Armadas","Inabilitação para o exercício de Função Pública","Incompatibilidade para o Oficialato","Indignidade para o Oficialato","Perda da Função Pública","Perda da Graduação das Praças","Perda do Posto e da Patente","Suspensão do pátrio poder, tutela ou curatela","Suspensão dos direitos políticos","Anistia","Indulto","Morte do Agente","Prescrição","Reabilitação","Ressarcimento do Dano","Estado de Necessidade","Estado de Necessidade exculpante","Estrito Cumprimento do Dever Legal","Exercício Regular do Direito","Inexigibilidade de Conduta Diversa","Legítima Defesa","Perda em favor da Fazenda Pública","Reparação do Dano","Conversão da Pena","Substituição da Pena","Criação ou simulação de incapacidade física","Favorecimento a convocado","Insubmissão","Insubmissão privilegiada","Substituição de convocado","Exercício de comércio por oficial","Concerto para deserção","Deserção","Deserção - casos assimilados","Deserção especial","Deserção por evasão ou fuga","Deserção privilegiada","Deserção qualificada","Favorecimento a desertor","Omissão de Oficial","Abandono de posto","Descumprimento de missão","Dormir em serviço","Embriaguez em serviço","Omissão de eficiência da força","Omissão de providências para evitar danos","Omissão de providências para salvar comandados","Omissão de socorro","Retenção indevida","Alteração de Limites","Aposição, supressão ou alteração de marca","Usura pecuniária","Chantagem","Extorsão","Extorsão indireta","Extorsão mediante sequestro","Latrocínio","Roubo","Roubo qualificado","Receptação","Furto","Furto de uso","Furto qualificado","Abuso de pessoa","Estelionato","Dano atenuado","Dano em aparelhos e instalações","Dano em navio de guerra ou mercante em serviço militar","Dano material ou aparelhamento de guerra","Dano qualificado","Dano simples","Desaparecimento,consunção ou extravio","Apropriação de coisa achada","Apropriação de coisa havida acidentalmente","Apropriação indébita","Abuso de radiação","Desabamento ou desmoronamento","Difusão de epizootia ou praga vegetal","Embriaguez ao volante","Emprego de gás tóxico ou asfixiante","Explosão","Fatos que expõem a perigo aparelhamento militar","Fuga após acidente de trânsito","Incêndio","Inundação","Perigo de inundação","Perigo resultante de violação de regra de trânsito","Subtração, ocultação ou inutilização de material de socorro","Arremesso de projétil","Atentado contra serviço de utilidade militar","Atentado contra transporte","Atentado contra viatura ou outro meio de transporte","Interrupção ou perturbação de serviço ou meio de comunicação","Perigo de desastre ferroviário","Corrupção ou poluição de água potável","Envenenamento com perigo extensivo","Epidemia","Fornecimento de substância alterada","Fornecimento de substancia nociva","Omissão de notificação de doença","Receita Ilegal","Tráfico, posse ou uso de entorpecente ou substância de efeito similar","Atos de jurisdição indevida","Consecução de notícia, informação ou documento para fim de espionagem","Desenho ou levantamento de plano ou planta de local militar","Entendimento para empenhar o Brasil à neutralidade ou à guerra","Entendimento para gerar conflito ou divergência com o Brasil","Hostilidade contra país estrangeiro","Hostilidade qualificada pelo resultado","Penetração com o fim de espionagem","Provocação a país estrangeiro","Revelação de notícia, informação ou documento","Revelação de notícia, informação ou documento, com o fim de espionagem militar","Sobrevôo em local interdito","Tentativa contra a soberania do Brasil","Turbação de objeto ou documento","Violação de território estrangeiro","Ato obsceno","Escrito ou objeto obsceno","Abandono de pessoa","Abandono de pessoa seguida de morte ou lesão grave","Maus tratos","Maus tratos seguido de morte ou lesão grave","Lesão grave","Lesão leve","Lesão levíssima","Lesão privilegiada","Lesão seguida de morte","Participação em rixa","Homicídio","Homicidio qualificado","Homicido privilegiado","Provocação direta ou auxílio a suicídio","Provocação indireta ao suicídio","Genocídio","Atentado violento ao pudor","Corrupção de menores","Estupro","Ato Libidinoso","Ameaça","Constragimento ilegal","Desafio para duelo","Divulgação de segredo","Sequestro ou cárcere privado","Violação de correspondência","Violação de domicílio","Violação de recato","Violação de segredo profissional","Calúnia","Difamação","Injúria","Injuria real","Ofensa às forças armadas","Violência contra militar de serviço","Violência contra superior","Abuso de requisição militar","Assunção de comando sem ordem ou autorização","Conservação ilegal de comando","Ofensa aviltante a inferior","Operação militar sem ordem superior","Ordem arbitrária de invasão","Rigor excessivo","Uso indevido de uniforme, distintivo ou insígnia militar por qualquer pessoa","Uso indevido por militar de uniforme, distintivo ou insígnia","Violência contra inferior","Resistência mediante ameaça ou violência","Conspiração","Motim","Omissão de lealdade militar","Organização de grupo para prática de violência","Revolta","Oposição a ordem de sentinela","Publicação ou crítica indevida","Recusa a obediência","Reunião ilícita","Amotinamento","Arrebatamento de preso ou internado","Evasão de preso ou internado","Fuga de preso ou internado","Aliciação para motim ou revolta","Apologia de fato criminoso ou do seu autor","Incitamento","Despojamento desprezível","Desrespeito a comandante, oficial-general ou oficial de serviço","Desrespeito a símbolo nacional","Desrespeito a superior","Peculato","Peculato mediante aproveitamento ou erro de outrem","Peculato-furto","Certidão ou atestado ideologicamente falso","Cheques sem fundos","Falsa identidade","Falsidade ideológica","Falsificação de documento","Supressão de documento","Uso de documento falso","Uso de documento pessoal alheio","Desacato a assemelhado ou funcionário","Desacato a militar","Desacato a superior","Desobediência","Ingresso clandestino","Impedimento, perturbação ou fraude de concorrencia","Inutilização de edital ou de sinal oficial","Subtração ou inutilização de livro, processo ou documento","Tráfico de influência","Usurpação de função","Abandono de cargo","Abuso de confiança ou boa fé","Aplicação ilegal de verba ou dinheiro","Condescendência criminosa","Exercício funcional ilegal","Extravio, sonegação ou inutilização de livro ou documento","Inobservancia de lei, regulamento ou instrução","Não inclusão de nome em lista","Obstáculo à hasta pública, concorrência ou tomada de preços","Patrocínio indébito","Prevaricação","Violação de sigilo de proposta de concorrência","Violação de sigilo funcional","Violação do dever funcional com o fim de lucro","Violação ou divulgação indevida de correspondência ou comunicação","Violência arbitrária","Corrupção ativa","Corrupção passiva","Participação ilícita","Concussão","Desvio","Excesso de exação","Auto-acusação falsa","Coação","Comunicação falsa de crime","Corrupção ativa de testemunha, perito ou intérprete","Denunciação caluniosa","Desacato","Desobediência a decisão judicial","Desobediência a decisão sobre perda ou suspensão de atividade ou direito","Exploração de prestígio","Falso testemunho ou falsa perícia","Favorecimento pessoal","Favorecimento real","Inutilização,sonegação ou descaminho de material probante","Publicidade opressiva","Recusa de função na Justiça Militar","Advertência","Internação com atividades externas","Internação sem atividades externas","Liberdade assistida","Obrigação de reparar o dano","Prestação de serviços à comunidade","Semiliberdade","Crimes contra portadores de deficiência","Conflito fundiário coletivo rural","Conflito fundiário coletivo urbano","Crime/contravenção decorrente de conflito fundiário coletivo","Violação sexual mediante fraude","Estupro de vulnerável","Satisfação de lascívia mediante presença de criança ou adolescente","Favorecimento da prostituição ou outra forma de exploração sexual de vulnerável","Favorecimento da prostituição ou outra forma de exploração sexual","Tráfico internacional de pessoa para fim de exploração sexual","Tráfico interno de pessoa para fim de exploração sexual","Intervenção Indevida de Autoridade Junto à Mesa Receptora","Não Observância de Chamamento de Eleitores para Votar","Práticas Irregulares que Determinam a Anulação da Votação","Votação de Eleitor na Seção Eleitoral na qual Não Está Inscrito ou Autorizado a Votar","Contagem de Votos de Eleitores em Relação aos quais Houve Impugnação","Fabricação, Aquisição, Fornecimento, Subtração ou Guarda de Materiais de Uso da Justiça Eleitoral","Inscrição Fraudulenta","Induzimento à Inscrição de Eleitor em Infração às Normas Legais","Inscrição Fraudulenta Efetivada pelo Juiz","Negativa ou Retardamento de Inscrição sem Fundamento Legal","Perturbação ou Impedimento de Alistamento","Promoção de Desordem nos Trabalhos Eleitorais","Majoração de Preços de Utilidade e Serviços Necessários à Realização das Eleições","Ocultação, Sonegação ou Recusa de Fornecimento de Utilidades, Alimentos e Transporte no Dia da Eleição","Retardamento ou Não-Publicação de Decisões, Citações ou Intimações da Justiça Eleitoral","Não-Apresentação de Denúncia ou Não Execução de Sentença Penal Condenatória no Prazo Legal","Não Cumprimento, pela Autoridade Judiciária, do Dever de Representar Contra o Órgão do Ministério que Não Oferece Denúncia no Prazo Legal","Recusa ou Abandono do Serviço Eleitoral","Não Cumprimento dos Deveres Impostos pelo Código Eleitoral à Autoridade Judiciária e aos Funcionários da Justiça Eleitoral","Utilização de Prédios ou Serviços de Repartições Públicas para Beneficiar Partido ou Organização de Caráter Político","Desobediência a Ordens ou Instruções da Justiça Eleitoral","Descumprimento do Dever de Informar o Número, Espécie e Lotação de Veículos e Embarcações","Desatendimento à Requisição de Veículo ou Embarcação","Descumprimento da Proibição de Fornecimento de Transporte ou Refeições a Eleitores","Estupro de Vulnerável","Favorecimento de Prostituição ou Outra Forma de Exploração Sexual","Favorecimento de Prostituição ou Outra Forma de Exploração Sexual de Vulnerável","Satisfação de Lascívia Mediante Presença de Criança ou Adolescente","Tráfico Interno de Pessoa para Fim de Exploração Sexual","Violação Sexual Mediante Fraude","Obstamento do Transporte e Fornecimento de Alimentos a Eleitores pela Justiça Eleitoral","Utilização de Veículos e Embarcações Públicas em Campanha Eleitoral","Dano a Equipamento Eleitoral","Omissão na Expedição de Boletim de Apuração","Omissão no Recolhimento das Cédulas Apuradas, no Fechamento e Lacração da Urna","Alteração do Resultado de Votação nos Mapas ou Boletins de Apuração","Omissão no Recebimento e Registro de Protestos, bem como, de sua Remessa à Instância Superior","Falsificação ou Alteração de Documento Público para Fins Eleitorais","Falsidade Ideológica","Reconhecimento Indevido de Firma ou Letras para Fins Eleitorais","Uso de Documento Falso para Fins Eleitorais","Obtenção de Documento Falso para Fins Eleitorais","Omissão da Entrega do Boletim de Urna","Obtenção de Acesso a Sistema de Dados Eleitorais","Destruição de Dados Eleitorais","Cerceamento da Fiscalização Partidária","Recusa de Entrega de Boletim","Divulgação de Fatos Inverídicos na Propaganda Eleitoral","Calúnia na Propaganda Eleitoral","Divulgação de Falsa Imputação","Difamação na Propaganda Eleitoral","Injúria na Propaganda Eleitoral","Injúria Eleitoral Violenta","Inutilização, Alteração ou Perturbação de Propaganda","Impedimento do Exercício da Propaganda","Utilização de Organização Comercial para Propaganda ou Aliciamento de Eleitores","Realização de Propaganda Eleitoral em Língua Estrangeira","Participação de Pessoa Não Detentora de Direitos Políticos em Atividades Partidárias e de Propaganda Eleitoral","Autorização de Transmissão de Pessoa Não Detentora de Direitos Políticos em Atividades Partidárias e de Propaganda Eleitoral","Pesquisa Fraudulenta","Uso, na Propaganda Eleitoral, de Símbolo de Órgãos de Governo","Retenção de Título Eleitoral ou do Comprovante de Alistamento Eleitoral","Impedimento ou Embaraço ao Exercício do Sufrágio","Prisão ou Detenção do Eleitor, Membro de Mesa Receptora, Fiscal, Delegado de Partido ou Candidato, Fora das Hipóteses Legais Permitidas","Corrupção Eleitoral","Coação Visando a Obtenção de Voto ou a sua Abstenção","Violência ou Grave Ameaça Visando a Obtenção de Voto ou a sua Abstenção","Concentração de Eleitores para Embaraçar ou Fraudar o Exercício do Voto","Fornecimento ao Eleitor de Cédula Oficial Já Assinalada ou Marcada","Fornecimento de Cédula Eleitoral Rubricada a Destempo","Votação Múltipla ou Realizada em Lugar de Outrem","Violação do Sigilo do Voto","Violação do Sigilo da Urna ou dos Invólucros","Destruição, Supressão ou Ocultação de Urna Contendo Votos ou Documentos Eleitorais","Destruição de Relação de Candidatos","Uso Proibido de Alto-Falantes","Arregimentação de Eleitor ou Boca de Urna","Divulgação de Propaganda Eleitoral no Dia da Eleição","Retenção de Título ou Comprovante de Alistamento Eleitoral","Subscrição de Mais de Uma Ficha de Filiação Partidária","Subscrição de Mais de Uma Ficha de Filiação Partidária","Inscrição Simultânea em Dois ou Mais Partidos","Coleta de Assinatura em Mais de Uma Ficha de Registro Partidário","Preterição à Prioridade Postal","Arguição de Ineligibilidade Temerária ou de Má-Fé","Não-Acesso dos Partidos aos Dados Relativos às Pesquisas Eleitorais","Irregularidades dos Dados Publicados em Pesquisas Eleitorais","Conduta Vedada a Agente Público","Improbidade Administrativa","Descumprimento de Prazos Eleitorais","Garantia de Autoridade de Decisão do Tribunal","Afastamento de Magistrado","Designação de Juiz Eleitoral","Lista Tríplice","Concurso Público","Remoção de Servidor","Requisição de Servidor","Requisição de Servidor - Renovação de Requisição de Servidor","Requisição de Força Federal","Alistamento Eleitoral - Cancelamento","Alistamento Eleitoral - Domicílio Eleitoral","Alistamento Eleitoral - Duplicidade/Pluralidade","Alistamento Eleitoral - Exclusão","Alistamento Eleitoral - Inscrição Eleitoral","Impugnação - Inscrição Eleitoral","Impugnação - Transferência de Domicílio Eleitoral","Condição de Elegibilidade - Alistamento Eleitoral","Condição de Elegibilidade - Domicílio Eleitoral na Circunscrição","Condição de Elegibilidade - Idade Mínima","Condição de Elegibilidade - Filiação Partidária","Condição de Elegibilidade - Nacionalidade Brasileira","Condição de Elegibilidade - Pleno Exercício dos Direitos Políticos","Condição de Elegibilidade - Quitação Eleitoral","Direito de Resposta","Identificação Numérica de Candidato","Inelegibilidade - Abuso do Poder Econômico ou Político","Inelegibilidade - Analfabetismo","Inelegibilidade - Condenação Criminal por órgão colegiado ou Transitada em Julgado","Inelegibilidade - Declaração de Indignidade ou de Incompatibilidade com o Oficialato","Inelegibilidade - Desincompatibilização","Inelegibilidade - Inalistável","Inelegibilidade - Parentesco","Inelegibilidade - Reeleição","Inelegibilidade - Rejeição de Contas Públicas","Inelegibilidade - Representação ou Ação de Investigação Judicial Eleitoral Jugada Procedente pela Justiça Eleitoral","Inelegibilidade - Terceiro Mandato","Inelegibilidade - Vida Pregressa","Inelegibilidade - Vínculo Afetivo","Inelegibilidade - Vínculo Conjugal","Inelegibilidade - Vínculo em União Estável","Nome do Candidato - Homonímia","Nome do Candidato - Variação Nominal","Prestação de Contas - De Candidato","Impugnação ao Registro de Candidatura","Registro de Candidatura - Preenchimento de Vaga Remanescente","Registro de Candidatura - RRC - Candidato","Registro de Candidatura - Substituição de Candidato - Por Cancelamento de Registro","Registro de Candidatura - Substituição de Candidato - Por Cassação de Registro","Registro de Candidatura - Substituição de Candidato - Por Expulsão Partidária","Registro de Candidatura - Substituição de Candidato - Por Falecimento","Registro de Candidatura - Substituição de Candidato - Por Indeferimento de Registro","Registro de Candidatura - Substituição de Candidato - Por Ineligibilidade","Registro de Candidatura - Substituição de Candidato - Por Renúncia","Cargo - Deputado Distrital","Cargo - Deputado Estadual","Cargo - Deputado Federal","Cargo - Governador","Cargo - Prefeito","Cargo - Presidente da República","Cargo - Primeiro Suplente de Senador","Cargo - Segundo Suplente de Senador","Cargo - Senador","Cargo - Vereador","Cargo - Vice-Governador","Cargo - Vice-Prefeito","Cargo - Vice-Presidente da República","Eleições - 1° Turno","Eleições - 2° Turno","Eleições - Anulação de Eleições","Eleições - Eleição Majoritária","Eleições - Eleição Proporcional","Eleições - Eleição Suplementar","Pesquisa Eleitoral - Divulgação de Pesquisa Eleitoral Fraudulenta","Pesquisa Eleitoral - Divulgação de Pesquisa Eleitoral Sem Prévio Registro","Pesquisa Eleitoral - Registro de Pesquisa Eleitoral","Propaganda Política - Propaganda Eleitoral - Adesivo","Propaganda Política - Propaganda Eleitoral - Alto-falante/Amplificador de Som","Propaganda Política - Propaganda Eleitoral - Banner/Cartaz/Faixa","Propaganda Política - Propaganda Eleitoral - Bem Particular","Propaganda Política - Propaganda Eleitoral - Bem Particular de Uso Comum","Propaganda Política - Propaganda Eleitoral - Bem Público","Propaganda Política - Propaganda Eleitoral - Bem Tombado","Propaganda Política - Propaganda Eleitoral - Boca de Urna","Propaganda Política - Propaganda Eleitoral - Cavalete","Propaganda Política - Propaganda Eleitoral - Comício/Showmício","Propaganda Política - Propaganda Eleitoral - Conduta Vedada a Emissora de Rádio/Televisão na Programação Normal","Propaganda Política - Propaganda Eleitoral - Confecção, Utilização ou Distribuição de Brinde","Propaganda Política - Propaganda Eleitoral - Contrariedade à Lei de Postura Municipal","Propaganda Política - Propaganda Eleitoral - Debate Político","Propaganda Política - Propaganda Eleitoral - Extemporânea/Antecipada","Propaganda Política - Propaganda Eleitoral - Folhetos/Volantes/Santinhos/Impressos","Propaganda Política - Propaganda Eleitoral - Distribuição de Tempo de Propaganda","Propaganda Política - Propaganda Eleitoral - Horário Eleitoral Gratuito/Inserções de Propaganda","Propaganda Política - Propaganda Eleitoral - Horário Eleitoral Gratuito/Programa em Bloco","Propaganda Política - Propaganda Eleitoral - Rádio","Propaganda Política - Propaganda Eleitoral - Rádio Comunitária","Propaganda Política - Propaganda Eleitoral - Televisão","Propaganda Política - Propaganda Eleitoral - Invasão de Horário Destinado a Outro Cargo/Partido/Coligação","Propaganda Política - Propaganda Eleitoral - Imprensa Escrita - Jornal/Revista/Tabloide","Propaganda Política - Propaganda Eleitoral - Inobservância do Limite Legal","Propaganda Política - Propaganda Eleitoral - Internet","Propaganda Política - Propaganda Eleitoral - Omissão de Informações Obrigatórias","Propaganda Política - Propaganda Eleitoral - Outdoors","Propaganda Política - Propaganda Eleitoral - Pintura em Muro","Propaganda Política - Propaganda Eleitoral - Truncagem, Montagem, Utilização de Gravação Externa, Computação Gráfica, Desenho Animado ou Efeito Especial","Captação ou Gasto Ilícito de Recursos Financeiros de Campanha Eleitoral","Prestação de Contas - De Comitê Financeiro","Registro de Comitê Financeiro - Eleição Majoritária","Registro de Comitê Financeiro - Eleição Proporcional","Registro de Comitê Financeiro - Único","Contas - Aprovação das Contas","Contas - Aprovação das Contas com Ressalvas","Contas - Desaprovação/Rejeição das Contas","Contas - Não Apresentação das Contas","Doação de Recursos Acima do Limite Legal - Pessoa Física","Doação de Recursos Acima do Limite Legal - Pessoa Jurídica","Habeas Corpus - Liberatório","Habeas Corpus - Preventivo","Impugnação - Identidade de Eleitor Admitido a Votar","Impugnação - Nomeação de Membro da Mesa Receptora","Impugnação - Nomeação de Membro da Junta Eleitoral","Impugnação - Sistema Eletrônico de Votação","Impugnação - Voto por Cédula","Registro de Candidatura - DRAP - Partido/Coligação","Apuração/Totalização de Votos","Quociente Eleitoral/Partidário","Abuso - De Poder Econômico","Abuso - De Poder Político/Autoridade","Abuso - Uso Indevido de Meio de Comunicação Social","Captação Ilícita de Sufrágio","Corrupção ou Fraude","Propaganda Política - Propaganda Institucional","Diplomação","Direitos Políticos - Perda dos Direitos Políticos","Direitos Políticos - Restabelecimento dos Direitos Políticos","Direitos Políticos - Suspensão de Direitos Políticos","Execução - De Astreintes","Execução - De Multa Eleitoral","Execução de Julgado","Cautelar Inominada - De Busca e Apreensão","Cautelar Inominada - De Produção Antecipada de Provas","Cautelar Inominada - Incidental","Cautelar Inominada - Preparatória","Exceção - De Coisa Julgada","Exceção - De Ilegitimidade de Parte","Exceção - De Impedimento","Exceção - De Incompetência","Exceção - De Litispendência","Exceção - De Pré-Executividade","Exceção - De Suspeição","Alteração Estatutária de Partido Político em Matéria Administrativa","Coligação Partidária - Majoritária","Coligação Partidária - Majoritária e Proporcional","Coligação Partidária - Proporcional","Convenção Partidária","Filiação Partidária - Cancelamento","Filiação Partidária - Duplicidade/Pluralidade","Filiação Partidária - Nulidade","Justificação de Desfiliação Partidária","Perda de Cargo Eletivo por Desfiliação Partidária","Fusão de Partidos Políticos","Incorporação de Partidos Políticos","Dissolução de Órgão de Direção Partidária","Partido Político - Órgão de Direção Estadual","Partido Político - Órgão de Direção Municipal","Partido Político - Órgão de Direção Nacional","Partido Político - Comissão Provisória","Propaganda Política - Propaganda Intrapartidária","Propaganda Política - Propaganda Partidária","Veiculação de Propaganda Partidária - Em Bloco","Veiculação de Propaganda Partidária - Em Inserções","Plebiscito","Requerimento","Contra o Ordenamento Urbano e o Patrimônio Cultural","Crimes contra o Ordenamento Urbano e o Patrimônio Cultural","Intimação","Citação","Oitiva","Diligências","Atos executórios","Entrada de aparelho telefônico de comunicação móvel em estabelecimento prisional","Programas de Arrendamento Residencial  PAR","Tabela Price","Empréstimo consignado","Tarifas","Capitalização e Previdência Privada","Transporte Aquaviário","Dever de Informação","Práticas Abusivas","Oferta e Publicidade","Transporte Ferroviário","Transporte Rodoviário","Infrações administrativas","Fundos","Classificação indicativa","Entidades de atendimento","Conselhos tutelares","Mineração","Reserva legal","Recursos Hídricos","Poluição","Zoneamento Ecológico e Econômico","Zona Costeira","Área de Preservação Permanente","Produtos Controlados / Perigosos","Patrimônio Cultural","Controle Externo da atividade policial","Conselho do Idoso","Conselho da Comunidade","Conselho sobre Drogas","Conselho de Direitos da Criança e Adolescente","Parcelamento do Solo","Comércio Ambulante","Operações Urbanas Consorciadas","Posturas Municipais","Segurança em Edificações","Política Agrícola","Pessoa Idosa","Pessoas com deficiência","Minorias Étnicas","Comunicação Social","Moradia","Assistência Social","Alimentação","Não Discriminação","Terceirização do SUS","Financiamento do SUS","Vigilância Sanitária e Epidemiológica","Controle Social e Conselhos de Saúde","Hospitais e Outras Unidades de Saúde","Tecidos e Partes do Corpo Humano","Recebimento de bolsa de estudos","Produto Impróprio","Combustíveis e derivados","Saneamento","Responsabilidade Fiscal","Irregularidade no atendimento","Vendas casadas","Jogos / Sorteios / Promoções comerciais","Produto Impróprio","Combustíveis e derivados","Saneamento","Utilização de bens públicos","Agências/órgãos de regulação","Sistema de proteção especial a vítimas e a testemunhas","Política fundiária e da reforma agrária","Responsabilidade fiscal","Fornecimento de Medicamentos","Unidade de terapia intensiva (UTI) ou unidade de cuidados intensivos (UCI)","Crimes Previstos no Estatuto do Torcedor","Previstos no Estatuto do Torcedor","Crédito Especial","Crédito Extraordinário","Resolução","Recomendação","Resolução Conjunta","Concurso para magistrado","Prova Objetiva","Prova Subjetiva","Prova Prática-Sentença","Prova Oral","Reserva de Vagas","Concurso para servidor","Concurso para serventia extrajudicial","Juiz Leigo","Composição de Turmas Recursais","Licença","Participação Societária","Plantão Judiciário","Residência","Substituição/Convocação em Tribunal","Violação Prerrogativa Advogado","Cargos de Direção","Órgão Especial","Fraude / Quebra de ordem cronológica","Pagamento","Vacância","Remoção","Desmembramento","Cumulação","Emolumentos","Fraude","Gratuidade","Princípio da Territorialidade","Provimento Irregular","Registro Civil de Nascimento","Registro de Imóveis","Cessão","Desvio de Função","Provimento de Cargos","Entrada de aparelho telefônico de comunicação móvel em estabelecimento prisional","Inclusão do 13º salário (gratificação natalina) no PBC","Art. 26 da Lei 8.870/1994","Art. 29, § 5º, da Lei 8.213/1991","Art. 29, II, da Lei 8.213/1991","Alteração do teto máximo para o valor do benefício previdenciário do RGPS (EC 20 e 41)","Revisão do valor do benefício no primeiro reajuste após a concessão (Art. 21, § 3º, da Lei 8.880/1994)","Deficiente","Idoso","Inspeção","Revisão/Desconstituição de Ato Administrativo","Morosidade no Julgamento do Processo","Investigação Patrimonial","Apuração de Infração Disciplinar","Acidente de Trabalho - Ressarcimento ao Erário","Difusão culposa de doença ou praga","Receptação culposa","Receptação culposa","Incêndio culposo","Incêndio culposo","Explosão culposa","Explosão culposa","Uso culposo de gás tóxico ou asfixiante","Uso culposo de gás tóxico ou asfixiante","Inundação culposa","Inundação culposa","Desabamento ou desmoronamento culposo","Desabamento ou desmoronamento culposo","Difusão culposa de doença ou praga","Desastre ferroviário culposo","Desastre ferroviário culposo","Cláusulas Abusivas","Alienação Parental","Invasão de Dispositivo Informático","Violência Doméstica Contra a Mulher","Usucapião Conjugal","Exploração Sexual","Separação de Corpos","Medidas Protetivas","Extinção das Obrigaões do Falido","Aborto","Conversão da união estável em casamento","Nulidade de ato administrativo","Usucapião de bem móvel","Adjudicação de herança","Registro Civil de Pessoas Jurídicas","Internação Compulsória","Internação Voluntária","Internação Involuntária","Encaminhamento aos pais ou responsável","Orientação e acompanhamento temporário","Matrícula e frequência obrigatória em escola oficial de ensino fundamental","Inclusão em programa de auxílio à família","Requisição de tratamento médico, psicológico ou psiquiátrico, em regime hospitalar ou ambulatorial","Inclusão em programa de auzílio, orientação e tratamento a alcoólatras e toxicômanos","Acolhimento institucional","Inclusão em programa de acolhimento familiar","Colocação em família substituta","Outras medidas de proteção","Evasão Escolar","Exploração Sexual ou Prostituição","Crimes da Lei do Planejamento Familiar","Roubo qualificado","Da Lei do Planejamento Familiar","Crimes Sobre Produção, Comércio e Transporte Clandestino de Açúcar e Álcool","Sobre Produção, Comércio e Transporte Clandestino de Açúcar e Álcool","Crime Relacionado à Exportação de Bens Sensíveis e Serviços Diretamente Relacionados","Relacionado à Exportação de Bens Sensíveis e Serviços Diretamente Relacionados","Crime da Lei Sobre Permissão de Lavra Garimpeira","da Lei Sobre Permissão de Lavra Garimpeira","Crimes da Lei que Proíbe Práticas Discriminatórias para Efeitos Trabalhistas","Da Lei que Proíbe Práticas Discriminatórias para Efeitos Trabalhistas","Crimes da Lei de Proteção à Propriedade Intelectual de Programa de Computador","Da Lei de Proteção à Propriedade Intelectual de Programa de Computador","Crimes da Lei de Locação de Imóveis Urbanos","Da Lei de Locação de Imóveis Urbanos","Crimes da Lei Geral da Copa","Da Lei Geral da Copa","Crimes do Estatuto do Torcedor","Estatuto do Torcedor","Composição Civil","Internação Compulsória","Desapropriação para Regularização de Comunidade Quilombola / Dec. 4887/2003","Análise de Crédito","Registro de Candidatura %u2013 DRAP","Registro de Candidatura - DRAP Partido/Coligação","Prestação de Contas - De Candidato","Prestação de Contas - De Comitê Financeiro","Prestação de Contas - De Exercício Financeiro","Impugnação - Identidade de Eleitor Admitido a Votar","Impugnação - Inscrição Eleitoral","Impugnação - Nomeação de Membro da Junta Eleitoral","Impugnação - Nomeação de Membro da Mesa Receptora","Impugnação - Sistema Eletrônico de Votação","Impugnação - Transferência de Domicílio Eleitoral","Impugnação - Voto por Cédula","Captação ou Gasto Ilícito de Recursos Financeiros de Campanha Eleitoral","Conduta Vedada ao Agente Público","Meios Processuais","Prestação de Contas - de Partido Político","Perda ou Modificação de Guarda","Feminicídio","Homicídio Agravado pela Prática de  Extermínio de Seres Humanos","Homicídio Agravado pelo Extermínio de Seres Humanos","Revisão de Tutela Antecipada","Suspensão do Poder Familiar","Internação Provisória","Registro de Ocorrência pela PM","Registro de Ocorrência pela PM","Requerimento de Reintegração de Posse","Requerimento de Apreensão de Veículo","Cremação/Traslado","Contra a Mulher","Contra pessoas não identificadas como mulher","Contra a mulher","Contra pessoas não identificadas como mulher","Seguro-defeso ao pescado artesanal profissional","Atos de Terrorismo","Organização Terrorista","Preparação de ato terrorista","Manutenção ou financiamento do terrorismo","Fornecimento de medicamentos","Tratamento médico-hospitalar","Unidade de terapia intensiva (UTI) ou Unidade de cuidados intensivos (UCI)","Reajuste contratual","Nomeação","Dispensa","Remoção","Remoção","Levantamento","Levantamento","Remoção","Dispensa","Nomeação","Promoção, constituição, financiamento ou integração de Organização Criminosa","Revelação de identidade, fotografia ou filmagem do colaborador","Imputação falsa de prática de infração penal ou revelação inverídica sobre estrutura de organização criminal","Descumprimento de determinação de sigilo","Recusa ou omissão de dados cadastrais, registros, documentos e informações","Provocação de tumulto ou conduta inconveniente","Apadrinhamento de Criança ou Adolescente","Crimes cometidos contra autoridade ou agente descrito nos arts. 142 e 143 da Constituição Federal, integrantes do sistema prisional e da Força Nacional de Segurança Pública","Fabrico, comércio ou detenção de arma branca ou munição","Porte de arma (branca)","Vias de fato","Omissão de cautela na guarda ou condução de animal","Perturbação do trabalho ou do sossego alheios","Simulação de Qualidade de Funcionário","Uso ilegítimo de uniforme ou distintivo","Jogo de azar","Loteria não autorizada","Jogo do bicho","Servir bebidas alcóolicas a vulneráveis","Perturbação da tranquilidade","Omissão de comunicação de crime","Recusa de dados sobre a própria identificação ou qualificação","Feminicídio","Matéria Administrativa","Proposta de Alteração de Resolução","Autorização de Divulgação de Publicidade Institucional","Cancelamento de Registro de Candidatura","Debate Eleitoral","Enquete","Execução - Cumprimento de Sentença","Fundo Partidário","Registro de Partido Político","Aposentadoria compulsória (art. 42, V, Lei Complementar 35/1979)","Remoção Compulsória","Aposentadoria Compulsória","Remoção Compulsória","Fusão de Partidos Políticos","Perda por crime contra titular do mesmo poder ou descendente (L. 13.715/2018)","Importunação Sexual","Divulgação de cena de estupro, sexo ou pornografia","Inelegibilidade - Renúncia a cargo político na pendência de representação ou petição que possa levar a outra causa de inelegibilidade","Inelegibilidade - Exclusão do exercício de profissão","Inelegibilidade - Aposentadoria Compulsória ou PAD (magistrados e membros do MP)","Inelegibilidade - Demissão do serviço público","Importunação Sexual","Divulgação de cena de estupro, sexo ou pornografia","Híbrida (Art. 48/106)","Nulidade - Ausência de publicidade de decisão","Nulidade - Ausência de Fundamentação de Decisão","Nulidade - Impedimento","Nulidade - Suspeição","Nulidade - Ausência de Nome das Partes","Nulidade - Ausência de Identificação de Advogado ou de Sociedade de Advogados","Nulidade - Ausência de Intimação do Ministério Público","Nulidade - Citação Sem Observância das Prescrições Legais","Nulidade - Título Extrajudicial Não Correspondente a Obrigação Certa, Líquida e Exigível","Nulidade - Intimação Sem Observância das Prescrições Legais","Nulidade - Título Extrajudicial Não Correspondente a Obrigação Certa, Líquida e Exigível","Nulidade - Ausência de Citação do Executado","Nulidade - Execução Instaurada Antes de Condição ou Termo","Ausência de Legitimidade para a Causa","Poderes, Deveres e Responsabilidades do Juiz","Tutela de Urgência","Tutela de Evidência","Revisão de Tutela Antecipada Antecedente","Provas em geral","Tempestividade","Proposta de Nova Resolução","Correição ordinária","Correição extraordinária","Providência","Brumadinho","Boate Kiss","Mariana","Unaí"]
assuntos = set([x.lower() for x in assuntos]) # lower case of all assuntos

data_less_4 = sp_data.loc[ind_range == False]

back_sp_data = pd.read_json('/media/rafael/D/2020 RAFAEL/Faculdade/ICs/IC Direito/codes/pre-processing/SP_PROCESSO_E_ASSUNTO_FEITOS_BCKP.json')

In [7]:
# compare all assuntos in the list in all assuntos which will problably stay find and in the next local which he's probably stay find

verified_assuntos = [np.nan] * len(assuntos_all)

start_time = time.time()
for j, possible_ass in enumerate(tqdm(assuntos_all)):
    if(possible_ass.lower() in assuntos or partes_all[j].lower in assuntos):
        verified_assuntos[j] = possible_ass
print('took {}'.format(time.time() - start_time))
verified_assuntos.count(np.nan)

100%|██████████| 98979/98979 [00:00<00:00, 1441906.72it/s]

took 0.07318472862243652


38395

In [8]:
# tentativa de adequação dos demais documentos, mas nao funciona

# all_right = back_sp_data['Assunto'].to_list()
# array = np.full(len(sp_data['Conteúdo']), float('inf'))

# for i,assunto in enumerate(assuntos):
#     positions = data_less_4['Conteúdo'].str.find(assunto)
#     positions = positions[positions > -1]
    
#     if(len(positions.index) > 0):
#         for j,position in enumerate(positions):

#             if(position < array[positions.index[j]]):
#                 array[positions.index[j]] = position
#                 all_right[int(positions.index[j])] = assuntos[i]

In [9]:
# writing the remanscent data Assuntos

copy = sp_data.copy()
copy.loc[ind_range,('Assunto')] = verified_assuntos
indice_n_nulos = ~copy['Assunto'].isnull()
sp_data.loc[indice_n_nulos,('Assunto')] = copy.loc[indice_n_nulos,('Assunto')]
sp_data['Assunto'].isnull().sum()
# back_sp_data.to_json('SP_PROCESSO_E_ASSUNTO_FEITOS_BCKP.json')

30068

![green-divider](https://user-images.githubusercontent.com/7065401/52071924-c003ad80-2562-11e9-8297-1c6595f8a7ff.png)
### FILTER NOMES

In [10]:
import spacy
from spacy import displacy
import pt_core_news_sm
from spacy.symbols import nsubj, VERB, NOUN
import time

# API for classify genero de nomes 
import csv
import gzip
import io
from unicodedata import normalize

def encode(name):
    ascii_name = normalize("NFKD", name).encode("ascii", errors="ignore").decode("ascii")
    return ascii_name.upper()

def load_data():
    fobj = io.TextIOWrapper(gzip.open("/media/rafael/D/2020 RAFAEL/Faculdade/ICs/IC Direito/codes/pre-processing/Nomes pessoas/nomes.csv.gz"), encoding="utf-8")
    csv_reader = csv.DictReader(fobj)
    data = {
        row["first_name"]: row["classification"]
        for row in csv_reader
    }
    fobj.close()
    return data

name_data = load_data()

def classify_download(name):
    encoded_name = encode(name)
    try:
        return name_data[encoded_name]
    except:
        return 'Nao Nome'

nlp = pt_core_news_sm.load()
Entidades = []
texts = []
completo = []
# sentence_spans = list(doc.sents)
# displacy.render(doc, style="dep")

![green-divider](https://user-images.githubusercontent.com/7065401/52071924-c003ad80-2562-11e9-8297-1c6595f8a7ff.png)
### FILTER Autores e Réus

In [11]:
# filtrando os reus e autores do conteudo
# a ideia é o seguinte. Para cada conteudo semi filtrado + partes vemos qual é a primeira entidade com nome classificada pelo spacy
# esta sera o autor, uma vez feito isso, as demais entidades classificadas serão os reús se e somente se estiverem entre estes separadores ' - '

# start_time = time.time()

# Autores = []
# Reus = []

# for i,conteudo in enumerate(tqdm(conteudo_all)):
#     texto_interesse = partes_all[i] + ' - ' + conteudo_all[i]
#     texto = nlp(texto_interesse)

#     primeiro_ponto_final = texto_interesse.find('.')
#     primeira_virgula = texto_interesse.find(',')
#     separador_oficial = texto_interesse.find(' X ')
#     separadores = []
    
#     for match in re.finditer(' - ',texto_interesse):
        
#         if (primeiro_ponto_final != -1 and primeira_virgula != -1):
            
#             if(match.start() < primeiro_ponto_final or match.start() < primeira_virgula):
#                 separadores.append(match.start())
            
#         else:
#             separadores.append(match.start())
      
#     autor = []
#     reus = []
#     if (separador_oficial > -1):
            
#         primeiro_espaço = -1
#         primeiro_espaço_depois = -1

#         for match in re.finditer(' - ', texto_interesse):
#             if (match.start() < separador_oficial):
#                 primeiro_espaço = match.start()
#             if (match.start() > separador_oficial and primeiro_espaço_depois == -1):
#                 primeiro_espaço_depois = match.start()

#         if (primeiro_espaço > -1):
#             autor.append(texto_interesse[primeiro_espaço:separador_oficial])
#         else:
#             autor.append(texto_interesse[:separador_oficial])

#         if (primeiro_espaço_depois > -1):
#             reus.append(texto_interesse[separador_oficial + 3:primeiro_espaço_depois])
#         else:

#             if (primeira_virgula < primeiro_ponto_final):
#                 reus.append(texto_interesse[separador_oficial + 3:primeira_virgula])
#             else:
#                 reus.append(texto_interesse[separador_oficial + 3:primeiro_ponto_final])
    
#     else:
#         for nome_proprio in texto.ents:

#             start_char_autor = 0

#             if len(autor) < 1:

#                 if separadores != []:

#                     autor.append(texto_interesse[:separadores[0]])
#                 else:
#                     autor.append(nome_proprio)

#             else:
#                 if (nome_proprio.start_char < primeiro_ponto_final and nome_proprio.start_char < primeira_virgula):

#                     if len(separadores) > 1:
#                         if (nome_proprio.start_char >= separadores[0] and nome_proprio.start_char <= separadores[1]):

#                             reus.append(texto_interesse[separadores[0] + 3:separadores[1]])

#                             del separadores[0]
#                     else:
#                         pass
     
#     Autores.append(autor)
#     print(reus)
#     Reus.append(reus)

# #         print(nome_proprio, nome_proprio.start_char, nome_proprio.end_char, nome_proprio.label_, i, primeiro_ponto_final, primeira_virgula)
    
# print("--- %s seconds ---" % (time.time() - start_time))

# ao final temos Autores = todos os autores e Reus = todos os reus encontrados


![green-divider](https://user-images.githubusercontent.com/7065401/52071924-c003ad80-2562-11e9-8297-1c6595f8a7ff.png)
### FILTER in Autores e Réus the gender and the best filter

In [12]:
# pd.DataFrame(Autores).to_csv('Autores.csv')
# pd.Series(Reus).to_csv('Reus.csv')

Reus_df = pd.read_csv('../Reus.csv')
Reus_df = Reus_df.drop(columns=[Reus_df.columns[0]])

series = Reus_df.T.squeeze()
listinha = series.values.tolist()

["['Aldrin Eduardo Dias dos Santos', '- Daniel Dias dos Santos']",
 "['Diretor do Departamento de Despesa Pessoal do Estado de São Paulo']",
 "['Banco Itauleasing S/A']",
 '[]',
 "['Fazenda Municipal de Araraquara']",
 "['BANCO DAYCOVAL S / A']",
 '[]',
 '[]',
 '[]',
 '[]',
 "['Raphael Parisi', '- Eduardo Bracher', '- Liliana Angela Cocito Armanino']",
 '[]',
 "['Telefônica Brasil S/A']",
 "['Renova Energia S/A']",
 "['VALOR EMPREENDIMENTOS IMOBILIÁRIOS LTDA e outro']",
 '[]',
 "['N. P. B.']",
 "['Thiago Araujo de Oliveira']",
 '[]',
 "['MARLIO GUIMARAES SANTOS']",
 "['M.E.M.D.']",
 "['Banco Santander S/A', 'CERTIDÃO']",
 "['Companhia Industrial Dox']",
 '[]',
 '[]',
 "['CERTIDÃO']",
 '[]',
 '[]',
 "['Recorrente: Município de Jahu', 'Recorrido: Celso Pereira']",
 '[]',
 "['ELEKTRO ELETRICIDADE E SERVIÇOS S.A.']",
 "['Jose Rodrigo de Oliveira']",
 "['- Rubens Pimenta Castro', '- Sandra Eliana Castro Vitorino']",
 '[]',
 '[]',
 '[]',
 '[]',
 '[]',
 '[]',
 '[]',
 '[]',
 "['O. J. \\\\<. D.

![green-divider](https://user-images.githubusercontent.com/7065401/52071924-c003ad80-2562-11e9-8297-1c6595f8a7ff.png)
### better approache of filter, i.e., retire the '', ' - ', and so on. In Reus

In [13]:
filtrados = []
for i in tqdm(listinha):
#     print(i)
    string = ""
    if(type(i) == type(string)):
        for char in i:

            if (char != '[' and char != ']' and char != ',' and char != "'" and char != "'" and char != '-'):

                if(string != ""):
                    if(string[-1] == ' ' and char  == ' '):
                        string = string[:-1] + ','
                    else:
                        string = string + char
                else:
                    string = string + char
    filtrados.append(string)

filtrados_reus = []
for i in filtrados:
    spl = i.split(',')
    filtrados_reus.append(spl)
filtrados_reus[0:10]

100%|██████████| 98979/98979 [00:01<00:00, 94533.05it/s]


[['Aldrin Eduardo Dias dos Santos', 'Daniel Dias dos Santos'],
 ['Diretor do Departamento de Despesa Pessoal do Estado de São Paulo'],
 ['Banco Itauleasing S/A'],
 [''],
 ['Fazenda Municipal de Araraquara'],
 ['BANCO DAYCOVAL S / A'],
 [''],
 [''],
 [''],
 ['']]

![green-divider](https://user-images.githubusercontent.com/7065401/52071924-c003ad80-2562-11e9-8297-1c6595f8a7ff.png)
### better approache of filter, i.e., retire the '', ' - ', and so on. In Autores

In [14]:
# pd.DataFrame(Autores).to_csv('Autores.csv')
# pd.Series(Reus).to_csv('Reus.csv')

Autores_df = pd.read_csv('../Autores.csv')
Autores_df = Autores_df.drop(columns=[Autores_df.columns[0]])
series = Autores_df.T.squeeze()
listinha = series.values.tolist()
listinha[0:10]

['Erick Carvalho dos Santos',
 'Hortencia Zambom de Paiva',
 'MARIA APARECIDA PANEGACI DE ARAUJO',
 'Silas Tiago Alves Xavier',
 'Mario Capobianco',
 'JEAN CARLO AMADO',
 'WASHINGTON LUIZ PEREIRA DE ALMEIDA',
 'Justiça Pública',
 'BV Financeira',
 'Estado de São Paulo']

In [15]:
filtrados_autores = []
for i in tqdm(listinha):
    
    string = ""
    if(type(i) == type(string)):
        for char in i:
            if (char != '[' and char != ']' and char != ',' and char != "'" and char != "'" and char != '-' and char != '('and char != ')'):

                if(string != ""):
                    if(string[-1] == ' ' and char  == ' '):
                        string = string[:-1] + ','
                    else:
                        string = string + char
                else:
                    string = string + char
    filtrados_autores.append(string)
filtrados_autores[0:10]

100%|██████████| 98979/98979 [00:01<00:00, 63635.32it/s]


['Erick Carvalho dos Santos',
 'Hortencia Zambom de Paiva',
 'MARIA APARECIDA PANEGACI DE ARAUJO',
 'Silas Tiago Alves Xavier',
 'Mario Capobianco',
 'JEAN CARLO AMADO',
 'WASHINGTON LUIZ PEREIRA DE ALMEIDA',
 'Justiça Pública',
 'BV Financeira',
 'Estado de São Paulo']

![green-divider](https://user-images.githubusercontent.com/7065401/52071924-c003ad80-2562-11e9-8297-1c6595f8a7ff.png)
### Defines the sex of Reus

In [16]:
count = 0
proximos_nomes = []
posicoes = []
primeiros_nomes = []

for i,reus in enumerate(tqdm(filtrados)):
    primeiro_nome = ""
    achou_espaço = 0
    achou_virgula = 0
    
    for j,char in enumerate(reus):
        
        if(char != ' ' and achou_espaço == 0):
            primeiro_nome = primeiro_nome + char
        else:
            if(char == ' ' and j > 0):
                achou_espaço = achou_espaço + 1
        
        if(char == ',' and achou_virgula > 0):
            proximos_nomes.append(proximo_nome)
            posicoes.append(i)
            proximo_nome = ""
            achou_virgula = achou_virgula + 1
            achou_espaço = 1000
            
        if(char == ',' and achou_virgula == 0):
            proximo_nome = ""
            achou_virgula = achou_virgula + 1
            achou_espaço = 1000
            
        if (char == ' ' and achou_espaço >= 1000):
            achou_espaço = achou_espaço + 1
            
        if(char != ' ' and char != ',' and achou_virgula > 0 and achou_espaço == 1000):
            proximo_nome = proximo_nome + char
            
    proximos_nomes.append(proximo_nome)
    posicoes.append(i)
    primeiros_nomes.append(primeiro_nome)
    proximo_nome = ""
    primeiro_nome = ""

100%|██████████| 98979/98979 [00:00<00:00, 112913.80it/s]


In [17]:
reus_classificados = [np.nan]*len(primeiros_nomes)

for i in tqdm(range(len(primeiros_nomes))):
    list_index = []
    string = ""
    for j in range(i, len(posicoes)):
        if(posicoes[j] == i):
            list_index.append(j)
        if(posicoes[j] > i):
            break
            
    if(classify_download(primeiros_nomes[i]) == 'F'):
        if (string == ""):
            string = string + "'Nome': '" + filtrados_reus[i][0]+ "', 'Gênero': " +"'Feminino'" + ", 'Pessoa': " + "'Física'"
            reus_classificados[i] = string
            
    if(classify_download(primeiros_nomes[i]) == 'M'):
        if (string == ""):
            string = string + "'Nome': '" + filtrados_reus[i][0]+ "', 'Gênero': " +"'Masculino'" + ", 'Pessoa': " + "'Física'"
            reus_classificados[i] = string
    for j,index in enumerate(list_index):
        
        if(classify_download(proximos_nomes[index]) == 'F'):
            if (string == ""):
                string = string + "'Nome': '" + filtrados_reus[i][j+1]+ "', 'Gênero': " +"'Feminino'" + ", 'Pessoa': " + "'Física'"
                reus_classificados[i] = string
            else:
                string = string + ",'Nome': '" + filtrados_reus[i][j+1]+ "', 'Gênero': " +"'Feminino'" + ", 'Pessoa': " + "'Física'"
                reus_classificados[i] = string
                
        if(classify_download(proximos_nomes[index]) == 'M'):
#             print(filtrados_reus[i][j+1], j, index)

            if (string == ""):
                string = string + "'Nome': '" + filtrados_reus[i][j+1]+ "', 'Gênero': " +"'Masculino'" + ", 'Pessoa': " + "'Física'"
                reus_classificados[i] = string
            else:
                string = string + ",'Nome': '" + filtrados_reus[i][j+1]+ "', 'Gênero': " +"'Masculino'" + ", 'Pessoa': " + "'Física'"
                reus_classificados[i] = string
reus_classificados[0:12]            

100%|██████████| 98979/98979 [01:01<00:00, 1609.39it/s] 


["'Nome': 'Aldrin Eduardo Dias dos Santos', 'Gênero': 'Masculino', 'Pessoa': 'Física','Nome': 'Daniel Dias dos Santos', 'Gênero': 'Masculino', 'Pessoa': 'Física'",
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 "'Nome': 'Raphael Parisi', 'Gênero': 'Masculino', 'Pessoa': 'Física','Nome': 'Eduardo Bracher', 'Gênero': 'Masculino', 'Pessoa': 'Física','Nome': 'Liliana Angela Cocito Armanino', 'Gênero': 'Feminino', 'Pessoa': 'Física'",
 nan]

In [18]:
primeiros_nomes_autor = []

for i,autor in enumerate(tqdm(filtrados_autores)):
    primeiro_nome = ""
    achou_espaço = 0
    achou_virgula = 0
    
    for j,char in enumerate(autor):
        
        if(char != ' ' and achou_espaço == 0):
            primeiro_nome = primeiro_nome + char
        else:
            if(char == ' ' and j > 0):
                achou_espaço = achou_espaço + 1
        
    primeiros_nomes_autor.append(primeiro_nome)
    primeiro_nome = ""

primeiros_nomes_autor[0:12]

100%|██████████| 98979/98979 [00:00<00:00, 174027.00it/s]


['Erick',
 'Hortencia',
 'MARIA',
 'Silas',
 'Mario',
 'JEAN',
 'WASHINGTON',
 'Justiça',
 'BV',
 'Estado',
 'Prefeitura',
 'Banco']

In [19]:
autores_classificados = [np.nan]*len(primeiros_nomes_autor)

for i in tqdm(range(len(primeiros_nomes_autor))):
    list_index = []
    string = ""
    
#     print(classify_download(primeiros_nomes_autor[i]) == 'F',classify_download(primeiros_nomes_autor[i]), primeiros_nomes_autor[i],i,filtrados_autores[i])
    if(classify_download(primeiros_nomes_autor[i]) == 'F'):
        string = string + "'Nome': '" + filtrados_autores[i]+ "', 'Gênero': " +"'Feminino'" + ", 'Pessoa': " + "'Física'"
        autores_classificados[i] = string
            
    if(classify_download(primeiros_nomes_autor[i]) == 'M'):
        string = string + "'Nome': '" + filtrados_autores[i]+ "', 'Gênero': " +"'Masculino'" + ", 'Pessoa': " + "'Física'"
        autores_classificados[i] = string

100%|██████████| 98979/98979 [00:00<00:00, 348009.65it/s]


![green-divider](https://user-images.githubusercontent.com/7065401/52071924-c003ad80-2562-11e9-8297-1c6595f8a7ff.png)
### Defines if the Autor/Reu is a Goverments or juridic person

In [20]:
# pd.DataFrame(filtrados_autores).to_csv('Autore_filtrados_test.csv')

# todas as cidades segundo o IBGE
cidades_ibge_2019 = pd.read_excel('/media/rafael/D/2020 RAFAEL/Faculdade/ICs/IC Direito/codes/pre-processing/DTB_2019/RELATORIO_DTB_BRASIL_MUNICIPIO.xls') 
cidades_ibge_2019 = cidades_ibge_2019.loc[cidades_ibge_2019['Nome_UF'] == 'São Paulo', ('Nome_Município')].to_list() # todas as cidades do estado de sao paulo dadas pelo ibge de 2010
cidades_ibge_2019 = set([x.lower() for x in cidades_ibge_2019]) # passando os nome das cidades para minúsculo
count = 0
for i,autor in enumerate(tqdm(filtrados_autores)):
    
    if(type(autores_classificados[i]) == type(np.nan)):
        # ve se o autor é uma pessoa juridica
        has_juridic = re.search(r'(?i)(( S( |\/|\.).*A)|(ltda)|( SA)|((.|^)Companhia( |\n))|(( |^)cia( |\.))|( [I]( |i+|\n))|(condom(i|í)nio)|(ind(u|ú)stria)|(col(e|é)gio)|(escola)|(residencial)|(financeira))',autor)
        if(has_juridic):
            string = ""
            string = string + "'Nome': '" + filtrados_autores[i] + "', 'Pessoa': " + "'Jurídica'"
            autores_classificados[i] = string
        else: # senão
            # ve se o autor esta em segredo de justiça
            has_secret = re.search(r'([A-Z]\.( )*){2,}',autor)
            if(has_secret):
                string = ""
                string = string + "'Nome': '" + filtrados_autores[i] + "', 'Pessoa': " + "'Segredo de Justiça'"
                autores_classificados[i] = string
            else:
                # ve se o auto é um governo pela regex
                has_gov = re.search(r'(?i)(justi(ç|a)a|p(u|ú)blica|estado|prefeitura|municipal|munic(í|i)pio|Secretaria|Instituto|Nacional|vara|judicial)',autor)
                if(has_gov):
                    string = ""
                    string = string + "'Nome': '" + filtrados_autores[i] + "', 'Pessoa': " + "'Governo'"
                    autores_classificados[i] = string
                else: 
                    # ve se o autor tem um nome de município
                    if(len(autor) < 200):
                        
                        for j,municipio in enumerate(cidades_ibge_2019):

                            has_city = autor.lower().find(municipio)
                            
                            if(has_city > -1):
                                string = ""
                                string = string + "'Nome': '" + filtrados_autores[i] + "', 'Pessoa': " + "'Governo'"
                                autores_classificados[i] = string
                                break
                            else:
                                has_city_in_assuntos = assuntos_all[i].find(municipio)
                                
                                if(has_city_in_assuntos > -1):
                                    string = ""
                                    string = string + "'Nome': '" + assuntos_all[i] + "', 'Pessoa': " + "'Governo'"
                                    autores_classificados[i] = string
                                    break

100%|██████████| 98979/98979 [00:04<00:00, 20164.13it/s]


In [21]:
teste_autores = sp_data.loc[ind_range]
teste_autores['Autor'] = autores_classificados

/home/rafael/.local/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [22]:
sp_data.loc[ind_range, ('Autor')] = teste_autores.loc[:,('Autor')]
sp_data.isnull().sum()

_id                0
Dados_Fórum        0
Processo           0
Conteúdo           0
Data               0
Assunto        30068
Autor          11063
Classe         77907
Reu            77907
dtype: int64

In [23]:
for i,reus in enumerate(tqdm(filtrados_reus)):
    
    if(type(reus_classificados[i]) == type(np.nan)):
        # ve se o autor é uma pessoa juridica
        string = ""
        
        for j,reu in enumerate(filtrados_reus[i]):
            
            has_juridic = re.search(r'(?i)(( S( |\/|\.).*A)|(ltda)|( SA)|((.|^)Companhia( |\n))|(( |^)cia( |\.))|( [I]( |i+|\n))|(condom(i|í)nio)|(ind(u|ú)stria)|(col(e|é)gio)|(escola)|(residencial)|(financeira))',reu)
            if(has_juridic):
                if(string == ""):
                    string = string + "'Nome': '" + filtrados_reus[i][j] + "', 'Pessoa': " + "'Jurídica'"
                    reus_classificados[i] = string
                else:
                    string = string + ",'Nome': '" + filtrados_reus[i][j] + "', 'Pessoa': " + "'Jurídica'"
                    reus_classificados[i] = string
            else: # senão
                # ve se o autor esta em segredo de justiça
                has_secret = re.search(r'([A-Z]\.( )*){2,}',reu)
                if(has_secret):
                    if(string == ""):
                        string = string + "'Nome': '" + filtrados_reus[i][j] + "', 'Pessoa': " + "'Segredo de Justiça'"
                        reus_classificados[i] = string
                    else:
                        string = string + ",'Nome': '" + filtrados_reus[i][j] + "', 'Pessoa': " + "'Segredo de Justiça'"
                        reus_classificados[i] = string
                else:
                    # ve se o auto é um governo pela regex
                    has_gov = re.search(r'(?i)(justi(ç|a)a|p(u|ú)blica|estado|prefeitura|municipal|munic(í|i)pio|Secretaria|Instituto|Nacional|vara|judicial)',reu)
                    if(has_gov):
                        if(string == ""):
                            string = string + "'Nome': '" + filtrados_reus[i][j] + "', 'Pessoa': " + "'Governo'"
                            reus_classificados[i] = string
                        else:
                            string = string + ",'Nome': '" + filtrados_reus[i][j] + "', 'Pessoa': " + "'Governo'"
                            reus_classificados[i] = string
                    else: 
                        # ve se o autor tem um nome de município
                        if(len(reu) < 200):

                            for municipio in cidades_ibge_2019:

                                has_city = reu.lower().find(municipio)

                                if(has_city > -1):
                                    if(string == ""):
                                        string = string + "'Nome': '" + filtrados_reus[i][j] + "', 'Pessoa': " + "'Governo'"
                                        reus_classificados[i] = string
                                        break
                                    else:
                                        string = string + ",'Nome': '" + filtrados_reus[i][j] + "', 'Pessoa': " + "'Governo'"
                                        reus_classificados[i] = string
                                        break
                            

100%|██████████| 98979/98979 [00:08<00:00, 12172.19it/s]


In [24]:
sp_data.loc[ind_range,('Reu')] = reus_classificados

In [25]:
sp_data.isnull().sum()

_id                0
Dados_Fórum        0
Processo           0
Conteúdo           0
Data               0
Assunto        30068
Autor          11063
Classe         77907
Reu            52774
dtype: int64

![green-divider](https://user-images.githubusercontent.com/7065401/52071924-c003ad80-2562-11e9-8297-1c6595f8a7ff.png)
### Filtrando as Classes

In [26]:
classes = ["Procedimento Ordinário","PROCESSO CÍVEL E DO TRABALHO","SUPERIOR TRIBUNAL DE JUSTIÇA","Procedimento Comum Cível","Procedimento Sumário","Procedimentos Especiais","Procedimentos Especiais de Jurisdição Contenciosa","Anulação e Substituição de Títulos ao Portador","Apreensão e Depósito de Coisa Vendida com Reserva de Domínio","Arrolamento Comum","Arrolamento Sumário","Consignação em Pagamento","Demarcação / Divisão","Depósito","Embargos de Terceiro Cível","Habilitação","Inventário","Monitória","Nunciação de Obra Nova","Prestação de Contas - Oferecidas","Ação de Exigir Contas","Restauração de Autos","Ação Rescisória","Sobrepartilha","Usucapião","Procedimentos Especiais de Jurisdição Voluntária","Abertura, Registro e Cumprimento de Testamento","Alienação Judicial de Bens","Arrecadação das Coisas Vagas","Confirmação de Testamento","Declaração de Ausência","Especialização de Hipoteca Legal","Herança Jacente","Interdição","Organização e Fiscalização de Fundação","Separação Consensual","Tutela e Curatela -  Nomeação","Procedimentos Regidos por Outros Códigos, Leis Esparsas e Regimentos","Ação Civil Coletiva","Ação Civil de Improbidade Administrativa","Ação Civil Pública Cível","Ação Popular","Alimentos - Lei Especial Nº 5.478/68","Alteração do Regime de Bens","Alvará Judicial - Lei 6858/80","Apreensão de Embarcações","Arribadas Forçadas","Regulação de Avaria Grossa","Avarias","Busca e Apreensão em Alienação Fiduciária","Cancelamento de Naturalização","Cautelar Fiscal","Cobrança de Cédula de Crédito Industrial","Compromisso Arbitral","Consignatória de Aluguéis","Conversão de Separação Judicial em Divórcio","Correição Parcial ou Reclamação Correicional","Depósito da Lei 8. 866/94","Desapropriação","Desapropriação Imóvel Rural por Interesse Social","Despejo","Despejo por Falta de Pagamento","Despejo por Falta de Pagamento Cumulado Com Cobrança","Direta de Inconstitucionalidade","Discriminatória","Dissolução e Liquidação de Sociedade","Divórcio Consensual","Divórcio Litigioso","Dúvida","Expropriação da Lei 8.257/91","Falência de Empresários, Sociedades Empresáriais, Microempresas e Empresas de Pequeno Porte","Habeas Data","Habilitação de Crédito","Homologação de Transação Extrajudicial","Imissão na Posse","Impugnação de Crédito","Inquérito Extrajudicial","Mandado de Injunção","Mandado de Segurança Coletivo","Mandado de Segurança Cível","Naturalização","Opção de Nacionalidade","Averiguação de Paternidade","Pedido de Resposta ou Retificação da Lei de Imprensa","Protesto Formado a Bordo","Recuperação Extrajudicial","Recuperação Judicial","Registro Torrens","Relatório Falimentar","Remição do Imóvel Hipotecado","Renovatória de Locação","Restituição de Coisa ou Dinheiro na Falência do Devedor Empresário","Revisional de Aluguel","Separação Litigiosa","Sonegados","Suprimento de Idade e/ou Consentimento","Suspensão de Liminar ou Antecipação de Tutela","Suspensão de Execução de Sentença","Procedimento de Liquidação","Liquidação por Arbitramento","Liquidação de Sentença pelo Procedimento Comum","Liquidação Provisória por Arbitramento","Liquidação Provisória de Sentença pelo Procedimento Comum","Procedimento de Cumprimento de Sentença/Decisão","Cumprimento de sentença","Cumprimento Provisório de Sentença","Processo de Execução","Execução de Título Extrajudicial","Insolvência Civil","Insolvência Requerida pelo Credor","Insolvência Requerida pelo Devedor ou pelo Espólio","Embargos","Embargos à Adjudicação","Embargos à Arrematação","Embargos à Execução","Embargos de Retenção por Benfeitorias","Processo Cautelar","Alimentos - Provisionais","Apreensão de  Títulos","Arresto ","Arrolamento de Bens","Atentado","Busca e Apreensão","Caução","Cautelar Inominada","Exibição","Homologação do Penhor Legal","Justificação","Protesto","Posse em Nome do Nascituro","Produção Antecipada da Prova","Regulamentação de Visitas","Separação de Corpos","Seqüestro","Recursos","Apelação Cível","Remessa Necessária Cível","Agravos","Agravo de Instrumento","Agravo de Instrumento em Recurso Especial","Agravo de Instrumento em Recurso Extraordinário","Agravo Regimental Cível","Embargos","Embargos Infringentes","Embargos Infringentes na Execução Fiscal","Recurso Ordinário Cível","Recurso Extraordinário","Recurso Especial","Outros Procedimentos","Incidentes","Incidente De Arguição de Inconstitucionalidade Cível","Assistência Judiciária","Conflito de competência cível","Exceções","Exibição de Documento ou Coisa Cível","Impugnação ao Cumprimento de Sentença","Impugnação ao Pedido de Assistência Litisconsorcial ou Simples","Impugnação ao Valor da Causa Cível","Incidente de Falsidade","Incidente de Uniformização de Jurisprudência","Remoção de Inventariante","Oposição","Atos e expedientes","Avocatória","Habilitação para Casamento","Instrução de Rescisória","Petição Cível","Comunicação","Reclamação","Registro de Casamento Nuncupativo","Representação por Excesso de Prazo","Cartas","Carta de Ordem Cível","Carta Precatória Cível","Carta Rogatória Cível","PROCESSO CRIMINAL","Medidas Preparatórias","Notícia-Crime","Representação Criminal","Representação Criminal/Notícia de Crime","Pedido de Arquivamento em Representação Criminal","Interpelações","Notificação para Explicações","Notificação para Explicações (Lei de Imprensa)","Procedimentos Investigatórios","Termo Circunstanciado","Inquérito Policial","Auto de Prisão em Flagrante","Procedimento Comum","Ação Penal de Competência do Júri","Ação Penal - Procedimento Ordinário","Processo Especial","Processo Especial do Código de Processo Penal","Crimes de Responsabilidade dos Funcionários Públicos","Crimes de Calúnia, Injúria e Difamação de Competência do Juiz Singular","Crimes Contra a Propriedade Imaterial","Processo Sumário (Detenção)","Restauração de Autos","Processo Especial de Leis Esparsas","Crimes Ambientais","Crimes Contra a Propriedade Industrial","Crimes Contra a Propriedade Intelectual","Crimes de Imprensa","Procedimento do Juizado Especial Criminal - Sumariíssimo","Procedimento Especial da Lei Antitóxicos","Procedimento Especial dos Crimes de Abuso de Autoridade","Medidas Garantidoras","Liberdade","Liberdade Provisória com ou sem fiança","Relaxamento de Prisão","Habeas Corpus Criminal","Medidas Cautelares","Pedido de Busca e Apreensão Criminal","Pedido de Quebra de Sigilo de Dados e/ou Telefônico","Medidas Investigatórias Sobre Organizações Criminosas","Pedido de Prisão","Pedido de Prisão Preventiva","Pedido de Prisão Temporária","Pedido de Prisão/ Liberdade Vigiada para Fins de Expulsão","Questões e Processos Incidentes","Exceções","Exceção de Suspeição","Exceção de Incompetência de Juízo","Exceção de Litispendência","Exceção de Ilegitimidade de Parte","Exceção de Coisa Julgada","Exceção de Impedimento","Exceção da Verdade","Conflito de Jurisdição","Restituição de Coisas Apreendidas","Embargos de Terceiro","Medidas Assecuratórias","Seqüestro","Arresto / Hipoteca Legal","Incidentes","Incidente de Falsidade","Insanidade Mental do Acusado","Cartas","Carta de Ordem Criminal","Carta Precatória Criminal","Carta Rogatória Criminal","Execução Criminal","Execução da Pena","Incidentes","Conversão de Pena","Excesso ou Desvio","Anistia","Indulto","Comutação de Pena","Recursos","Agravo de Execução Penal","Apelação em Mandado de Segurança","Apelação Criminal","Carta Testemunhável","Correição Parcial Criminal","Embargos de Declaração Criminal","Embargos Infringentes e de Nulidade","Protesto por Novo Júri","Recurso de Medida Cautelar Criminal","Recurso de Sentença Criminal","Recurso em Sentido Estrito","Remessa Necessária Criminal","Revisão Criminal","Desaforamento de Julgamento","Incidente de Uniformização de Jurisprudência","Procedimento do Juizado Especial Cível","Pedido de Uniformização de Interpretação de Lei Cível","Recurso Inominado Cível","JUIZADOS DA INFÂNCIA E DA JUVENTUDE","Ação Anulatória de Cláusulas Convencionais","Ação de Cumprimento","Ação Trabalhista - Rito Ordinário","Inquérito para Apuração de Falta Grave","Dissídio Coletivo","Dissídio Coletivo de Greve","Execução de Título Extrajudicial","Execução de Termo de Ajuste de Conduta","Execução de Termo de Conciliação de CCP","Execução de Certidão de Crédito Judicial","Execução Provisória em Autos Suplementares","Agravo","Agravo de Instrumento em Agravo de Petição","Agravo de Instrumento em Recurso de Revista","Agravo de Instrumento em Recurso Ordinário","Agravo de Petição","Agravo Regimental Trabalhista","Embargos","Embargos Infringentes","Recurso de Revista","Recurso Ordinário Trabalhista","Agravo em Recurso de Habeas Corpus","Embargos de Divergência em Recurso de Mandado de Segurança","Embargos de Divergência em Agravo Regimental","Embargos à Execução em Mandado de Segurança","Embargos à Execução em Ação Recisoria","Embargos à Execução em Medida Cautelar","Execução em Ação Rescisória","Execução em Sentença Estrangeira","Execução em Sentença Estrangeira Contestada","Execução em Mandado de Segurança","Execução em Medida Cautelar","Incidente de Deslocamento de Competência ","Sentença Estrangeira","Sentença Estrangeira Contestada","Sindicância","Mandado de Segurança","Reclamação","Recurso Ordinário","Recurso Especial","Ação Penal","Interpelação Judicial","Apelação Cível","Suspensão de Liminar e de Sentença","Embargos Infringentes em Apelação Cível","Representação","Requisição de Pequeno Valor","Inquérito","Agravo de Instrumento","Agravo em Recurso Extraordinário","Agravo em Recurso de Mandado de Segurança","Carta Rogatória","Habeas Data","Conflito de Atribuição","Conflito de Competência","Petição","Medida Cautelar","Exceção da Verdade","Remessa Necessária em Habeas Corpus","Embargos Infringentes em Ação Rescisória","Procedimentos Trabalhistas","Processo de Execução Trabalhista","Incidentes Trabalhistas","Recursos Trabalhistas","Pedido de Revisão do Valor da Causa","Processo de Conhecimento","Procedimento de Conhecimento","Execução de Título Judicial","Execução de Alimentos","Execução Contra a Fazenda Pública","Execução Fiscal","Execução Hipotecária do Sistema Financeiro da Habitação","Embargos à Execução Fiscal","Tutela e Curatela - Remoção e Dispensa","Justificação de Dinheiro a Risco","Ação Trabalhista - Rito Sumaríssimo","Ação Trabalhista - Rito Sumário (Alçada)","Embargos de Divergência em Recurso Especial","Conflito de Competência","Incidente de Arguição de Inconstitucionalidade Criminal","PROCEDIMENTOS ADMINISTRATIVOS","Pedido de Providências","Reclamação","Agravo Interno Cível","Agravos","Exceção de Impedimento","Exceção de Suspeição","Exceção de Incompetência","Efeito Suspensivo","Processo Administrativo Disciplinar em face de Servidor","Processo Administrativo Disciplinar em face de Magistrado","Precatório","Requisição de Pequeno Valor","Recurso de Multa","Medidas Protetivas de urgência (Lei Maria da Penha) Criminal","Habeas Corpus Cível","Recurso de Medida Cautelar Cível","Superveniência de doença mental","Unificação de penas","Conflito de atribuição","Transferência entre estabelecimentos penais","Outras medidas provisionais","Reabilitação","Outros procedimentos de jurisdição voluntária","Alvará Judicial","Declaratória de Constitucionalidade","Intervenção em Município","Processo Administrativo","Recurso Administrativo","Reclamação Disciplinar","Correição Extraordinária","Inspeção","Recurso em Processo Administrativo Disciplinar em face de Servidor","Correição Ordinária","Sindicância","SUPREMO TRIBUNAL FEDERAL","Ação Cautelar","Ação Cível Originária","Ação Declaratória de Constitucionalidade","Ação Direta de Inconstitucionalidade","Ação Originária","Ação Originária Especial","Ação Penal","Ação Rescisória","Agravo Interno","Agravo de Instrumento","Agravo Regimental","Arguição de Descumprimento de Preceito Fundamental","Arguição de Suspeição","Carta Rogatória","Comunicação","Conflito de Competência","Embargos de Declaração","Embargos de Divergência","Embargos Infringentes","Extradição","Habeas Corpus","Habeas Data","Inquerito","Intervenção Federal","Mandado de Injunção","Mandado de Segurança","Medida Cautelar","Petição","Prisão Preventiva para Extradição","Processo Administrativo","Questão de Ordem","Reclamação","Recurso Criminal","Recurso em Habeas Corpus","Recurso em Habeas Data","Recurso em Mandado de Injunção","Recurso em Mandado de Segurança","Recurso Extraordinário","Revisão Criminal","Sentença Estrangeira","Sentença Estrangeira Contestada","Suspensão de Liminar","Suspensão de Segurança","Suspensão de Tutela Antecipada","Siglas dos Processos Extintos","Ação Ordinária Regressiva","Ação Regressiva","Apelação Cível","Apelação Criminal","Arguição de Relevância","Carta Testemunhável","Conflito de Atribuições","Conflito de Jurisdição","Denúncia","Embargos Remetidos","Exceção da Verdade","Exceção de Suspeição","Inquérito Administrativo","Inquérito Policial","Inquérito Policial Especial","Interpelação","Liquidação de Sentença","Notificação","Pedido de Avocação","Processo Crime","Processo Judicial","Queixa Crime","Recurso de Apreensão de Livro","Recurso de Liquidação de Sentença","Recurso de Revista","Representação","Retificação de Nome Estrangeiro","Sentença Arbitral","Suspensão de Direitos","Seção Cível","Processo de Conhecimento","Ação de Alimentos","Revisão Judicial de Decisão do Conselho Tutelar","Apuração de Irregularidades em Entidades de Atendimento","Apuração de Infração Administrativa às Normas de Proteção à Criança ou Adolescente","Tutela Infância e Juventude","Tutela c/c Destituição do Poder Familiar","Adoção","Adoção c/c Destituição do Poder Familiar","Suprimento de Capacidade ou de Consentimento para Casar","Emancipação","Regularização de Registro Civil","Guarda","Providência ","Prestação de Contas Infância e Juventude","Perda ou Suspensão do Poder Familiar","Processo de Execução","Execução de Alimentos Infância e Juventude","Execução de Medida de Proteção à Criança e Adolescente","Execução de Multa","Processos Cautelares","Busca e Apreensão Infância e Juventude","Cautelar Inominada Infância e Juventude","Cartas","Carta de Ordem Infância e Juventude","Carta Precatória Infância e Juventude","Seção Infracional","Procedimentos Investigatórios","Auto de Apreensão em Flagrante","Relatório de Investigações","Boletim de Ocorrência Circunstanciada","Processo de Apuração de Ato Infracional","Execução de Medidas Sócio-Educativas","Internação com Atividades Externas","Internação sem Atividades Externas","Semiliberdade","Liberdade Assistida","Prestação de Serviços a Comunidade","Obrigação de Reparar o Dano","Advertência ","Cartas","Carta de Ordem Infracional","Carta Precatória Infracional","Ação Rescisória","Ação de Improbidade Administrativa","Comunicação","Exceção de Impedimento","Exceção de Suspeição","Intervenção Federal","Mandado de Injunção","Precatório","Revisão Criminal","Suspensão de Segurança","Consulta Administrativa","Retificação ou Suprimento ou Restauração de Registro Civil","Retificação de Registro de Imóvel","Remessa Necessária Trabalhista","Embargos de Declaração Cível","Ação Civil Pública Infância e Juventude","Mandado de Segurança Infância e Juventude","Nomeação de Advogado","Impugnação de Assistência Judiciária","Autorização judicial","Embargos de Terceiro Infância e Juventude","Remoção, modificação  e dispensa de tutor ou curador","Procedimento Comum Infância e Juventude","Reintegração / Manutenção de Posse","Interdito Proibitório","Mandado de Segurança Criminal","Agravo de Instrumento em Recurso Especial","Agravo de Instrumento em Recurso Extraordinário","Execução Provisória","Embargos do Acusado","Alienação de Bens do Acusado","Avaliação para atestar dependência de drogas","Habeas Corpus","Recurso Ordinário em Mandado de Segurança","Recurso Ordinário em Habeas Corpus","Contraprotesto Judicial","Notificação","Interpelação","Petição Criminal","Apelação / Remessa Necessária","Agravo Regimental Criminal","Recurso Ordinário Criminal","Investigação contra magistrado","Procedimento Investigatório Criminal (PIC-MP) ","Habilitação para Adoção","Correição Parcial Cível","Ação Penal - Procedimento Sumário","Ação Penal - Procedimento Sumaríssimo","Incidente de Sanidade Mental","Medidas Protetivas - Estatuto do Idoso Criminal ","Incidentes","Conflito de Competência Infância e Juventude","Exceções","Exibição de Documento ou Coisa Infância e Juventude","Impugnação ao Valor da Causa Infância e Juventude","Incidente de Falsidade Infância e Juventude","Exceção de Impedimento Infância e Juventude","Exceção de Incompetência Infância e Juventude","Exceção de Suspeição Infância e Juventude","Petição Infracional","Cumprimento Provisório de Decisão","Impugnação ao Cumprimento  de Decisão","Petição Infância e Juventude Cível","Remessa Necessária / Recurso Ordinário","PROCESSO MILITAR","PROCEDIMENTOS ESPECIAIS PREVISTOS EM LEIS ESPARSAS","PROCESSO CRIMINAL","Procedimento Especial","Procedimentos Investigatórios","Recursos","Conselho de Justificação ","Representação p/ Declaração de Indignidade/Incompatibilidade ","Representação p/ Perda da Graduação ","Ação Penal Militar - Procedimento Ordinário","Embargos em Ação Penal Militar","Reclamação Militar","Recurso Inominado Militar","Inquérito Policial Militar","Correição Parcial Militar","Deserção","Insubmissão","Deserção de Oficial","Deserção de Praça","PROCEDIMENTOS PRÉ-PROCESSUAIS DE RESOLUÇÃO CONSENSUAL DE CONFLITOS","Extinção das obrigações do falido","Recurso em sentido estrito/Recurso ex officio","Execução de Medida de Segurança","Agravo de Instrumento em Recurso Ordinário para o STF","PROCESSO ELEITORAL","Processos Cíveis-Eleitorais","Ação de Impugnação de Mandato Eletivo","Ação de Investigação Judicial Eleitoral","Ação Penal Eleitoral","Procedimentos Relativos a Realização de Eleição","Apuração de Eleição","Prestação de Contas","Registro de Candidatura","Recurso contra Expedição de Diploma","Procedimentos Relativos a Partidos Políticos","Cancelamento de Registro de Partido Político","Propaganda Partidária","Registro de Órgão de Partido Político em Formação","Registro de Comitê Financeiro","Registro de Partido Político","Procedimentos Administrativos da Justiça Eleitoral","Representação","Correição","Criação de Zona Eleitoral ou Remanejamento","Instrução","Lista Tríplice","Revisão de Eleitorado","Recursos Eleitorais","Recurso Eleitoral","Recurso Especial Eleitoral","Recurso Ordinário Eleitoral","Consulta","Pedido de Desaforamento","Instrução Provisória de Deserção","Suspensão de Liminar e de Sentença","Suspensão de Segurança Cível","Assistência Judiciária","Exibição de Documento ou Coisa Criminal","Impugnação ao Valor da Causa","Impugnação de Assistência Judiciária","Pedido de Uniformização de Interpretação de Lei Criminal","Produção Antecipada de Provas Criminal","Restituição de Coisas Apreendidas Infracional","Processo de Aplicação de Medida de Segurança Por Fato Não Criminoso","Instrução Provisória de Insubmissão","Instrução Provisória de Deserção","Reclamação Pré-processual","Recurso Extraordinário com Agravo","Agravo em Recurso Especial","Recurso de Revista com Agravo","Recurso Ordinário - Rito Sumaríssimo","Acompanhamento de Cumprimento de Decisão","Ato Normativo","Nota Técnica","Parecer de Mérito sobre Anteprojeto de Lei","Procedimento de Controle Administrativo","Revisão Disciplinar","Reclamação para Garantia das Decisões"," Comissão","Arguição de Suspeição e de Impedimento","Cautelar Inominada Criminal","Embargos de Divergência em Agravo em Recurso Especial","Remição de Pena","Pedido de Busca e Apreensão Infracional","Reexame Necessário Eleitoral","Suspensão de Segurança","Suspensão de Segurança - Eleitoral","Exceção","Ação Cautelar","Pedido de Medida de Proteção","Procedimentos Cautelares","Busca e Apreensão Infracional","Internação Provisória","Pedido De Desinternação/Reavaliação/Substituição/Suspensão da Medida","Procedimento Conciliatório","Restabelecimento do Poder Familiar","Homologação em Acordo de Colaboração Premiada","Cumprimento de Sentença contra a Fazenda Pública","Execução de Título Extajudicial contra a Fazenda Pública","Incidente de Impedimento Cível","Incidente de Suspeição Cível","Carta Arbitral","Tutela Antecipada Antecedente","Tutela Cautelar Antecedente","Incidente de Resolução de Demandas Repetitivas","Dissolução Parcial de Sociedade","Incidente de Assunção de Competência","Liquidação Provisória de Sentença pelo Procedimento Comum","Renovação de Permanência em Estabelecimento Penal Federal","Suspensão em Incidente de Resolução de Demandas Repetitivas","Suspensão em Incidente de Resolução de Demandas Repetitivas","Incidente de Desconsideração de Personalidade Jurídica","Agravo em Execução de Medidas Sócio-Educativas","Auto de Prisão","Reclamação Criminal","Roteiro de Pena","Roteiro de Pena","Progressão de Regime","Regressão de Regime","Remição","Livramento Condicional","Outros Incidentes de Execução Iniciados de Ofício","Incidente de Julgamento de Recurso de Revista e de Embargos Repetitivos","Tutela Provisória","Tutela Cautelar Antecedente","Tutela Antecipada Antecedente","Pedido de Mediação Pré-Processual","Requerimento de Apreensão de Veículo","Requerimento de Reintegração de Posse","Contestação em Foro Diverso","Embargos Parciais à Ação Monitória","Execução de Título Extrajudicial","Suspensão Nacional do Incidente de Resolução de Demandas Repetitivas","Tutela Provisória Antecedente","Tutela Provisória Incidental","Prestação de Contas Eleitorais","Notificação","Interpelação","Protesto","Protesto formado a bordo","Guarda c/c destituição do poder familiar","Cumprimento de sentença - Lei Arbitral (Lei 9.307/1996)","Ação de Partilha","Tutela Cível","Curatela","Cumprimento de Sentença de Obrigação de Prestar Alimentos","Execução Extrajudicial de Alimentos","Pedido de Cooperação Jurisdicional","Suspensão em IRDR","Suspensão em IRDR","Execução de Título Judicial - CEJUSC","Pedido de Efeito Suspensivo à Apelação","Tomada de Decisão Apoiada","Tomada de Decisão Apoiada","Alteração de Regime de Bens","Divórcio Consensual","Divórcio Litigioso","Homologação da Transação Extrajudicial","Reclamação","Regulação de Avaria Grossa","Prestação de Contas - Partidos Políticos","Incidente de Impedimento Infância e Juventude","Incidente de Suspeição Infância e Juventude","Ação de Partilha","ação de partilha","Incidente de Uniformização de Jurisprudência","Revisão Criminal","Medidas Protetivas de Urgência (Lei Maria da Penha) Infracional","Medidas Protetivas - Estatuto do Idoso Infracional","Corregedoria Eleitoral","Petição Corregedoria","Inquérito Administrativo"]
classes = [classe.lower() for classe in classes]

In [27]:
# compare all assuntos in the list in all assuntos which will problably stay find and in the next local which he's probably stay find

verified_classes = [np.nan] * len(classes_all)

start_time = time.time()
for j, possible_class in enumerate(tqdm(classes_all)):
    if(possible_class.lower() in classes or classes_all[j].lower in classes):
        verified_classes[j] = possible_class
print('took {}'.format(time.time() - start_time))
verified_classes.count(np.nan)

100%|██████████| 98979/98979 [00:00<00:00, 380768.43it/s]

took 0.26360177993774414


5311

In [28]:
sp_data.loc[ind_range,('Classe')] = verified_classes

![green-divider](https://user-images.githubusercontent.com/7065401/52071924-c003ad80-2562-11e9-8297-1c6595f8a7ff.png)
### Filtrando as Valores

In [30]:
count = 0

verified_valores = [np.nan] * len(conteudo_all)

for i,conteudo in enumerate(tqdm(conteudo_all)):
    string = ""
    found_valor = re.findall(r'R\$[ |]?[0-9]*[\.|,|]?[0-9]+,[0-9]+', conteudo, re.IGNORECASE)
    
    if(found_valor):
        
        for f in found_valor:
            nums = re.findall(r'[0-9]*[\.|,|]?[0-9]+,[0-9]+', f, re.IGNORECASE)
            if(string == ""):
                string = "'Valores': " + "'R$': '" + nums[0] + "'"
                verified_valores[i] = string
            else:
                string = string + ",'R$': '" + nums[0] + "'"
                verified_valores[i] = string
verified_valores.count(np.nan)

100%|██████████| 98979/98979 [00:01<00:00, 84540.17it/s]


86940

In [31]:
verified_valores
sp_data.loc[ind_range,'Valores'] = verified_valores 

In [46]:
sp_data.loc[sp_data['Assunto'].isnull(), :]
sp_data.isnull().sum(), new_sp_data.isnull().sum()
# sp_data.to_json('Amostra100KFiltrada.json',orient='records',lines=True, force_ascii=False)
# sp_data['Conteúdo'][4]

### TESTE TRAIN

In [33]:
from __future__ import unicode_literals, print_function

import plac
import random
from pathlib import Path

# new entity label
LABEL = 'VALOR'


TRAIN_DATA = [
    ("Telefônica Brasil S/A - Em execução individual de sentença coletiva, a TELEFÔNICA impugnou o pedido principal. Intimada, o exequente não se manifestou (fls. 298).Devo reconhecer que muito do que eu tenho por convicção pessoal sobre o tema é matéria superada pelo Tribunal de Justiça de São Paulo, que definiu, ponto a ponto, a solução para cada questionamento. Adoto, doravante, sem prejuízo de minha convicção pessoal, tudo quanto decidido pelo Tribunal Bandeirante e pelo Superior Tribunal de Justiça. Apresenta-se correta a escolha de foro e a via eleita para execução do julgado. O deslocamento do procedimento coletivo de execução leva em conta critério diverso na fixação da competência, calcado na Efetividade do Provimento Jurisdicional. A adstrição ao juízo prolator da sentença coletiva é critério severamente rechaçado, eis que aplica sistemática liberal e individualista do CPC a feito eminentemente social.Quando a liquidação depende de mero cálculo aritmético, compete ao exequente apresentá-lo desde logo, iniciando-se a fase de execução. Cabe ao executado, se desejar, impugnar o valor, desde que apresente aquele que entende devido.O espelho de contratação fornecido pela própria executada é suficiente para demonstração do direito e início de execução. Se houver excesso em qualquer ponto compete à impugnante apontá-los em concreto, já apresentando seu cálculo. O contrato está englobado na sentença (pós 25.08.1996). E a alienação das ações em 1997 não impede a execução da diferença a que teria direito o adquirente das ações, devendo ser indenizado pelo equivalente computados dividendos apenas até alienação.O parâmetro de cálculo foi estabelecido na sentença coletiva e não mais é passível de alteração. O STJ, em recurso repetitivo, já declarou que é devido respeito à coisa jugada.RECURSO ESPECIAL REPRESENTATIVO DA CONTROVÉRSIA. CIVIL E PROCESSUAL CIVIL. BRASIL TELECOM S/A. CONTRATO DE PARTICIPAÇÃO FINANCEIRA. CESSÃO DE DIREITOS. LEGITIMIDADE ATIVA DO CESSIONÁRIO. COMPLEMENTAÇÃO DE AÇÕES. CONVERSÃO DA OBRIGAÇÃO EM PERDAS E DANOS. CRITÉRIOS. COISA JULGADA. RESSALVA. 1. Para fins do art. 543-C do CPC: 1.1. O cessionário de contrato de participação financeira tem legitimidade para ajuizar ação de complementação de ações somente na hipótese em que o instrumento de cessão lhe conferir, expressa ou tacitamente, o direito à subscrição de ações, conforme apurado nas instâncias ordinárias. 1.2. Converte-se a obrigação de subscrever ações em perdas e danos multiplicando-se o número de ações devidas pela cotação destas no fechamento do pregão da Bolsa de Valores no dia do trânsito em julgado da ação de complementação de ações, com juros de mora desde a citação. 1.3. Os dividendos são devidos durante todo o período em que o consumidor integrou ou deveria ter integrado os quadros societários. 1.3.1. Sobre o valor dos dividendos não pagos, incide correção monetária desde a data de vencimento da obrigação, nos termos do art. 205, § 3º, Lei 6.404/76, e juros de mora desde a citação. 1.3.2. No caso das ações convertidas em perdas e danos, é devido o pagamento de dividendos desde a data em que as ações deveriam ter sido subscritas, até a data do trânsito em julgado do processo de conhecimento, incidindo juros de mora e correção monetária segundo os critérios do item anterior. 1.4. Ressalva da manutenção de critérios diversos nas hipóteses de coisa julgada. 2. Caso concreto: 2.1. Recurso Especial de BRASIL TELECOM S/A: Ausência de indicação do dispositivo de lei federal que fundamenta a alegada divergência jurisprudencial, o que atrai a incidência do óbice da Súmula 284/STF. 2.2. Recurso Especial de SÉRGIO MARQUES ASSESSORIA IMOBILIÁRIA LTDA: 2.2.1. Inocorrência de maltrato ao art. 535 do CPC quando o acórdão recorrido, ainda que de forma sucinta, aprecia com clareza as questões essenciais ao julgamento da lide. Ademais, o magistrado não está obrigado a rebater, um a um, os argumentos deduzidos pelas partes. 2.2.2. Ausência de indicação do dispositivo de lei federal que fundamenta a alegada divergência jurisprudencial no que tange à questão da legitimidade ativa. Óbice da Súmula 284/STF. 2.2.3. “Nos contratos de participação financeira para a aquisição de linha telefônica, o Valor Patrimonial da Ação (VPA) é apurado com base no balancete do mês da integralização” (Súmula 371/STJ). 2.2.4. Aplicação do item 1.2 ao caso concreto. 2.2.5. Aplicação do item 1.3.2. ao caso concreto. 2.2.6. Carência de interesse recursal no que tange ao critério de arbitramento dos honorários advocatícios, devido à sucumbência recíproca. 3. RECURSO ESPECIAL DE BRASIL TELECOM S/A NÃO CONHECIDO E RECURSO ESPECIAL DE SÉRGIO MARQUES ASSESSORIA IMOBILIÁRIA LTDA PARCIALMENTE CONHECIDO E, NESSA PARTE, PARCIALMENTE PROVIDO. (REsp 1301989/RS, Rel. Ministro PAULO DE TARSO SANSEVERINO, SEGUNDA SEÇÃO, julgado em 12/03/2014, DJe 19/03/2014)O Tribunal de Justiça, porém, interpretou a situação de forma a determinar a correção monetária contada do trânsito em julgado da sentença coletiva: Agravo de instrumento - Ação civil pública - Fase de liquidação - Controvérsia a respeito dos critérios de liquidação do crédito do acionista - Execução que pode ser proposta no foro do domicílio do exequente (art. 98, §2º, I do CDC) - Juros de mora que são devidos a partir da citação da TELEFONICA na fase de conhecimento, conforme decisão proferida em sede de recurso repetitivo (REsp 1.361.800/SP) - Possibilidade de adoção dos critérios estabJelecidos pelo C. STJ para cálculo do valor devido, considerando-se o valor da ação na data do trânsito em julgado da ação civil pública (EDcl no AREsp 266175/RS e AgRg no REsp 1351033/RS) - Provimento, em parte. (Relator(a): Enio Zuliani; Comarca: Votuporanga; Órgão julgador: 4ª Câmara de Direito Privado; Data do julgamento: 14/06/2016; Data de registro: 14/06/2016) Agravo de instrumento - Ação civil pública - Fase de liquidação - Controvérsia a respeito dos critérios de liquidação do crédito do acionista, aplicação da multa de R$ 3.000,00 e dobra acionária - Possibilidade de adoção dos critérios estabelecidos pelo C. STJ para cálculo do valor devido, considerando-se o valor da ação na data do trânsito em julgado da ação civil pública (EDcl no AREsp 266175/RS e AgRg no REsp 1351033/RS) - Juros de mora que são devidos a partir da citação da TELEFONICA na fase de conhecimento, conforme decisão proferida em sede de recurso repetitivo (REsp 1.361.800/SP) - Multa fixada na sentença que, caso venha a ser exigida, deve ser revertida ao Fundo Estadual de Reparação de Interesses Difusos - Decisão que expressamente excluiu a indenização da dobra acionária, o que revela a falta de interesse recursal, no ponto. Recurso conhecido, em parte, e na parte conhecida, provido em parte. (Relator(a): Enio Zuliani; Comarca: Lins; Órgão julgador: 4ª Câmara de Direito Privado; Data do julgamento: 11/08/2016; Data de registro: 11/08/2016)Privilegio aqui a manifestação do Tribunal, delimitando a forma de aplicação da decisão do STJ, e ressalvado meu entendimento pessoa. Correção monetária será do trânsito em julgado da ação coletiva.Juros moratórios variados no tempo são devidos desde a citação no processo de conhecimento. (0,6% mensal pré NCC e 1% mensal pós NCC). Matéria, inclusive, pacificada no STJ (juros de mora correm do processo originário e não da execução).AÇÃO CIVIL PÚBLICA - CADERNETA DE POUPANÇA - PLANOS ECONÔMICOS - EXECUÇÃO - JUROS MORATÓRIOS A PARTIR DA DATA DA CITAÇÃO PARA A AÇÃO COLETIVA - VALIDADE - PRETENSÃO A CONTAGEM DESDE A DATA DE CADA CITAÇÃO PARA CADA EXECUÇÃO INDIVIDUAL - RECURSO ESPECIAL IMPRÓVIDO .1.- Admite-se, no sistema de julgamento de Recursos Repetitivos (CPC, art. 543-C, e Resolução STJ 08/98), a definição de tese uniforme, para casos idênticos, da mesma natureza, estabelecendo as mesmas consequências jurídicas, como ocorre relativamente à data de início da fluência de juros moratórios incidentes sobre indenização por perdas em Cadernetas de Poupança, em decorrência de Planos Econômicos. 2.- A sentença de procedência da Ação Civil Pública de natureza condenatória, condenando o estabelecimento bancário depositário de Cadernetas de Poupança a indenizar perdas decorrentes de Planos Econômicos, estabelece os limites da obrigação, cujo cumprimento, relativamente a cada um dos titulares individuais das contas bancárias, visa tão-somente a adequar a condenação a idênticas situações jurídicas específicas, não interferindo, portando, na data de início da incidência de juros moratórios, que correm a partir da data da citação para a Ação Civil Pública. 3.- Dispositivos legais que visam à facilitação da defesa de direitos individuais homogêneos, propiciada pelos instrumentos de tutela coletiva, inclusive assegurando a execução individual de condenação em Ação Coletiva, não podem ser interpretados em prejuízo da realização material desses direitos e, ainda, em detrimento da própria finalidade da Ação Coletiva, que é prescindir do ajuizamento individual, e contra a confiança na efetividade da Ação Civil Pública, O que levaria ao incentivo à opção pelo ajuizamento individual e pela judicialização multitudinária, que é de rigor evitar. 3.- Para fins de julgamento de Recurso Representativo de Controvérsia (CPC, art. 543-C, com a redação dada pela Lei 11.418, de 19.12.2006), declara-se consolidada a tese seguinte: “Os juros de mora incidem a partir da citação do devedor na fase de conhecimento da Ação Civil Pública, quando esta se fundar em responsabilidade contratual, se que haja configuração da mora em momento anterior.” 4.- Recurso Especial impróvido (REsp 1361800/SP, Rel. Ministro RAUL ARAÚJO, Rel. p/ Acórdão Ministro SIDNEI BENETI, CORTE ESPECIAL, julgado em 21/05/2014, DJe 14/10/2014)Quanto à multa de R$ 3.000,00. Ressalvado meu entendimento pessoal, o Tribunal de Justiça definiu que a multa é destinada ao Fundo Estadual de Reparação de Interesses Difusos, pelo que falta, ao exequente, legitimidade para pleiteá-la.Agravo de instrumento - Ação civil pública - Fase de liquidação - Controvérsia a respeito dos critérios de liquidação do crédito do acionista, aplicação da multa de R$ 3.000,00 e dobra acionária - Possibilidade de adoção dos critérios estabelecidos pelo C. STJ para cálculo do valor devido, considerando-se o valor da ação na data do trânsito em julgado da ação civil pública (EDcl no AREsp 266175/RS e AgRg no REsp 1351033/RS) - Juros de mora que são devidos a partir da citação da TELEFONICA na fase de conhecimento, conforme decisão proferida em sede de recurso repetitivo (REsp 1.361.800/SP) - Multa fixada na sentença que, caso venha a ser exigida, deve ser revertida ao Fundo Estadual de Reparação de Interesses Difusos - Decisão que expressamente excluiu a indenização da dobra acionária, o que revela a falta de interesse recursal, no ponto. Recurso conhecido, em parte, e na parte conhecida, provido em parte. (Relator(a): Enio Zuliani; Comarca: Lins; Órgão julgador: 4ª Câmara de Direito Privado; Data do julgamento: 11/08/2016; Data de registro: 11/08/2016). Agravo de instrumento - Ação civil pública - Fase de liquidação - Controvérsia a respeito dos critérios de liquidação do crédito do acionista, aplicação da multa de R$ 3.000,00 e dobra acionária - Execução que pode ser proposta no foro do domicílio do exequente (art. 98, §2º, I do CDC) - Possibilidade de adoção dos critérios estabelecidos pelo C. STJ para cálculo do valor devido, considerando-se o valor da ação na data do trânsito em julgado da ação civil pública (EDcl no AREsp 266175/RS e AgRg no REsp 1351033/RS) - Juros de mora que são devidos a partir da citação da TELEFONICA na fase de conhecimento, conforme decisão proferida em sede de recurso repetitivo (REsp 1.361.800/SP) - Exequente que alienou suas ações depois da cisão da Telesp Celular, o que lhe assegura o direito à dobra acionária - Multa fixada na sentença que deve ser excluída das contas, já que deve ser revertida ao Fundo Estadual de Reparação de Interesses Difusos - Provimento, em parte. (Relator(a): Enio Zuliani; Comarca: Votuporanga; Órgão julgador: 4ª Câmara de Direito Privado; Data do julgamento: 15/06/2016; Data de registro: 15/06/2016)Quanto à dobra acionária. Eu entendo tratar-se de fato posterior à sentença, que deve ser incluído na condenação, posto que reflete prejuízo direto do consumidor por ato exclusivo da executada. Haveria evidente quebra de sinalagma caso o direito não fosse reconhecido - a empresa, que não pagou corretamente ao consumidor o que ele tinha direito, locupletou-se da sua falta, sem repassar a quem tinha direito, o quanto recebeu pelo valor da diferença não pago. Para que haja direito à dobra, basta que as ações não tenham sido alienadas ANTES da cisão da Telesp.O Tribunal de Justiça de São Paulo tem entendido dessa forma:Agravo de instrumento - Ação civil pública - Fase de liquidação - Controvérsia a respeito dos critérios de liquidação do crédito do acionista, aplicação da multa de R$ 3.000,00 e dobra acionária - Execução que pode ser proposta no foro do domicílio do exequente (art. 98, §2º, I do CDC) - Possibilidade de adoção dos critérios estabelecidos pelo C. STJ para cálculo do valor devido, considerando-se o valor da ação na data do trânsito em julgado da ação civil pública (EDcl no AREsp 266175/RS e AgRg no REsp 1351033/RS) - Juros de mora que são devidos a partir da citação da TELEFONICA na fase de conhecimento, conforme decisão proferida em sede de recurso repetitivo (REsp 1.361.800/SP) - Exequente que alienou suas ações depois da cisão da Telesp Celular, o que lhe assegura o direito à dobra acionária - Multa fixada na sentença que deve ser excluída das contas, já que deve ser revertida ao Fundo Estadual de Reparação de Interesses Difusos - Provimento, em parte. (Relator(a): Enio Zuliani; Comarca: Votuporanga; Órgão julgador: 4ª Câmara de Direito Privado; Data do julgamento: 15/06/2016; Data de registro: 15/06/2016)O Superior Tribunal de Justiça, porém, entende de forma diversa:AGRAVO REGIMENTAL NO RECURSO ESPECIAL. BRASIL TELECOM S.A. CUMPRIMENTO DE SENTENÇA. DOBRA ACIONÁRIA. AUSÊNCIA DE PREVISÃO NO TÍTULO EXECUTIVO JUDICIAL. INCLUSÃO NOS CÁLCULOS APRESENTADOS. IMPOSSIBILIDADE. PRECEDENTES. RECURSO IMPRÓVIDO 1. É necessário que, na ação de conhecimento, tenha havido reconhecimento expresso ao direito à dobra acionária (telefonia móvel), não cabendo, no cumprimento de sentença, tal inclusão na memória de cálculo em razão da coisa julgada ter-se realizado sobre o direito da complementação acionária da telefonia fixa.2. Agravo regimental impróvido .(AgRg no REsp 1546740/SC, Rel. Ministro MARCO AURÉLIO BELLIZZE, TERCEIRA TURMA, julgado em 09/08/2016, DJe 19/08/2016).A decisão do Superior Tribunal de Justiça NÃO é vinculante, ainda, posto que não sumulada nem submetida a apreciação definitiva da Corte em regime próprio. Adoto, pois, posição da minha Corte, inclusive em consonância com meu entendimento pessoal, mantendo o direito ao recebimento da Dobra Acionária.Ante o exposto, JULGO PARCIALMENTE PROCEDENTE a impugnação para: 1) determinar a correção monetária de acordo com o Resp 1301989/RS; 2) afastar a cobrança da multa de R$ 3.000,00 por ilegitimidade do exequente para reclamá-la.O exequente arcará com honorários fixados em 10% sobre o valor cobrado em excesso.Transcorrido o prazo de recurso desta, o exequente deve apresentar planilha de débito atualizada.", {
        'entities': [(5980, 6031, 'VALOR'), (15106,15195,'VALOR')]
    }),

    ("Renova Energia S/A - Vistos. O exequente deve ficar ciente que a sucessão de petições com o mesmo requerimento (fls. 290, 303 e 309) não agilizará a tramitação do feito, muito ao contrário. Certifique a serventia, se o caso, o decurso do prazo de pagamento e embargos, já que a petição de fls. 217/223 não o interrompeu ou suspendeu. Os pedidos de homologação de acordo e suspensão do processo (fls. 278, 280/285 e 286/287) estão prejudicados pelo teor das petições de fls. 290, 303 e 309, comunicando o “descumprimento” do acordo antes mesmo de sua homologação por sentença. Em outras palavras, o ajuste entre as partes permaneceu como mera tentativa extrajudicial infrutífera de composição, cujo único reflexo na presente execução foi o de amortização da dívida, diante do pagamento, pela executada, de R$ 428.260,57. Segundo a exequente, resta o saldo devedor, já acrescido de multa de 10%, no valor de R$ 942.173,25 (planilha de fls. 315). Defiro o arresto de valores em conta bancária/aplicações financeiras da empresa executada RENOVA (CNPJ 08.534.605/0001-74), até o limite do item 4 (taxa já recolhida a fls. 308). Int após o protocolo da ordem de bloqueio.", {
        'entities': [(694,820, 'VALOR'), (865, 920, 'VALOR')]
    }),

    ("Data da Inscrição: * Nº da Inscrição no Registro da Dívida Ativa: * Valor da Dívida: R$ 2.008,88 ", {
        'entities': [(69, 98, 'VALOR')]
    }),

    ("Nos termos do Comunicado 165/2014, publicado em 13/02/2014), DJE, providencie a parte autora o recolhimento das custas para reprodução de peças do processo para composição de contrafé. As custas deverão ser recolhidas na Guia do Fundo Especial de Despesa do Tribunal, código 201, no valor de R$0,50 (cinquenta centavos) por folha, observando-se a necessidade de instrução dos mandados com uma contrafé para cada requerido.Prazo (05) cinco dias. Com a devida regularização , e/ou ou decorridos, seguem os autos a decisão, independentemente de nova intimação.", {
        'entities': [(281, 300, 'VALOR')]
    }),

    ("Intimação do(a,s) requerente(s) para recolher custas de declaração de IR - pessoa física, no valor de R$11,00, de acordo com o Provimento CSM nº 1826/2010, no prazo de 05 dias . Int.", {
        'entities': [(91, 110, 'VALOR')]
    }),

    ("Vistos. Tendo decorrido do prazo para oposição de embargos à execução, relativo à penhora realizada via BacenJud, julgo EXTINTO o presente feito nos termos do art. 794, I, do CPC. Expeça-se mandado de levantamento no valor de R$1.098,03, em favor da exequente. Decorrido o prazo de 90 dias estabelecido no item 30.2. do Provimento CSM 1670/09, com as alterações estabelecidas no Provimento CSM 1679/09, inutilizem-se os autos. P.R.I.C.", {
        'entities': [(215, 260, 'VALOR')]
    }),

    ("Marcelo Francisco dos Santos - ( X ) requerer o levantamento do valor depositado em favor do Oficial de Justiça, o qual não foi integralmente utilizado, indicando o Banco, agência e número de conta para eventual transferência (R$ 27,18 R$ 13,59 = R$ 13,59), no prazo de 05 (cinco) dias. No silêncio, arquive-se.", {
        'entities': [(213, 258, 'VALOR')]
    }),

    ("XISTO POLIZEL - (rel. 292)Vistos. Defiro o pedido de fl. 44/45, expedindo-se o necessário Int. (SERROTE: 4 ATOS, SOBRA DE DILIGÊNCIAS DA GUIA RECOLHIDA À FL. 36 = r$ 13,59, RECOLHER + 3 DILIGÊNCIAS NO VALOR DE r$ 40,77)", {
        'entities': [(114, 173, 'VALOR'), (187,220,'VALOR')]
    }),

    ("Fazenda Pública Estadual - Vistos.Fls. 143: Consoante o depósito de fls. 124, promova a serventia expedição de mandado de levantamento judicial da quantia de R$ 424,02 em favor da FAZENDA PÚBLICA ESTADUAL, com as cautelas de praxe.Oportunamente, ao arquivo.Intime-se.", {
        'entities': [(123, 169, 'VALOR')]
    }),

    ("Uma vez que o autor quedou-se inerte quanto à intimação de fls.43 no prazo que lhe foi assinalado, INDEFIRO LIMINARMENTE A INICIAL, com fundamento no art. 284, parágrafo único do CPC. Aguarde-se por cinco dias o recolhimento das custas iniciais, sob pena de inscrição na dívida ativa. Oportunamente, anote-se a extinção do feito, remetendo-se ao arquivo. P. R. I. - CUSTAS DEVIDAS EM EVENTUAL APELAÇÃO (ressalvado o caso de beneficiário da gratuidade ou hipótese de isenção de custas) A recolher em guia própria (DARE) pelo Código 230-6 (Ao Estado) R$ 106,25, equivalente a 2% sobre o valor da causa, ou sobre o valor da condenação, conforme o caso, atualizado de acordo com a Tabela do E. Tribunal de Justiça de SP, ressalvado, o valor mínimo de 5 {cinco} e máximo de 3.000 {três mil} UFESP’s, em consonância à Lei nº 11.608/03.", {
        'entities': [(486, 559, 'VALOR')]
    }),

    ("Controle nº 1786/2009Vistos.Recolha a autora as custas pertinentes, nos termos do Provimento 1864/2011 (R$ 12,20 - Guia do Fundo de Despesas do TJSP (FEDTJ - código 434-1).Após, tornem.Intimem-se.", {
        'entities': [(29, 113, 'VALOR')]
    }),

    ("Banco Itaucard S/A - - Itaú Seguros S/A - Nilson Antonio Leal - Ante o teor da manifestação de fls. 166/167 e comprovante de depósito de fls. 168, expeça-se mandado de levantamento judicial em favor do autor da quantia de R$ 4.090,60. No mais, diga o autor acerca do cumprimento integral da obrigação de fazer disposta na sentença, alertando-o que o silêncio implicará em presunção da satisfação e a consequente extinção do feito. Intime-se. Itanhaem, 18 de março de 2015.", {
        'entities': [(203, 235, 'VALOR')]
    }),

    ("Marco Ernani Hyssa Luiz busca nesta via a condenação de Ademir Feliciano, mantenedor do site altiaqui.com.br, à obrigação de não fazer, consistente na ?cessação imediata dos atos ilegais e abusivos que vem sendo perpetrados pelo requerido?, bem como o pagamento de indenização por danos morais. Pleiteia, em antecipação de tutela, a cessação imediata dos atos ilegais e abusivos, a exclusão do texto ofensivo do site em que publicado e a concessão de direito de resposta. DECIDO. Observo que parte dos textos impugnados se destina a demonstrar o anseio pela cassação do prefeito e incutir a dúvida no leitor sobre a honestidade do requerente ? o que o requerido iria ?desmascarar?. Tais pontos não demandariam repreensão e se inseririam no rol de críticas a que o homem público estaria sujeito e até obrigado a tolerar por se prestar ao meio político. No entanto, ingressa o requerido na esfera individual do requerente e o associa a ?ladrão, safado e desonesto?, além de pessoa ligada à ?corrupção? ? o que se revela abusivo e merece ser retirado da mídia em que publicada. Ressalto que não há como separar do restante da publicação que contém a crítica moderada, como acima ressaltado, pois se trata, como o próprio requerido afirma, de paródias, uma delas baseada no que teria sido o jingle da última campanha à eleição do requerente. Assim, merece a exclusão de todos os dois textos publicados sob o título ?Modinha para Nanão!? e ?Modinha para Nanão ? Peça nº 2?. No mais, não merece acolhimento o pedido de abstenção de novas publicações ofensivas à imagem do autor. Trata-se de comando legalmente instituído e cuja observação é imposta a todos. O comportamento ilícito, portanto, já é proibido, sob pena de sanções, sejam elas as reparadoras na esfera civil, ou as que cominam pena no âmbito criminal. Desta forma, não há espaço para tal vedação neste âmbito, se a lei, por si só, já a estabelece. Por fim, indefiro a publicação da resposta pretendida às fls. 08, pois não se limita à versão própria sobre os fatos questionados, mas busca apenas impor uma retratação forçada ao requerido - que não se sabe, ademais, se há. Nesses termos, determino que proceda o requerido à retirada do site dos dois textos referidos, no prazo de 48 (quarenta e oito) horas, bem como se abstenha de publicá-los em quaisquer outras mídias, sob pena de pagamento de multa diária no montante de R$500,00 por dia de descumprimento, tanto pela determinação da exclusão, como por eventual nova publicação. No entanto, constatada pela serventia o equívoco no recolhimento das guias de fls. 23 e a ausência de pagamento da diligência de oficial de justiça, proceda o requerente, primeiramente, à adequação da situação e apenas após constatada a regularidade dos pagamentos, expeça-se mandado para cumprimento da medida de urgência e citação do réu.", {
        'entities': [(2342, 2392, 'VALOR')]
    }),

    ("Banco Santander Sa - VISTOS. PRISCILLA ALVES BOCCI propôs a presente AÇÃO DE REVISÃO DE CONTRATO BANCÁRIO c.c DANOS MORAIS COM PEDIDO DE ANTECIPAÇÃO DE TUTELA contra BANCO SANTANDER S/A. Alegando, em síntese, que no dia 30 de junho de 2011, efetivou contratação de uma cédula de crédito bancário de nº. 00330002320000409590 para a renegociação de contratos de empréstimos efetivados através de sua conta corrente, no valor de R$ 15.587,85. Não obstante o valor financiado total deste contrato ficou em R$ 17.262,81 e com os encargos financeiros o valor do crédito chegou ao patamar de R$ 53.485,20. Assevera que se sujeitou a práticas abusivas levadas a efeito pelo requerido e, sobre o saldo devedor eram acrescidos juros capitalizados e acima dos limites legais, além de comissão de permanência cumulada com correção monetária e juros de mora. Requereu a procedência do pedido, com a revisão do contrato, bem como a concessão da antecipação da tutela para impedir o apontamento de negativação junto ao Serasa e SPC, e a condenação do requerido na devolução em dobro do que foi cobrada a maior, bem como na indenização por danos morais. Com a petição inicial vieram os documentos de fls. 20/90. O pedido de antecipação da tutela foi deferido, conforme decisão de fls. 95. O réu, devidamente citado, apresentou contestação às fls. 101/124. Réplica às fls. 138/158. É O RELATÓRIO. DECIDO. A presente demanda comporta julgamento no estado em que se encontra, porque a matéria em discussão é unicamente de direito, tendo em vista que está relacionada à interpretação e validade de cláusulas contratuais, inexistindo razão para a realização de prova pericial ou oral em audiência, sendo suficientes, pois, as já juntadas aos autos (artigo 330, inciso I, Código de Processo Civil). O pedido é procedente em parte. Cuida-se de ação através da qual a autora pretende a declaração de nulidade de cláusulas contratuais, com a revisão do débito para que se afaste a cobrança de juros excessivos, incidentes de forma capitalizada, além da comissão de permanência cumulada com correção monetária, com condenação do réu na restituição dos valores pagos a maior, bem como na indenização por danos morais. Conveniente, com o fim de propiciar uma melhor análise das questões controvertidas, apontar a natureza do negócio jurídico inicialmente firmado pelas partes, que tem por objeto contrato bancário Cédula de Crédito (Confissão e Renegociação de Dívida). Por meio de tal contrato o banco adianta ao cliente, recursos necessários a certo empreendimento, normalmente destinados à aquisição de bens, mediante cessão ou caução de crédito ou com outras garantias. Assim sendo, é certo que o cliente está se beneficiando desse valor que lhe foi “emprestado” e daí, o banco faz jus à remuneração do capital, até porque essa, em última análise, é a razão de ser de sua atividade empresarial. Não se perca de vista que, por outro lado, malgrado a irresignação manifestada pelas instituições bancárias nos últimos tempos, dúvidas não pode haver quanto à aplicabilidade do Código de Defesa do Consumidor à relação jurídica existente entre as partes, eminentemente de consumo por envolver, em pólos opostos, pessoas enquadráveis nos conceitos de ‘fornecedor’ e ‘consumidor’, e ter por objeto o ‘produto’ ou o ‘serviço’. É a conclusão que se tira da análise do disposto no artigos 2º, caput, e 3º da Lei 8.078/90, ressaltando-se que este último dispositivo, em seu segundo parágrafo, incluiu, textualmente, as atividades de natureza bancária no conceito de ‘serviço’ (... inclusive as de natureza bancária...). E, ainda, a jurisprudência está pacificada pela súmula 297 do Superior Tribunal de Justiça, com o seguinte teor: “O Código de Defesa do Consumidor é aplicável às instituições financeiras”. Sem dúvida, pois, que as relações contratuais bancárias entre o autor e o requerido devem submeter- se às regras previstas pela legislação consumeirista. Pois bem. A título de esclarecimento, contrato, mesmo que de adesão, por si só, não compromete o vínculo existente entre as partes. Não há vedação legal a tal forma de instrumentalização dos negócios, aliás, predominante nos contratos bancários. A respeito são relevantes as considerações de Humberto Theodoro Júnior, em artigo publicado na Revista dos Tribunais, volume 765, página 18: “É importante ressaltar, todavia, que o direito moderno não repele o contrato de adesão. Ao contrário, convive amplamente com ele e reconhece-lhe a eficácia própria dos negócios bilaterais. É que o contrato de adesão, por si só, não se reveste de ilicitude nem incorre em qualquer tipo de censura. O que se recrimina é o abuso cometido eventualmente dentro do contrato de adesão pela parte que dispõe da força de determinar o conteúdo de suas cláusulas. Se, porém, não se entrevê nenhum desvio ético na estipulação de tais condições, o contrato de adesão é tão jurídico e tão obrigatório para os contratantes como qualquer outro contrato”. Firmou-se o entendimento de que as disposições do Decreto n. 22.626/33 não se aplicam às instituições financeiras no que se refere às taxas de juros, conforme preceitua a Súmula n. 596 do STF, uma vez que a matéria foi, neste particular, regulada por lei especial posterior (Lei n. 4.595/64). A respeito, confira-se: RTJ 115/1270, JSTJ 30/158, RT 698/100. O Supremo Tribunal Federal, em mais de uma oportunidade (RT 729/131, 732/139), manifestou-se também pela necessidade de regulamentação do artigo 192, parágrafo 3º, da Constituição da República, que limita a taxa de juros em 12% ao ano, entendendo que a norma não é auto-aplicável (vide também: RT 656/128, 677/127, 679/119, 698/100, 708/118, 704/125). A orientação gerou a Súmula n. 648 do STF: “A norma do § 3º, do art. 192 da Constituição, revogada pela Emenda Constitucional 40/2003, que limitava a taxa de juros reais a 12% ao ano, tinha sua aplicabilidade condicionada à edição de lei complementar”. Em junho de 2008 foi editada sobre o tema a SÚMULA VINCULANTE N. 7, que transcrevo: “A NORMA DO §3º DO ARTIGO 192 DA CONSTITUIÇÃO, REVOGADA PELA EMENDA CONSTITUCIONAL Nº 40/2003, QUE LIMITAVA A TAXA DE JUROS REAIS A 12% AO ANO, TINHA SUA APLICAÇÃO CONDICIONADA À EDIÇÃO DE LEI COMPLEMENTAR” (Fonte de Publicação DJE nº 112/2008, p. 1, em 20/6/2008). Portanto, se permitida a cobrança de juros em montante superior a 12% ao mês, em princípio, não pode ser taxada de abusiva a prática. Viável, da mesma forma, a cobrança segundo as taxas de mercado (Súmula n. 296, do STJ). Não está evidenciada também a lesão. A propósito, ensina Caio Mário da silva Pereira (“Lesão nos Contratos”, 6ª edição, Editora Forense, Rio de Janeiro 1997): “... O segundo elemento consiste no abuso da premente necessidade, inexperiência ou leviandade da outra parte. É a base da teoria da lesão, segundo De Page. Não basta que um contrato seja prejudicial, acrescenta, mas é preciso que, além da vantagem que um contratante aufira, se verifique, ainda, a especulação em torno da situação particular que levou o outro a celebrar o contrato, que lhe é tão desfavorável. Diante disto, apura-se que outro contratante, o beneficiado, praticou um ato consciente, positivo, ao se aproveitar daquelas condições desfavoráveis do lesado ...” (p. 164/165). Além da possibilidade de cobrança de juros em percentual acima dos 12% ao mês, a inexperiência ou ingenuidade do autor não foi alegada e não se presume. Deve estar fundada em argumentos consistentes e concretos, o que não ocorre no caso. No que se refere ao anatocismo, por muito tempo se reconheceu que taxa de juros não se confunde com capitalização de juros, matéria não disciplinada pela Lei n. 4.595/64, especial. Sendo assim, prevalecia a vedação constante do Decreto n. 22.626/33 (art. 4º) e o disposto na Súmula n. 121 do Supremo Tribunal Federal (“é vedada a capitalização de juros ainda que expressamente convencionada”). Neste sentido: R. T. J. 124/616; R. Esp. 3.894, Sr. Min. Nilson Naves; R. Esp. N. 27.935-9 PR 4ª T. j. 28-6-1994 rel. Min. Fontes de Alencar. Com o advento da Medida Provisória nº 1963-17/2000, o quadro alterou-se, havendo manifestações do Superior Tribunal de Justiça no sentido da possibilidade de tal prática (STJ - EDcl no AgRg no RECURSO ESPECIAL Nº 664.669 - RS (2004/0070404-1) - RELATORA : MINISTRA NANCY ANDRIGHI TERCEIRA TURMA unanimidade Data do julgamento: 19 de maio de 2005 DJ: 06/06/2005). É certo que existe orientação no sentido da violação da regra contida no artigo 7º da Lei complementar 95/98, entendimento este que cheguei a agasalhar, acompanhando precedente do Primeiro Tribunal de Alçada Civil do Estado de São Paulo (1º TAC/SP 12ª Câmara Ap. nº 1.234.745-4-SP; rel. designado Juiz José Araldo da Costa Telles; j. 10.08.04; maioria de votos pub. no Boletim da AASP nº 2400, pág. 330/334). No entanto, melhor refletindo sobre o tema, tenho que as regras da mencionada legislação (Lei Complementar n. 95/98) têm conteúdo programático e uma vez aprovada a lei, ainda que com inobservância da técnica legislativa por ela preconizada, subentende-se a vontade do legislador, que deve ser prestigiada e acatada. A eficácia da medida provisória é assegurada pelo disposto no artigo 2º da Emenda Constitucional n. 32/2001 (“Art. 2º As medidas provisórias editadas em data anterior à da publicação desta emenda continuam em vigor até que medida provisória ulterior as revogue explicitamente ou até deliberação definitiva do Congresso Nacional”). O STF, na época da sistemática em que foi editada a medida provisória, orientou-se ainda no sentido de que a avaliação dos requisitos de necessidade e urgência era da alçada do Presidente da República, vedada a valoração a respeito pelo Poder Judiciário (STF AgRg em RE 231.630-0-PR 2ª T. j. 24.08.1999 rel. Min. Néri da Silveira DJU 24.09.1999, V. U. RT 771/177). Enfim, viável é a capitalização. Já quanto à multa, os instrumentos que acompanharam a inicial contêm previsões limitando-a 2% (fl. 28-cláusula 30), portanto, conforme preceitos do Código de Defesa do Consumidor. No tocante à utilização da TR ainda é admitida em nosso ordenamento jurídico. Nesse sentido: STF, RE 175.678, j. 19.11.94, Rel. Min. Carlos Velloso, DJU 04.08.95, p. 22.549. Confira-se ainda o disposto na Súmula n. 295 do STJ. Quanto à comissão de permanência, o Superior Tribunal de Justiça não veda incondicionalmente o encargo, tanto que sumulou o entendimento no sentido de que “não é potestativa a cláusula contratual que prevê a comissão de permanência, calculada pela taxa média de mercado apurada pelo Banco Central do Brasil, limitada à taxa do contrato” (SÚMULA Nº 294, STJ). A orientação da mesma Corte (STJ), no entanto, é firme no sentido de que não se admite a cumulação com encargos decorrentes da mora, relativamente ao mesmo período, dentre eles, juros, correção e multa - grifei (vide Resolução n. 1.129/86 e STJ - Recurso Especial N° 407.096 - RS (2002/0005883-4) Rel. Min. Aldir Passarinho Júnior Quarta Turma Votação unânime Data do Julgamento: 27 de junho de 2002 D.J.: 26/08/2002). Tal cobrança acumulativa pode, exemplificativamente, ser identificada no contrato, na qual é clara quanto à cumulação de juros+comissão +multa+correção. Em atenção à orientação cristalizada no STJ, nos períodos em que a cobrança se deu de forma cumulativa, o encargo (comissão de permanência) será afastado. O quadro evidencia que, embora afastada a comissão de permanência, não há onerosidade excessiva passível de reconhecimento ou mesmo que se tenha verificado lesão ou causa autorizadora da anulação do pactuado. Enfim, é o caso de rever em parte o saldo. A hipótese não comporta restituição em dobro, na forma do artigo 42, do CDC. A existência de cláusula contratual autorizando a incidência do encargo justifica a incidência da parte final do parágrafo único do artigo 42, que exclui a devolução dobrada no caso de “engano justificável”. Isso não bastasse, o dispositivo (art. 42, CDC) reclama pagamento superior ao devido. É o que se extrai da expressão “pagou em excesso” referida no parágrafo único, que transcrevo: “Art. 42. ... Parágrafo único. O consumidor cobrado em quantia indevida tem direito à repetição do indébito, por valor igual ao dobro do que pagou em excesso, acrescido de correção monetária e juros legais, salvo hipótese de engano justificável” (grifo meu). Essa a lição de Antônio Herman de Vasconcellos e Benjamin (Código Brasileiro de Defesa do Consumidor Comentado pelos Autores do Anteprojeto, Editora Forense Universitária, 4ª edição, 1995, p. 264), ao comentar a norma em questão (art. 42, § único, CDC): “ ... Note-se que diversamente do que sucede com o regime civil, há necessidade de que o consumidor tenha, efetivamente, pago indevidamente. Não basta a simples cobrança ...”. No mesmo sentido os comentários de Arruda Alvim, Thereza Alvim, Eduardo Arruda Alvim e James Marins (Código do Consumidor Comentado, Editora Revista dos Tribunais, 2ª edição, 1995, p. 224): “Naturalmente, não é suficiente para que a aplicabilidade da sanção cominada por este parágrafo único, que o consumidor seja apenas cobrado em excesso. Faz-se necessário que o consumidor tenha realmente efetivado o pagamento indevido”. Ainda, o sistema de amortização francês, conhecido como Tabela Price, não implica em juros capitalizados, mas na simples distribuição dos juros e do capital em parcelas justas durante o período estabelecido no contrato para amortização do empréstimo. De acordo com PAULO SANDRONI, a Tabela Price consiste em um “sistema de amortização de dívidas em prestações iguais, compostas de duas parcelas, uma de juros e a outra do principal, isto é, do capital inicialmente emprestado. A Tabela Price deve seu nome provavelmente ao inglês R. Price, que durante o século XVIIII relacionou a teoria dos juros compostos às amortizações de empréstimos, e se denomina também sistema francês de capitalização. Na medida em que a prestação é composta de dois elementos uma de juros e outra do principal -, a fórmula permite calcular os juros devidos na primeira parcela e, por subtração da prestação que se deseja pagar, a parcela do principal que se deseja amortizar” (DICIONÁRIO DE ECONOMIA E ADMINISTRAÇÃO, ED. NOVA CULTURAL, 1.996, P. 404). A fórmula da Tabela Price é desenvolvida para determinar um fator que multiplicado pelo valor do principal, venha resultar num valor de prestação constante no tempo. O mérito dessa fórmula é o de permitir que um valor seja amortizado no tempo estipulado, apropriando-se, sempre, uma parcela de juros que se apura multiplicando a taxa mensal pelo saldo devedor. Esse valor de juros, deduzido do valor da prestação calculada pelo fator da fórmula, resultará no valor da amortização que será deduzida do saldo. No período seguinte, é sobre este novo saldo apurado que a mesma taxa de juros voltará a incidir para se apurar os novos juros da prestação. Assim ocorre sucessivamente. Pode-se observar que, em nenhum momento, se processa qualquer mecanismo de capitalização, vale dizer, de incorporação dos juros ao saldo devedor que sirva como base para cálculo de novos juros. O que se percebe, na realidade, é a ilegítima pretensão da autora em desvirtuar totalmente as características do mútuo que livremente foi pactuado por partes maiores, capazes e esclarecidas. Tal pretensão é inadmissível, sob pena de ofensa ao ato jurídico perfeito e ao princípio do pacta sunt servanda, inexistindo no caso em exame qualquer nulidade ou ilegalidade que deva ser declarada pelo Julgador, ausentes, ainda, os requisitos necessários para a revisão do contrato com base na teoria da imprevisão. No presente caso, o contrato apresentado leva à conclusão de que a autora tinha clara ciência (como exigido legalmente) das obrigações ou conseqüências das cláusulas que lhe eram impostas com a contratação, posto que explicitadas nos documentos, preenchendo os requisitos exigidos, inclusive, pelo Código de Defesa do Consumidor (conforme artigos 46 e 52), com a ressalva da cláusula 2.3, conforme fundamentado. Destarte, o referido princípio “pacta sunt servanda”, tem incidência ampla, não incumbindo ao Juiz, ao seu próprio talante, aplicá-lo simplesmente quando lhe convier. Assim, não sendo caso de nulidade, imprevisão e outras exceções taxativas e limitadas, o contrato faz lei entre as partes, devendo ser respeitada e cumprida as demais cláusulas contratuais. Assim, é de rigor a procedência em parte do pedido para rever o saldo devedor quando da constatação da cumulação da comissão de permanência e demais e encargos. Oportuno esclarecer, posto que pertinente, não há que se falar em devolução em dobro, vez que o artigo 42, parágrafo único, do Código de Defesa do Consumidor prevê sua incidência na hipótese de cobrança indevida, o que não se verificou na espécie, considerando-se que a parte requerida, ainda que tenha agido de forma equivocada, o fez embasada em cláusula contratual que, a princípio, era válida, até sua desconstituição pela presente decisão judicial. Assim, não configurada quaisquer das hipóteses previstas na legislação vigente. Quanto ao pedido de indenização por danos morais, em que pese os fatos declinados na petição inicial, não trouxe a autora elementos capazes de demonstrar os danos por ela experimentados, deste modo e, pelas razões supra mencionadas, igualmente não merece acolhimento, vez que tal circunstância é incapaz de gerar os danos alardeados. Isto posto, JULGO PROCEDENTE EM PARTE o pedido inicial, declarando a nulidade das cláusulas que implicam a cumulação da comissão de permanência com outros encargos da mora, determinando-se ao réu a revisão do saldo devedor, com afastamento da comissão de permanência nos períodos em que incidiu de forma cumulativa com juros, correção monetária e/ou multa e/ou encargos, nos termos da fundamentação supra. A restituição, se o caso, de valores a autora se dará de forma simples, de eventuais valores pagos a maior, ou a compensação, tudo a ser apurado quando do cumprimento da sentença. Os cálculos deverão ser apurados na forma do disposto no artigo 475-B do Código de Processo Civil. Em razão da sucumbência recíproca, cada parte arcará com o pagamento das custas e despesas processuais que deram causa, inclusive os honorários advocatícios de seus patronos. O preparo para a hipótese de interposição do recurso de apelação, se devido, corresponderá a 2% (dois por cento) do valor singelo atribuído à causa, ressalvado o disposto no artigo 4º, § 1º, da Lei Estadual nº 11.608/03, sem prejuízo, ademais, da necessidade de comprovação do recolhimento do valor relativo ao porte de remessa e retorno dos autos (§ 4º do mesmo dispositivo), na forma do artigo 511 do Código de Processo Civil. P.R.I.", {
        'entities': [(415, 440, 'VALOR'),(441, 516, 'VALOR'),(534, 598, 'VALOR')]
    }),

    ("Vistos.FÁBIO VALLEJO GARCIA ajuizou ação de prestação de contas contra RACHEL HELENA NICOLELLA BALSEIRO.Alega que, na ação de alimentos nº 1014102-17.2016.8.26.0562, foram arbitrados alimentos provisórios de 30% (trinta por cento) de seus vencimentos líquidos, que correspondem atualmente a R$ 5.231,67 (cinco mil, duzentos e trinta e um reais e sessenta e sete centavos), em favor de seu filho João Henrique Balseiro Garcia, nascido em 23.09.2006, atualmente com 10 (dez) anos de idade.Está evidente que o autor não pretende a prestação de contas, mas sim demonstrar que o valor arbitrado a título de alimentos provisórios naquela ação de alimentos, nas palavras do autor, “é excessivo e elevado” (fl 03).Além disso, menciona que a mãe teria que destinar igual quantia em favor do menor e que então o valor destinado ao filho seria de R$ 10.463,34 (dez mil, quatrocentos e sessenta e três reais e trinta e quatro centavos).Em razão disso, pede a prestação de contas.Os argumentos do autor devem ser apresentados na ação de alimentos, na qual não foi finalizada a instrução do feito, bem como, também, pode interpor recursos durante o trâmite do processo.Somente na ação de alimentos é que pode ser avaliado se o montante arbitrado a título de verba alimentar está ou não adequado, assim como, se a mãe e o pai tem rendas idênticas, uma vez que cada qual colabora com o percentual de sua renda e, por consequência, é muito raro o cálculo feito pelo autor de que o montante destinado ao menor deve ser o dobro daquele fixado a título de alimentos provisórios.Em toda a leitura da petição inicial não resta dúvida da indignação do autor quanto ao valor dos alimentos provisórios, salientando, ainda, que se trata de “farta quantia a título de pensão alimentícia” (fl 03).A avaliação se o menor precisa ou não desse valor integral de R$ R$ 5.231,67 (cinco mil, duzentos e trinta e um reais e sessenta e sete centavos) a cada mês, correspondente a 30% dos vencimentos líquidos do pai, é matéria a ser debatida e solucionada na ação de alimentos, onde ainda não há decisão com trânsito em julgado.O autor não atendeu ao pressuposto processual de validade objetiva intrínseca no sentido que deixou de respeitar o procedimento adequado, ao pretender comprovar que o valor dos alimentos provisórios é exorbitante por meio de ação de prestação de contas, enquanto que tal matéria deve ser apresentada na ação de alimentos que se encontra nem fase de instrução, respeitados, naquele processo, os prazos processuais.O autor é carecedor desta ação por inadequação da via processual eleita.Pelo exposto, reconheço a ausência do pressuposto processual adequado e, nos termos do artigo 485, inciso IV, do Código de Processo Civil, extingo este processo.Custas pelo requerente, devidamente recolhidas às fls 10/14.Certifique-se o oportuno trânsito em julgado da sentença e comunique-se a extinção do feito no sistema eletrônico do Tribunal de Justiça do Estado de São Paulo.Ciência do Ministério PúblicoP.R.I.C.", {
        'entities': [(280, 305, 'VALOR'), (764,851,'VALOR'), (1815,1847,'VALOR')]
    }),

    ("Vistos. Defiro ao(à) requerente os benefícios da Justiça Gratuita.Trata-se de ação objetivando a concessão do benefício assistencial de prestação continuada ao(à) portadora(a) de deficiência. Deve-se, portanto, apurar-se através de indispensável perícia médica a existência e extensão da ventilada deficiência, sendo necessária, ainda, a comprovação da condição de miserabilidade alegada na inicial. Note-se, ainda, o perigo da irreversibilidade, com possibilidade de dano à parte acionada. Não se faz, portanto, legítima a antecipação pugnada na inicial nesta fase processual. Posto isto, ausentes os requisitos legais mencionados linhas atrás, INDEFIRO, neste momento, o pedido de antecipação de tutela de evidência postulado pelo(a) autor(a). Realize-se estudo social e perícia médica com urgência.Para tanto, nomeio a assistente social LUCILEI ARAÚJO GUERRA e formulo os seguintes quesitos: 1- Quais são os integrantes da família? 2- Qual é a renda familiar per capita? 3- Qual é a situação socioeconômica do(a) autor(a)? 4- Quais são as despesas mensais do(a) autor(a)? 5- O(A) autor(a) recebe ajuda de parentes ou filhos casados? 6- Existem parentes próximos e estes exercem atividade remunerada? 7- Quais as condições da habitação e quais os móveis que a guarnecem? 8- Existem veículos ou imóveis em nome de algum dos integrantes da família? 9- Algum dos integrantes da família recebe benefício do INSS ou de outro órgão assistencial? 10- Qual a profissão, os rendimentos, CPF/MF, data de nascimento dos integrantes do núcleo familiar? 11- Se residirem netos/sobrinhos com o(a) requerente, porque motivo, e o nome, profissão, rendimentos e qualificação de seus pais? 12- Outras considerações importantes para a apreciação do pedido do(a) requerente.Para a realização da perícia médica, nomeio o DR. Marcelo Aelton Cavaleti. Desde já formulo os seguintes quesitos: 1- O(a) requerente é portador de alguma deficiência? 2- Essa deficiência o(a) torna incapaz de cuidar de si mesmo(a), necessitando de auxílio de terceiros para vestir-se, alimentar-se e cuidar da higiene pessoal? 3- Quando se iniciou a doença e/ou a deficiência? 4- É temporária ou definitiva? 5- O(a) requerente tem condição de desempenhar alguma atividade? Intime-se o perito para designar data para realização da perícia, através de email, o qual poderá retirar os autos em carga pelo tempo necessário para a análise e confecção do laudo. Designada a data, intime-se o(a) requerente para comparecimento, consignando-se que deverá apresentar todas as receitas, exames, declarações e outros documentos médicos que possam interessar à perícia. Arbitro desde logo os honorários da assistente social em R$ 248,53 e do médico nomeado em R$ 400,00, nos termos da Resolução nº 541/2007, de 18/01/2007, do Conselho da Justiça Federal. Com a juntada do laudo expeçam-se certidões.Cite-se o INSS na pessoa de seu procurador, devendo este providenciar a juntada de CNIS e HIPNET em nome do(a) requerente e seu cônjuge/companheiro, se necessário.Deixo de designar audiência de conciliação visando conferir maior efetividade à tutela do direito visto que o Procurador do INSS não comparece às audiências nesta comarca.Int.", {
        'entities': [(2638, 2716, 'VALOR')]
    }),

    ("Expeça-se certidão de honorários (R$654,75 - cód 203) e arquivem-se os autos. Intimem-se. Ciência ao Ministério Público.", {
        'entities': [(11, 44, 'VALOR')]
    }),

    ("Banco Pecunia S.a - Nos termos do artigo 475-A, §1º do CPC, fica o(a) advogado(a) do(a) executado(a) intimado(a) da liquidação da sentença no importe de R$446,21, para efetuar o pagamento, no prazo de 15 dias, sob pena de multa (art. 475-J) no percentual de 10%, com expedição de mandado de penhora e avaliação.", {
        'entities': [(117, 163, 'VALOR')]
    }),

    ("Fls. 168 - Sentença nº 1266/2013 registrada em 09/10/2013 no livro nº 128 às Fls. 65: Processo nº 115/12 - Cartório Cível Vistos. Face ao que consta às fls. 161 e 165 e a inércia do patrono da autora certificada às fls. 167, final, julgo extinto o feito com fulcro no artigo 794, inciso I do Código de Processo Civil. Fixo os honorários advocatícios à advogada de fls. 14, em R$ 435,39 (cód. 206). Com o trânsito em julgado, expeça-se certidão e arquivem-se os autos com as anotações de praxe e estilo. P.R.I.", {
        'entities': [(327, 386, 'VALOR')]
    }),

    ("Disbraban Distribuidora Ltda - Vistos. Trata-se de ação ordinária que REDE UNILOJAS COMÉRCIO DE PRODUTOS PARA SUPERMERCADO LTDA ME move contra DISBRABAN DISTRIBUIDORA LTDA, pretendendo, em suma, a declaração de inexigibilidade de títulos e reparação por danos morais, sob o argumento de que desconhece a origem dos títulos protestados pela ré, eis que não possuem lastro fiscal ou aceite aptos a validar a duplicata em comento. Em apenso processou-se a cautelar de sustação de protesto, deferindo-se a liminar. O réu contestou ambas as ações afirmando que a autora integra uma rede de lojas com vários nomes fantasias, às quais foram entregues as mercadorias fornecidas pelo réu, mas que os carimbos de recebimento eram sempre da empresa denominada “Empório Malaquias”. Aduziu que era de praxe a entrega de mercadorias durante todo o mês, com emissão de apenas uma nota fiscal do total após o fechamento, mas a requerente na intensão de lesionar o réu, não lançou o aceite. Pugnou pela improcedência da ação. Réplica pelo autor. As partes especificaram provas. O feito foi saneado, sendo deferida a produção de prova oral. Em audiência de instrução e julgamento foi declarada encerrada a instrução, após a oitiva da testemunha arrolada pelo réu. Este é o Relatório, DECIDO: Tratam-se de ações cautelar de sustação de protesto e principal declaratória de inexigibilidade de débito c.c. indenização por danos morais. Ambas as ações são improcedentes. Alega o autor, em síntese, que não manteve relação jurídica com a ré, mas que esta protestou os títulos descritos na inicial, de forma ilegítima, o que lhe causou danos morais. Em suma, a despeito das razões iniciais, não é o que se verifica nos autos, porque restou demonstrada a relação jurídica existente entre as partes; a entrega das mercadorias; a rede de lojas a que pertence a autora e, por fim, o inadimplemento, o que é causa subjacente aos títulos de crédito (duplicatas) emitidos pelo réu. Os documentos carreados aos autos pelo réu traduzem uma séria de negociatas que foram travadas entre as partes para entrega de mercadorias e recebimento de valores, através de relatórios quinzenais, onde ao final, era emitida duplicata com valor total do período. Contata-se que este comportamento comercial vem sendo mantido pelas partes de longa data (desde 2011). A testemunha arrolada pelo réu afirmou que trabalhava na parte financeira do Supermercado Itaquerão quando foi feito o acordo para o pagamento da ré. Disse que o Itaquerão era o responsável por fazer o pagamentos dos fornecedores. Afirmou ainda a existência de vários supermercados com nomes diferentes. Afirmou que as mercadorias foram entregues pela ré e por isso o pagamento é devido. O débito acumulado pelo autor perante a ré, em razão da constante entrega de mercadorias, se convolaria em crédito à disposição da ré para fins de emissão de duplicata mercantil. Em verificar o volume do débito acumulado pela autora, contudo, o acordo não se aperfeiçoou. Todo esse roteiro é narrado pela ré foi confirmado pela testemunha. A procedência do pedido, em outros termos, enfrenta como barreira intransponível a manutenção do estado de inadimplemento que se buscou reverter com a confusão dos estabelecimentos comerciais que pertencem a uma mesma rede. As mercadorias entregues a uma, e o recebimento, pagamento e tratativas, feitos a outras. Os documentos trazidos com a defesa e o depoimento da testemunhas, atestam que houve a relação jurídica material entre as partes, com a entrega das mercadorias, o que reflete dados suficientes à formação do convencimento de que os protestos são legítimos, ante o inadimplemento do autor em face da obrigação. Diante do exposto, JULGO IMPROCEDENTES as ações principal e cautelar, ambos nos termos do artigo 269, I, do Código de Processo Civil. Por consequência, revogo a liminar concedida. Oficie-se aos cartórios de protesto, comunicando o teor da presente decisão e para que procedam aos protestos, de forma definitiva. Condeno ainda o autor ao pagamento das custas e despesas processuais, além de honorários advocatícios, que fixo em 20% do valor da causa principal, para ambas as ações. P.R.I. (As custas por fase de apelação importam o valor de : R$ 96,85 - GARE cód. 230. Taxa de porte por volume R$ 29,50 X 01 = R$ 29,50, na guia de recolhimento Fundo Especial de Despesas do TJFEDTJ - cód. 110-4 (OBS: EM CASO DE NÃO RECOLHIMENTO DO PREPARO DO RECURSO NO PRAZO LEGAL, SUJEITO À PENA DE DESERÇÃO DO RECURSO).", {
        'entities': [(4190, 4220, 'VALOR'), (4238, 4288, 'VALOR')]
    }),

    ("Complemente a exequente a diligência do Sr. Oficial de Justiça (R$33,90).", {
        'entities': [(65, 72, 'VALOR')]
    }),

    ("Maria Isabel Mendes Marçura - Sueli Moreira Miglioranza - Vistos. Iniciada a execução do título judicial consistente na sentença condenatória de fls. 58/60, realizou-se a penhora de bens do(a) devedor(a) para garantia da execução(fls.100). Entrementes, o(a) exequente não promove o regular andamento da execução. Por outro lado a constrição sobre os bens do(a) devedor(a) não poderá perdurar por prazo indeterminado e sem objetivo, mormente se tal ocorre em razão da indolência do(a) credor(a), que demonstra menoscabo para com a execução, motivo pelo qual determino o levantamento da penhora de fls. 100. Providencie-se o necessário. Nos termos do que dispõe o Enunciado 75 do Fórum Permanente de Juízes Coordenadores dos Juizados Especiais Cíveis e Criminais do Brasil, atualizado até o XIII Encontro, realizado em Campo Grande/ MS, em junho de 2003, “a hipótese do § 4o, do artigo 53, da lei 9.099/95, também se aplica às execuções de título judicial, entregando-se ao exeqüente, no caso, certidão de seu crédito como título para futura execução, sem prejuízo da manutenção do nome do executado no Cartório do Distribuidor”. Ante o exposto, em razão da ausência de bens penhoráveis de propriedade do(a) devedor(a), JULGO EXTINTA a presente execução que José Milton Marçura, Maria Isabel Mendes Marçura move em face de Sueli Moreira Miglioranza, com fundamento no artigo 53, § 4o, da Lei nº 9.099/95. Procedi ao desbloqueio do valor de R$ 5,96, bloqueado às fls. 111. Arbitro os honorários ao(à) advogado(a) nomeado(a) às fls. 52/53 em R$ 222,29, nos termos da tabela vigente. Expeça-se certidão. Ao Contador, para elaboração de cálculo de atualização do valor do débito. Após, expeça- se em favor do(a) credor(a) certidão de seu crédito, intimando-se-o(a) para retirar o documento. Na mesma oportunidade, deverá ser o(a) exequente intimado(a) de que, nos termos do item 30.2 do Provimento 1.670/2009, do E. Conselho Superior da Magistratura, os autos serão destruídos após o decurso do prazo de 90 (noventa) dias contados da extinção da execução, facultando-lhe a retirada de documentos, o que, desde já, fica autorizado. Comunique-se a extinção. P. R. I. C. (Certidão de crédito expedida, retirar em cartório.).", {
        'entities': [(1427, 1446, 'VALOR'), (1482, 1549, 'VALOR')]
    }),

    ("Seller - MNT Magazine Ltda - Trata-se de ação cautelar de exibição de documentos que ADILSON FRANCISCO BATISTA move em face de SELLER - MNT MAGAZINE LTDA, alegando que teve seu nome indevidamente inserido nos cadastros de proteção ao crédito. Afirma que a ré se recusa a entregar o documento que comprovaria a existência da dívida. Requer, liminarmente, a exibição das cópias autenticadas dos contratos e documentos que comprovem a exigibilidade e plausibilidade do dos débitos apontados no SERASA/SPC. Por fim, pleiteia os benefícios da Assistência Judiciária Gratuita.Foram concedidos os benefícios da Justiça Gratuita (fl. 17).Devidamente citada (fl. 25), a ré deixou de contestar, cf. certidão de fls. 27.RELATEI. DECIDO. A causa comporta julgamento no estado em que se encontra, não sendo útil a produção de outras provas para a análise do mérito, já que com a revelia presumem-se verdadeiros os fatos alegados pelo autor que, ademais, vêm corroborados pelo demonstrativo do SCPC e pedido administrativo juntado aos autos.O réu, pois, de forma inconteste, deixou de apresentar documentos necessários à comprovação da relação entre autor e ré.Isto posto, JULGO PROCEDENTE o pedido do autor, CONDENANDO a ré a exibir os documentos referentes ao negócio jurídico que originara a negativação, no prazo de 15 dias, sob pena de multa diária de R$ 100,00, com fundamento no parágrafo único do artigo 400 do CPC, observando-se que, em se tratando de medida preparatória, não há como se admitir a veracidade de fatos prevista do caput do mesmo artigo. Sucumbente, o réu arcará com as custas, despesas processuais e honorários advocatícios do patrono do autor, que fixo, por equidade, em R$500,00, com correção monetária e juros moratórios contados a partir do trânsito em julgado desta sentença, considerando que não houve condenação", {
            'entities': [(1328, 1353, 'VALOR'), (1612, 1693, 'VALOR')]
    }),

    ("COMPANHIA PAULISTA DE FORÇA E LUZ - Vistos. Presentes os requisitos legais defiro a liminar para que a ré restabeleça o fornecimento da energia no ponto da autora, em 24 horas, sob pena de multa diária de R$10.000,00, oficiando-se. Cite-se Int.", {
        'entities': [(190, 217, 'VALOR')]
    }),

    ("Fls. 31 - CERTIDÃO Certifico e dou fé de que decorreu o prazo legal, sem que houvesse manifestação por parte do(a) exequente, apesar de regularmente intimado(a). Sorocaba, 19/02/2013. Eu, subscr. Clelia Regina M. Rosa Chefe de Seção Judiciário - Matr. 809.955-1 CONCLUSÃO Em 19/02/2013, faço estes autos cls. ao MM. Juiz de Direito da 2ª Vara do Juizado Especial Cível de Sorocaba, Dr. DOUGLAS AUGUSTO DOS SANTOS. Clélia Regina Marchi Rosa Chefe de Seção Judiciário - Matric. 809.955-1-(Cl) Proc. n.º 1507/11 Exequente: FABIANA APARECIDA DE SOUZA Executado(a): HELEN VIRGINIE DOS S. ARAUJO Vistos, etc... Julgo extinto o presente feito, nos termos do artigo 53, § 4º, da Lei n.º 9.099/95, em face da inexistência de bens de propriedade da parte executada. Autorizo o desentranhamento dos documentos que instruíram a inicial, em favor da parte exeqüente, mediante recibo nos autos. Arquivem-se os autos, observando-se que a destruição dos autos poderá ser feita depois de decorridos noventa dias do arquivamento (por extensão do item 30.2 da Subseção VIII, art. 3º, Seção V, Cap. IV das NSCGJ, com redação conforme Provimento 1679/2009), prazo em que os interessados poderão pedir a restituição de documentos. Em caso de recurso, o valor do preparo é de R$ 193,70. P.R.I.C. Sorocaba, data supra. DOUGLAS AUGUSTO DOS SANTOS JUIZ DE DIREITO DATA Na data supra, recebi estes autos em cartório com a r. sentença supra. José Luiz Garcia Coordenador - Matr. 312.502-0", {
        'entities': [(1229, 1264, 'VALOR')]
    }),

    ("Qualicorp Administradora de Benefícios S/A e outro - VISTOS. LUANA MEDEIROS MATHEO DE MORAES move a presente ação cominatória c.c. indenizatória contra QUALICORP ADMINISTRADORA DE BENEFÍCIOS S/A e UNIMED PAULISTANA, aduzindo, em síntese, que era beneficiária do plano de saúde da UNIMED de Santa Catarina e, em agosto de 2011, quando se mudou para São Paulo, migrou para a UNIMED Paulistana, com compra de carência junto à administradora QUALICORP. Ocorre que, em novembro de 2011, teve a notícia que estava grávida, mas, quando foi agendar exames, foi surpreendida com a informação sobre a carência de dez meses para parto. Requer a antecipação dos efeitos da tutela, para que as rés sejam compelidas a prestar os serviços de natureza hospitalar relativos ao parto iminente da requerente, bem como a condenação das rés a pagar-lhe indenização por danos morais (fls. 2/13). Deferida a antecipação dos efeitos da tutela (fls. 37), as rés foram citadas e ofereceram contestações individuais. A corré UNIMED PAULISTANA ofertou resposta a fls. 128/156, pugnando pela improcedência do pedido, sustentando, em resumo, a legalidade de sua conduta e a validade da carência para parto. A corré QUALICORP ADMINISTRADORA DE BENEFÍCIOS S/A ofereceu contestação a fls. 222/235, aduzindo, preliminarmente, sua ilegitimidade passiva “ad causam”. No mérito, pugnou pela improcedência do pedido, reiterando os argumentos expostos pela corré Unimed. Réplica a fls. 309/313. A audiência de tentativa de conciliação das partes resultou infrutífera (fls. 322). É O RELATÓRIO. FUNDAMENTO E DECIDO. Sendo desnecessária a produção de provas em audiência, passo ao julgamento antecipado da lide, na forma do artigo 330, I do Código de Processo Civil. Inicialmente, indefiro a preliminar de ilegitimidade passiva “ad causam” arguida pela corré QUALICORP, considerando que, como administradora do plano de que é beneficiária a autora, responde por eventuais falhas na prestação dos serviços no que concerne à suposta inobservância das compras de carência quando da migração do plano da autora. No mérito, impositiva a procedência dos pedidos. É dos autos que a autora era beneficiária de plano de saúde junto à Unimed de Santa Catarina, desde 2 de julho de 2008. Em agosto de 2011, a autora se mudou para São Paulo, migrando para o plano da Unimed Paulistana, com compra de carências relativa ao plano anterior. Ocorre que as saber da notícia de sua gravidez em novembro de 2011, a autora, ao solicitar exames, veio a ser surpreendida com a alegação da existência de carência de dez meses para parto. As alegações da autora na inicial, quanto à abusividade da exigência de carência para parto, encontram respaldo nos autos. De efeito, a carta de compra de carências acostada a fls. 23 não faz qualquer ressalva quando à existência de novo cumprimento de carência para parto. E mais, a autora solicitou a migração entre UNIMEDs intrafederativas, de modo que não fazia qualquer sentido ela cumprir novamente carências que já havia cumprido no contrato anterior do qual migrou. Portanto, a exigência de carência para parto realizada pelas rés foi abusiva, devendo elas arcar com o tratamento médico e hospital relativos ao parto da autora, tornando-se definitiva a tutela de urgência concedida a fls. 37. Os danos morais também procedem. Em virtude da falha na prestação dos serviços da requerida, que exigiram novo cumprimento de carência já cumprida no plano anterior, a autora, na 33ª semana de gestação, teve que ajuizar a presente ação, com todos os ônus e gastos que tal providência acarreta, para o fim de serem as rés compelidas a arcar com as despesas do parto. Tais circunstâncias trouxeram insegurança, preocupação e aborrecimentos em momento especial da vida da requerente, quando deveria concentrar-se e tranquilizar-se para o parto que se avizinhava. A autora foi submetida, assim, por conta da conduta manifestamente abusiva das rés, a sofrimento, tensão e intranquilidade de espírito em momento muito particular de sua vida. Exatamente naquele delicado momento em que a pessoa deve concentrar-se, e reunir suas forças, a autora foi comunicada que não teria o respaldo do prestador de serviços. Não se olvida, ainda, que danos morais são caracterizados justamente pela privação ou diminuição de valores precípuos na vida das pessoas, como paz, tranquilidade de espírito, liberdade individual, integridade física e honra, entre outros, devidamente aferidos no caso, como ressalvado nos parágrafos anteriores. Feitas tais considerações, passo à fixação do montante devido. O arbitramento da condenação a título de dano moral deve operar-se com moderação, proporcionalmente ao grau de culpa, ao porte empresarial/pessoal das partes, suas atividades comerciais e, ainda, o valor do negócio, orientando-se o juiz pelos critérios sugeridos pela doutrina e jurisprudência, com razoabilidade, valendo-se da experiência e do bom senso, atento à realidade da vida, notadamente, à situação econômica atual e às peculiaridades do caso concreto. Considerando os elementos acima discriminados, bem como as circunstâncias em que foi negada a cobertura da cirurgia, entendo razoável a fixação do dano moral sofrido pela requerente na quantia de R$ 10.000,00. Valor inferior certamente em nada puniria a conduta lesiva, considerado o porte empresarial, ostensiva publicidade e grande número de associados da ré, sempre com vistas à denominada “Teoria do Desestímulo”. Neste sentido: “INDENIZAÇÃO. DANO MORAL. ARBITRAMENTO. CRITÉRIO. JUÍZO PRUDENCIAL. A indenização por dano moral é arbitrável, mediante estimativa prudencial que leva em conta a necessidade de, com a quantia, satisfazer a dor da vítima e dissuadir, de igual e novo atentado, o autor da ofensa” (Apelação Cível nº 198.945-!, 2ª Câmara do E. TJSP, rel. Des. César Peluzo, j. 21.12.93, JTJ 156/96). Feitas todas estas considerações e parâmetros, tenho como justa entre as partes a fixação do dano moral sofrido pela autora em R$ 10.000,00. POSTO ISSO e considerando o que mais dos autos consta, JULGO PROCEDENTE o pedido, para o fim de compelir as rés a arcar com a cobertura do tratamento médico e hospital relativos ao parto da autora, confirmando a tutela de urgência de fls. 37 e condenando as rés a pagar à autora, a título de indenização por danos morais, a importância de R$ 10.000,00, corrigida monetariamente e acrescida de juros moratórios legais de 1% ao mês, tudo contado a partir do arbitramento, conforme recente entendimento do C. STJ. Sucumbente, arcarão as rés com as custas e despesas processuais, assim como com os honorários advocatícios da parte contrária, estes ora fixados em 20% do valor atualizado da condenação, na forma do artigo 20, §3º do Código de Processo Civil. P.R.I.C. - CERTIDÃO DE PREPARO: Certifico e dou fé que o valor do preparo em eventual apelação, importa em R$ 96,85, e a taxa referente ao porte e remessa de autos é de R$ 1.053,20 por volume ( 02 volume(s)).", {
        'entities': [(5201, 5228, 'VALOR'), (5912, 5972, 'VALOR'), (6253, 6325, 'VALOR'), (6823, 6842, 'VALOR'), (6874, 6907, 'VALOR')]
    }),

    ("Vistos.Fls. 596/830: ante o disposto no Comunicado Nº 564/2016, da Corregedoria Geral de Justiça do TJ/ SP, não há a possibilidade, por ora, de cadastramento no sistema de incidente de desconsideração da personalidade jurídica, em que pese a nova disciplina processual sobre o tema com relação aos casos em que o requerimento não é articulado na inicial (CPC, arts. 133 a 137), razão pela qual o pedido de desconsideração ora formulado deverá ser processado nestes próprios autos.Diante dos fatos narrados pela requerente, apontado para o abuso da personalidade jurídica da empresa-executada, há, em cognição sumária, evidências tanto de confusão patrimonial envolvendo os sócios e as empresas que comporiam um grupo econômico, quanto do desvio de finalidade da empresa-executada (que, ainda, teria sido dissolvida irregularmente, o que pode explicar a informação de fls. 353 de “inexistência de relacionamentos” com as instituições financeiras) e das demais empresas de tal grupo. Ademais, diante dos abusos noticiados, há, por certo, risco ao resultado útil do processo, ainda mais quando os elementos trazidos indicam que se encontram em curso outras execuções em face dos requeridos, e que tem sido enfrentadas dificuldades para a localização destes (o que, inclusive, restou constatado nestes autos, sendo a empresa-executada intimada por edital, conforme fls. 332).Dessa forma, presentes os requisitos legais para a sua concessão (art. 300 do CPC), defiro parcialmente a tutela de urgência de natureza cautelar requerida, determinando, como primeira medida, a tentativa de arresto, via Bacen-jud, dos valores existentes em contas-correntes ou aplicações financeiras dos sócios (descritos a fls 596/597), até o valor do débito indicado a fls. 608.Valores considerados irrisórios deverão ser imediatamente desbloqueados.No caso de efetivação do arresto, os valores deverão permanecer bloqueados, aguardando-se a necessária intimação dos requeridos - não havendo, pois, qualquer risco de irreversibildade dos efeitos desta decisão.Recolha a requerente, no prazo de cinco dias, a taxa judiciária devida para a realização de tal medida (R$ 12,20 por CPF/CNPJ e por pedido), assim como as custas necessárias à citação dos requeridos por carta.Após, citem-se e intimem-se os sócios indicados as fls. 596/597, para que se manifestem e requeiram as provas cabíveis no prazo de 15 dias, na forma do art. 135 do Código de Processo Civil.Anotem-se os nomes dos sócios no cadastro das partes no sistema, para o regular reflexo no Distribuidor, sem necessidade de expedição de ofício ou remessa do processo ao referido setor (Comunicado CG Nº 564/2016, item 5).Fica determinada a suspensão do presente feito até que resolvido o pedido de desconsideração da personalidade jurídica, na forma do §3º, do art. 134, do Código de Processo Civil.Int.", {
        'entities': [(2339, 2148, 'VALOR')]
    }),

    ("Covepi Empreendimentos Ltda - Pela presente fica a parte autora intimada para providenciar o recolhimento da taxa de publicação do edital nos termos do Provimento nº 1668/09 do CSM, com nova redação dada pelo Provimento nº 1758/10 do CSM, na guia do Fundo Especial de Despesa do TJSP, código 435-9, cujo valor deve corresponder a 0,15 quinze centavos) por caractere, observando que o edital expedido contém 1133 caracteres, nos termos do Comunicado nº 62/09 da Presidência do E. TJSP. ", {
        'entities': [(305, 335, 'VALOR')]
    }),

    ("Telefonica Brasil S/A. - Trata-se de ação declaratória de inexistência de débitos e indenização por danos morais ajuizada por MARCOS ROBERTO SILVÉRIO em face de TELEFÔNICA S.A.. Alega o autor, em síntese, que foi surpreendido com a inscrição de seu nome, pela ré, em cadastro de inadimplentes, em virtude do inadimplemento de débito. Ocorre que o autor não foi notificado do débito e o desconhece, não tendo mantido qualquer relação com a ré, razão pela qual entrou em contato com a ré, em várias oportunidades, mas não obteve solução para o problema. Considerando que não há qualquer débito a fundamentar a restrição do crédito, e que tal ato ilícito gerou profundo abalo do crédito de sua pessoa, bem como de sua moral, requer a declaração da inexistência do débito e a condenação da ré ao pagamento de indenização por danos morais, no valor de R$ 20.000,00. A tutela antecipada foi concedida. Citada, a ré ofertou resposta, na qual sustentou que entabulou o contrato em vista dos documentos aparentemente legítimos que lhe foram apresentados. Sustenta, ainda, que não concorreu com culpa para o evento, tendo agido de boa-fé e em exercício regular de direito. Por fim, afirma que não há prova de nexo causal e dano moral indenizável, razões pelas quais postula a improcedência da demanda. Subsidiariamente impugnou o quantum pretendido.Houve réplica.É o relatório. Decido. Versando a controvérsia sobre matéria passível apenas de prova documental, já produzida ou cuja oportunidade de produção foi atingida pela preclusão, passo ao conhecimento direto do pedido. A ação procede. Com efeito, alega a ré que não contribuiu com culpa para o evento, na medida em que habilitou a linha telefônica em vista da documentação apresentada a seu agente credenciado, de forma que agiu em exercício regular de direito ao cobrar os serviços prestados. Sem razão, contudo, a requerida. Em primeiro lugar porque a alegação que funda a contestação não contou com qualquer amparo probatório. De fato, note-se que a contestação não trouxe aos autos: (a) o alegado instrumento contratual; (b) cópias dos documentos em nome do autor que teriam sido utilizados para a obtenção da habilitação. Ora, nestas circunstâncias, não demonstrado sequer o fundamental da versão defensiva, não há como simplesmente acolhê-la em prejuízo do que está incontroverso nos autos “negativação” indevida do nome do autor. Em segundo lugar porque, ainda que tivesse sido provada a narrativa dos fatos contida na contestação, esta, por si só, não seria suficiente a excluir a contribuição culposa para o evento, na medida em que cabe ao agente fornecedor de produtos e serviços, antes de entabular o contrato, e, principalmente, de proceder às comunicações aos serviços de proteção ao crédito, certificar-se de que a pessoa a cujo nome está sendo imputada determinada inadimplência trata-se, realmente, daquela que contraiu e deixou de quitar o débito. Ao assim não proceder a ré contribuiu, de forma relevante e preponderante para o fato lesivo ao bom nome do autor. A isto se acrescenta que, na hipótese, incide a responsabilização objetiva (art. 14, § 3º, do CDC) ou independente de culpa, pois se trata de risco da atividade (art. 927, parágrafo único, do Código Civil), fortuito interno, cujos resultados negativos não podem ser suportados por terceiros estranhos à atividade e que não participam de suas vantagens. Neste sentido:DANO MORAL Banco de dados Inscrição indevida do nome do autor em órgão de proteção ao crédito Contrato de habilitação de linha telefônica, celebrado por telefone, por terceira pessoa e com a utilização ilícita dos dados pessoais do autor Responsabilidade objetiva da concessionária, à luz do artigo 37, § 6º, da Constituição Federal Culpa exclusiva do consumidor não comprovada Inteligência das disposições do art. 14, § 3º, II, do Código de Defesa do Consumidor Dever de indenizar inconteste, pois a obrigação de checar a veracidade e a fidedignidade dos dados fornecidos é da empresa concessionária, pois é a responsável pela segurança e eficiência do serviço Redução do montante indenitário determinada Recurso da empresa-ré parcialmente provido para este fim. (Apelação Cível com Revisão n. 975.521-0/7 São Paulo 35ª Câmara de Direito Privado Relator: Des. Clóvis Castelo 14.08.06 V.U. Voto n. 9.891) DANO MORAL Banco de Dados Inclusão do nome do autor como devedor em órgão de proteção ao crédito Contrato de habilitação de linha telefônica celebrado por terceiro com utilização ilícita de seus dados Ônus de comprovar ter o acionante contratado o serviço, a cargo da prestadora, não evidenciado Impossibilidade de exigir-se dele a prova negativa (a não celebração da avença) Gravame moral configurado Dever de indenizar inconteste, pois, aceitando a contratação sem maiores formalidades, deve a concessionária arcar com eventuais fraudes praticadas por terceiros, posto que, quem explora atividade econômica, responsabiliza-se pelas falhas em suas rotinas de realização de negócios “Quantum” indenitário inalterado Condenação da ré no pagamento da verba sucumbencial Recurso do autor parcialmente provido, improvido o da demandada. (Apelação Cível com Revisão n. 1.020.142-0/5 Conchal/Mogi Mirim 35ª Câmara de Direito Privado Relator: Des. Mendes Gomes 23.10.06 V.U. Voto n. 11.964)BANCO - Dano moral - Indenização Responsabilidade civil - Abertura de conta corrente por estelionatário, com uso de documentos falsos - Instituição bancária que agiu de maneira negligente, sem tomar as mínimas precauções a fim de verificar a autenticidade da documentação entregue quando da abertura da conta corrente - Verba devida (1º TACivSP) - RT 807/262 DANO MORAL - Banco de dados - Instituição bancária que abre conta-corrente a pedido de estelionatário, em nome de vítima que teve seus documentos subtraídos, mediante a simples apresentação da cédula de identidade do lesado, fornecendo talonário de cheques os quais passaram a ser emitidos e a retornar por ausência de fundos, negativando o nome daquele - Ausência das cautelas necessárias por parte do banco na abertura da conta - Negligência - Ocorrência - Indenização - Necessidade - Recurso do réu improvido. (Apelação n. 340.875-4/7 - São Paulo - 3ª Câmara de Direito Privado - Relator: Beretta da Silveira - 22.11.05 - V.U. - Voto n. 9.748)DANO MORAL - Responsabilidade civil - Abertura de conta corrente com dados falsificados por estelionatário - Encaminhamento e negativação do nome da vítima junto a bancos de dados de proteção ao crédito - Ocorrência de fortuito interno, que se incorpora ao risco da atividade bancária - Danos morais “in re ipsa” decorrentes da negativação - Fixação em valor equivalente a oitenta salários mínimos - Ação procedente em parte - Honorários do advogado do autor fixados sobre o valor da indenização, sem sucumbência recíproca, diante do caráter estimativo da inicial - Recurso do réu não provido - Recurso do autor provido em parte. (Apelação cível n. 309.370-4/5-00 - São Paulo - 4ª Câmara de Direito Privado - Relator: Francisco Loureiro - 01.12.05 - V.U. - Voto n. 622)DANO MORAL Responsabilidade civil Indenização - Abertura de conta corrente e contrato de mútuo com dados e documentos falsificados por estelionatário Encaminhamento e negativação do nome da vítima junto a bancos de dados de proteção ao crédito e tabelionatos de protesto Ocorrência de fortuito interno, que se incorpora ao risco da atividade bancária Danos morais “in re ipsa” decorrentes da negativação Fixação em valor equivalente a cinqüenta mil reais, rateados igualmente entre as duas instituições financeiras Ação procedente em parte Recursos improvidos. (Apelação Cível n. 447.504-4/5-00 São Paulo - 4ª Câmara de Direito Privado - Relator: Francisco Loureiro 14.09.06 - V.U. - Voto n. 1779)A culpa da ré pela indevida “negativação” do nome do autor perante as instituições de crédito, portanto, restou suficientemente demonstrada. Alega a ré, entretanto, que estes fatos não configuram dano moral indenizável, pois não trouxe o autor aos autos qualquer prova de que sofreu prejuízos ou lesão à sua honra por força do ato culposo. Sem razão, contudo, a ré. Isto porque o dano simplesmente moral, como o abalo do crédito e a lesão à honra, não necessita de outras provas, existindo apenas pela ofensa, e dela sendo presumido, o que, por si só, justifica a indenização. De fato, não há como se negar que se atribuir a alguém, injustamente, a qualidade de inadimplente, ofende a esfera íntima do indivíduo e causa dificuldades no seu cotidiano, o que merece a devida compensação. Consoante jurisprudência firmada no Superior Tribunal de Justiça, o dano moral decorre do próprio ato lesivo de inscrição indevida nos cadastros de proteção ao crédito, “independentemente da prova objetiva do abalo à honra e à reputação sofrido pelo autor, que se permite, na hipótese, facilmente presumir, gerando direito a ressarcimento” (Resp. 110.091/MG, Rel. Min. ALDIR PASSARINHO JUNIOR, DJ 28.08.00; Resp. 196.824, Rel. Min. CÉSAR ASFOR ROCHA, DJU 02.08.99; Resp. 323.356/SC, Rel. Min. ANTÔNIO PÁDUA RIBEIRO, DJ 11.06.2002)Assim, a pretensão inicial declaratória merece procedência. Considerando a capacidade econômica das partes, o valor indevidamente cobrado de pequena monta - e o tempo razoável pelo qual permaneceu a restrição indevida ao crédito, entendo suficiente para compensar o dano sofrido pelo autor o arbitramento de indenização no montante de R$ 10.000,00, que, consoante súmula n. 54, do Superior Tribunal de Justiça, será acrescido de juros moratórios de 1% ao mês desde o evento danoso data da inscrição indevida e correção monetária a partir desta data quando liquidado o dano. Ante o exposto, JULGO PROCEDENTE a ação para: DECLARAR A A INEXIGIBILIDADE DO DÉBITO apontado pela ré em relação ao autor nos Serviços de Proteção ao Crédito e, em consequência, determinar o seu cancelamento, tornando definitiva, assim, a tutela antecipada concedida; CONDENAR a ré ao pagamento de indenização ao autor no valor de R$ 10.000,00 (dez mil reais), que sofrerá correção monetária a partir desta data e a incidência de juros moratórios de 1% ao mês, por se tratar de ato ilícito, desde a inscrição indevida do nome do autor em serviço de proteção ao crédito; CONDENAR a ré ao pagamento das custas processuais e honorários advocatícios que arbitro em 10% do valor atualizado da condenação. Aplicável à hipótese a súmula n. 326, do STJ, não há de se falar em condenação do autor ao pagamento de verbas de sucumbência. Após o trânsito em julgado, aguarde-se em cartório por 20(vinte) dias. Nada sendo requerido, arquive-se. P.R.I.C.", {
        'entities': [(836, 861, 'VALOR'), (9391, 9418, 'VALOR'), (9964, 9989, 'VALOR')]
    }),

    ("SASSOM Serviço de Assistência à Saúde dos Munic. de Ribeirão Preto - POSTO ISSO e considerando o mais que consta dos autos, declaro incidenter tantum a inconstitucionalidade dos artigos 9º e 25, inciso “a”, ambos da Lei Complementar Municipal nº 441/95; julgo procedente o pedido para confirmar os efeitos da tutela deferida; determinar o desligamento da parte autora da condição de contribuinte do referido órgão; e declarar a inexigibilidade da contribuição ao SASSOM, determinando que o réu se abstenha de promover referidos descontos sobre os vencimentos percebidos pela parte autora. Diante da sucumbência, condeno o Sassom ao pagamento das custas e despesas processuais de reembolso, com a incidência de correção monetária desde cada desembolso, bem como dos honorários do advogado da parte autora que fixo em R$600,00, nos termos do art. 85, § 8º, do Código de Processo Civil, com a incidência de correção monetária, a partir desta data.Decorrido o prazo para interposição de recursos voluntários, subam os autos ao Egrégio Tribunal de Justiça do Estado de São Paulo para o reexame necessário, por se tratar de sentença ilíquida proferida contra a Fazenda Pública (nesse sentido: Resp 103.025-SP, Rel. Min. Ari Pargendler, j. 12.04.10; Súmula 490 STJ; Súmula 108 STJ).P. Intimem-se.", {
        'entities': [(805, 825, 'VALOR')]
    }),

    (" Marcia de Lourdes Zanetti Silvestre - Vistos. Providencie a parte ré o refazimento do cálculo de liquidação da sentença de fls. 90 a 92, posto que o apresentando à fl. 104 corrige monetariamente o valor dos honorários advocatícios a partir de outubro de 2011, ao passo que o valor fixado em sentença (R$ 724,00) deve ser corrigido a partir da prolação da decisão, ocorrida e publicada em novembro de 2014. Intimem-se. Mongaguá, ", {
        'entities': [(303, 312, 'VALOR')]
    }),

    ("Luiz Carlos Sacca - - Dulcelene da Costa Brito Sacca - Vistos. MARIA APARECIDA DOS SANTOS FRAIZ VASQUES, qualificada nos autos, ajuizou “Ação DE OBRIGAÇÃO DE FAZER C.C. PEDIDO LIMINAR E INDENIZAÇÃO POR DANOS MATERIAIS” em face de LUIZ CARLOS SACCÁ e DULCELENE DA COSTA BRITO SACCÁ, todos nos autos qualificados. Alegou a autora, em síntese, que é co-proprietária do imóvel residencial situado na Rua Ayda Baganha Ferreira, nº 1.176, matriculado sob nº 10.054 no R.I. de Garça, imóvel esse que se encontra em processo de inventário perante este juízo. Aduziu que, por sua vez, os réus são proprietários do terreno urbano constituído pelo lote 2, da quadra 40, na Vila Santa Terezinha, localizado na Avenida Martiniano Inácio Gonçalves, nº 730, matriculado no RI de Garça sob nº 13.499, que faz limite nos fundos com o imóvel da requerente. Acrescentou que os requeridos escavaram um enorme buraco junto ao muro de divisa, sem qualquer preocupação com a sua propriedade, deixando um barranco de aproximadamente 6 metros de altura, o que acabou por provocar a queda de seu muro divisório. Disse, ainda, que sua casa dista menos de dois metros do citado barranco, correndo riscos de desmoronamento, sendo necessária a construção de um muro de arrimo, de responsabilidade dos réus, que se negam a arcar com os custos da obra. Por essas razões, requereram antecipação de tutela, a fim de que os réus fossem compelidos à construção do mencionado muro de arrimo, sob pena de multa diária, e, ao final, a confirmação da liminar, condenando-se os réus, ainda, ao pagamento dos danos materiais do muro que desabou em função da indevida escavação por eles realizada. Indeferida antecipação de tutela (fls. 20/21). Regularmente citados (fls. 24), os réus apresentaram contestação (fls. 26/30), negando os fatos que lhe foram imputados na inicial. Asseveraram que a autora não residia no imóvel em questão e tampouco acompanhou sua construção, pois passou a nele residir somente a partir de meados de 2008. Afirmaram que o imóvel do qual a requerente é co-proprietária foi construído por sua genitora em terreno elevado (acima do nível da rua) e que eles jamais removeram terras do imóvel deles, que sempre foi em nível inferior, desde que o adquiriram. Salientaram que a extensão da divisa entre os dois imóveis é de cerca de 20 m e que, por acordo celebrado com a genitora da requerente, Sra. Nympha, ficou combinado que eles, requeridos, edificariam 10 m de muro, o que efetivamente fizeram, e que o restante (10 m) seria edificado pela mãe da requerente, a qual lamentavelmente não cumpriu a avença e tempos depois veio a falecer. Asseveraram, ainda, que a causa da queda do antigo muro foi a falta de manutenção e a ausência de canalização das águas pluviais do telhado do imóvel da própria requerente, que lança toda a água na encosta do barranco desde longa data, fato em relação ao qual sempre solicitaram providências à genitora da requerente e à ela própria, sem êxito, no entanto. Por fim, postularam a improcedência dos pedidos iniciais. Réplica (fls. 42/44). Instados à especificação de provas, a autora requereu a produção de prova pericial (fls. 47/48), enquanto que os réus pleitearam a prova técnica e também oral. Decisão saneadora (fls. 52). Laudo pericial juntado a fls. 68/72, sobre o qual se manifestam as partes (réus: fls. 77/78; autora: fls. 83/84). Na audiência de instrução e julgamento, a partes solicitaram a suspensão do processo pelo prazo de 60 (sessenta) dias para tentativa de composição, asseverando, ainda, que não tinham interesse na produção de outras provas (fls. 94). Deferida a suspensão do processo, e decorrido o prazo (fls. 95/verso), as partes não informam se houve composição (fls. 98). É o relatório. Fundamento e decido. Os pedidos deduzidos na inicial são improcedentes, uma vez que a parte autora não produziu prova apta a comprovar suas alegações, não se desincumbindo a contento do ônus que lhe competia (art. 333, I, do CPC). Conforme ensina Vicente Greco Filho, “o autor, na inicial afirma certos fatos porque deles pretendem determinada consequência de direito; estes são os fatos constitutivos que lhe incube provar sob pena de perder a demanda. A dúvida ou insuficiência de prova quanto a fato constitutivo milita contra o autor. O juiz julgará o pedido improcedente se o autor não provar suficientemente o fato constitutivo de seu direito.” (Direito Processual Civil, 2o volume, 11a edição, 1996, Saraiva, p. 204). No caso em tela, nada há nos autos comprovando que os autores realizaram escavação no imóvel de propriedade deles e que tal conduta teria provocado a queda do antigo muro que dividia os fundos das duas propriedades. Não bastasse isso, a genérica réplica apresentada pela parte autora não impugnou especificadamente fatos relevantes arguidos pelos réus em contestação, os quais restaram incontroversos, portanto. Nesse ponto, também vale mencionar lição de Vicente Grego Filho: “...na réplica o autor tem o ônus de impugnar, sob pena de ser presumido como aceito o fato impeditivo ou extintivo alegado, passando a situação a ser favorável ao réu. Com a alegação de fato dessa natureza se diz que o reus fit actor (o réu se torna autor).” (Direito Processual Civil Brasileiro, 2º volume, Editora Saraiva, 11ª edição, 1996, pág. 162). Sendo assim, restou incontroverso que os autores haviam entabulado acordo com a genitora da requerente, ex-proprietária do imóvel, para a meação da construção do muro de arrimo e que eles cumpriram a parte deles na avença. Para além da presunção de veracidade por falta de impugnação específica, tal assertiva é corroborada pelo laudo pericial (fls. 68/72), no qual se observa que a divisa de fato entre os dois imóveis é de 16,90 metros e que foi constatada a existência de um muro de arrimo de 7,70 metros, muro esse que, segundo informado pela autora, fora edificado pelo réu Luiz Carlos (fls. 70 resposta ao quesito 14). Ademais, referido laudo pericial dá conta de que: a) a casa de co-propriedade da autora foi edificada a apenas 1,00 metros da divisa com o imóvel dos requeridos e que não existe calha para escoamento de águas pluviais, que direciona as águas para o terreno dos requeridos (respostas aos quesitos 3, 5 e 6 fls. 69); b) a casa de co- propriedade da requerente foi edificada em nível superior à via pública (cerca de 1,50 metros), ao passo que a construção dos requeridos encontra-se, na maior parte, ao nível da outra rua (respostas aos quesitos 11 e 12 - fls. 70), o que confere verossimilhança à alegação dos réus de que o antigo muro de divisa não desmoronou por conta de escavação por eles realizada, mas sim por falta de manutenção da requerente e de sua genitora, bem como pela ausência da devida canalização das águas pluviais do imóvel da própria parte autora. Diante desse quadro, não se afigura razoável compelir os réus à construção da parte restante do muro de arrimo e tampouco à indenização postulada, porquanto, segundo os elementos de convicção constantes dos autos, eles já construíram a partes que lhes cabia, nos termos do art. 1297 do Código Civil, e não há provas de que deram causa ao desmoronamento do antigo muro divisório. Ante o exposto e pelo mais que dos autos consta, nos termos do art. 269, inciso I, do CPC, JULGO IMPROCEDENTES os pedidos deduzidos na inicial da presente ação movida por MARIA APARECIDA DOS SANTOS FRAIZ VASQUES em face de LUIZ CARLOS SACCÁ e DULCELENE DA COSTA BRITO SACCÁ, todos nos autos qualificados. Em razão da sucumbência, condeno a autora ao pagamento das custas, despesas processuais e honorários advocatícios do patrono da parte contrária, estes últimos arbitrados, com fundamento no art. 20, § 4º, do Código de Processo Civil, em R$ 700,00 (setecentos reais), devendo ser observado, no entanto, o disposto no art. 12 da Lei nº 1.060/50, pois beneficiária da justiça gratuita. P. R. I.", {
        'entities': [(7558, 7714, 'VALOR')]
    }),

    ("Epp - Cuida-se de ação declaratória de inexistência de débitos com indenização por perdas e danos, anulatória de títulos c/c pedido de liminar de sustação de protesto e negativação, ajuizada por Cromar Cromados e Aramados em face de HDLG FIDC MULTISETORIAL, pelo motivos expostos na inicial.Concedo a autora o prazo de 15 (quinze) dias úteis para o pagamento da taxa judiciária (R$ 223,74 guia DARE/SP código 230-6) e taxa de mandato(R$ 17,60 guia DARE/SP código 304-9), bem como das despesas postais (R$ 15,50 guia FEDTJ código 120-1) sob pena de cancelamento da distribuição e indeferimento da inicial.Sem prejuízo, promova-se os apensamentos das ações cautelares de nºs. 1000084-92.2016 e 1000115- 15.2016, envolvendo as mesmas partes. ", {
        'entities': [(350, 389, 'VALOR'),(419, 443, 'VALOR'),(485, 511, 'VALOR')]
    }),

    ("Inss Instituto Nacional de Seguridade Social - Rejeito a impugnação de fls. 392/397, posto que os juros moratórios e a atualização monetária foram corretamente computados no cálculo diferencial de fls. 389/390 elaborado pelo exequente não incidindo os primeiros períodos no período compreendido entre a data de expedição do precatório e a do efetivo pagamento (Rec. Extraordinário nº 298.616, de 31/10/02) e apurada a segunda pela aplicação do IPCA-E para pagamento no orçamento do exercício seguinte em conformidade com a legislação orçamentária vigente. Fica, pois, rejeitada a impugnação em tela e, por via de consequência acolhido o cálculo diferencial de fls. 389/390 que apurou o valor de R$ 15.593,77. Expeça-se o oficio requisitório, na forma do artigo 730, I, do C.P.C. Intime-se. ", {
        'entities': [(685, 709, 'VALOR')]
    }),

    ("Porto Seguro Companhia de Seguros Gerais - DIRETORIA TÉCNICA SETOR DE PERICIAS DE SANTOS - *Vistos.Trata-se de ação de cobrança de indenização de seguro DPVAT.A autora sofreu lesão corporal grave em acidente de trânsito ocorrido em 1º.7.2013, e recebeu administrativamente a quantia de R$ 1.687,50 valor bem inferior ao que determina a Lei n.º 6.194/74, que é de R$ 13.500,00. Pretende, assim, a procedência do pedido, para o pagamento da diferença entre o valor pago e o que é estabelecido pela Lei.A ré oferece contestação, alegando que efetuou o pagamento no valor de R$ 1.687,50 observada a legislação aplicável à época do sinistro. Pugna pela improcedência da demanda devido ao pagamento já realizado.Réplica regular.Saneador deferindo a produção de prova pericial.Prova produzida. Em manifestação sobre o laudo, a autora alega que o laudo apresentado pelo perito comprova sua invalidez permanente; a ré alega que a parte autora já recebeu administrativamente o valor de R$ 1.687,50, vigente à época do sinistro e, portanto não tem direito a receber qualquer diferença de indenização. Requer a improcedência da ação.Relatei.Decido.A prova necessária à solução da lide se reduz seguramente à prova pericial, não havendo minimamente a necessidade de produção de qualquer outro meio.Portanto, é caso de prolação da sentença desde já, eis que foi oportunizada a manifestação das partes sobre o laudo.Para a procedência do pedido devem concorrer dois fatores: o nexo de causalidade; a lesão em grau a ensejar a indenização.Pois bem. Extrai-se do laudo pericial que: “Diante do exposto conclui-se que: Há nexo de causalidade entre o ferimento e o acidente. Há dano patrimonial físico sequelar estimado em 2,5% em analogia com a Tabela do DPVAT. Não há incapacidade laboral.”A autora já recebeu administrativamente valor equivalente apurado pela ré, portanto, o presente pedido não prospera, eis que inexistente diferença a ser paga. Aliás, a autora recebeu valor superior a esse apurado pelo perito.A propósito, o STJ editou a súm. 474 com o seguinte teor: “A indenização do seguro DPVAT, em caso de invalidez parcial do beneficiário, será paga de forma proporcional ao grau de invalidez”.Desse modo, não existe diferença a ser recomposta. É o que basta à solução da lide, não havendo a necessidade de nenhuma outra providência.Assim, JULGO IMPROCEDENTE o pedido.Condeno a parte vencida, por conseguinte, ao pagamento das custas, das despesas processuais e de honorários advocatícios de dez por cento do valor corrigido dado à causa, observando-se os §§ 2º e 3º do art. 98 do NCPC. Quanto a recurso, agora o controle total acerca do cabimento e da admissibilidade é do Tribunal, incumbindo ao recorrente, em relação ao preparo, observar a Lei estadual n. 11.608/03 (com os acréscimos dados pela Lei n. 15.855/15), quer no que pertine à base de cálculo quer no que pertine à alíquota (ou, ainda, a valor máximo de recolhimento ou a valor mínimo). O controle em relação ao preparo igualmente, pois, é do Tribunal com exclusividade, não competindo a este juízo nenhuma providência a respeito, ainda que seja preparatória. Caberá à parte recorrente, por seu advogado, quando o preparo for devido, realizar a conta e proceder ao recolhimento, comprovando-o no ato da interposição do recurso; se o relator, em juízo de admissibilidade, quando o recurso chegar a ele, decidir pela insuficiência ou pela incidência, em caso de inexistência, abrirá prazo para a complementação ou para a realização - em dobro. A propósito, remete-se aos arts. 1.007 e 1.010 do NCPC. Dito de outro modo, este juízo apenas processará, mecanicamente, o recurso, competindo qualquer decisão ao relator. Sequer análise acerca de gratuidade de justiça competirá a este juízo nesse estágio pós-sentença (art. 99, § 7º).P.R.I.C. (quando estiver em termos, independentemente de despacho, certifique-se e adote-se a providência pelo arquivamento).Santos, 19 de abril de 2016.JOSÉ WILSON GONÇALVES Juiz de Direito", {
        'entities': [(276, 299, 'VALOR'), (560, 583, 'VALOR'),(966, 989, 'VALOR')]
    }),

    ("Deposite a parte ré o valor do saldo devedor apurado pela Contadoria (R$ 409,50 - setembro/2014 - fls. 150), no prazo de cinco (5) dias, sob pena de penhora.", {
        'entities': [(71, 80, 'VALOR')]
    }),

    ("Ante o exposto, nos termos do inciso VI, do artigo 267, do Código de Processo Civil, JULGO EXTINTO o pedido, sem a resolução de mérito. Não há que se falar em condenação de sucumbência. Fica desde já autorizado o desentranhamento dos documentos que instruíram a presente ação para entrega à parte autora, que deverá providenciar as cópias necessárias para a substituição. P.R.I.C. Após as cautelas de praxe, arquivem-se. [PREPARO DA APELAÇÃO E DO RECURSO ADESIVO - Ao Estado: valor singelo R$100,70; Ao Estado: valor corrigido R$100,70 (Guia Gare/Dare - Código 230-6); Ao F.E.D.T.J.:Porte de remessa e do retorno dos autos R$32,70 (01) volume (Guia F.E.D.T.J - Código 110-4)]. ", {
        'entities': [(477, 500, 'VALOR'), (512, 536, 'VALOR'), (593, 632, 'VALOR')]
    }),

    ("Fls. 176/181 - Vistos. Sebastião Lima da Silva, qualificado nos autos, ajuizou a presente ação para a concessão de aposentadoria por tempo de contribuição em face do Instituto Nacional do Seguro Social ? INSS alegando, em síntese que nasceu em 28/02/1950 e desde criança laborava em lides rurais juntamente com sua família até o ano de 1973. O requerente asseverou que iniciou o labor urbano aos 14/02/1973, com registro em sua CTPS e, desde 01/12/1999 até os dias atuais contribui como contribuinte individual para a autarquia previdenciária. Requereu a conversão de tempo especial para comum nos períodos de 01/07/1989 a 07/08/1994 e 01/09/1997 a 17/01/1998, vez que laborou na função especial, penosa e insalubre de guarda noturno e vigilante noturno (fls. 02/10). A inicial juntou documentos (fls. 11/62). Decisão indeferindo os benefícios de assistência judiciária gratuita (fls. 86). Agravo de instrumento (fls. 89/101) o qual foi dado provimento (fls.104/105). Decisão indeferindo o pedido de antecipação dos efeitos da tutela e concedendo os benefícios da assistência judiciária gratuita (fls. 107). Juntada dos extratos do CNIS (fls. 113/115). Citado, o requerido apresentou contestação, aduzindo em síntese que as provas materiais as quais foram juntadas aos autos não podem ser consideradas como comprovação do labor rural, vez que não provam o início e o fim do efetivo exercício das atividades campesinas, aduziu também a impossibilidade de conversão de tempo especial para comum após 28/05/1998. Asseverou que é impossível promover-se a conversão pleiteada pela atividade de vigia noturno em empresa particular, vez que somente é considerada especial atividades vinculadas a empresas especializadas em vigilância. Alegou que o tempo de contribuição está muito aquém do exigido pela Constituição e pela Lei n° 8.213/91. Por fim, requereu a improcedência do pedido (fls. 121/143). A contestação juntou documentos (fls. 144/150). Instadas as partes a especificarem provas que pretendem produzir, a parte autora requereu a produção de prova testemunhal (fls. 155). A autarquia ré pugnou para a não ordenação da produção da perícia técnica, pois, como o labor foi prestado há muito tempo, as condições de trabalho não permanecem iguais, sendo impossível reproduzir tais condições, restando a análise das provas contemporânea e regularmente produzidas, sob a luz da legislação previdenciária (fls. 160/162). A audiência de tentativa de conciliação restou infrutífera, oportunidade em que foram ouvidos duas testemunhas do requerente, e as partes reiteraram os termos da inicial e da contestação (fls. 172/174). É o relatório. Fundamento e decido. A lei nº 8.213/91 combinada com o artigo 201, § 7º, I, da Constituição Federal confere aposentadoria por tempo de contribuição integral, ao segurado que completar 35 anos de contribuição, se homem, e 30 anos, se mulher, e preencher a carência do benefício que é de 180 meses de contribuição, conforme consta do artigo 25, inciso II da referida lei. Ainda, administrativamente, o Instituto Nacional de Seguro Social, acabou com a discussão da necessidade do pedágio para aposentadoria por tempo de contribuição integral, conforme demonstrado nas instruções normativas 118/2005, artigo 109 e 45/2010, artigo 223. O requerente, buscando sua inclusão nos quadros de beneficiários da Previdência Social, sustentou que, além do trabalho registrado em sua CTPS, exerceu atividades rurais. Para comprovar suas alegações, carreou aos autos o certificado de dispensa de incorporação (fls. 14), demonstrando sua profissão de lavrador, certidão de casamento (fls. 15), requerimento do benefício de aposentadoria por tempo de contribuição em vias administrativas (fls. 16/22), cópia da CTPS (fls. 23/58), pesquisa interna homologada, solicitada pela agência da previdência social (fls. 59/61), guia de recolhimento de contribuinte individual (fls. 62/63), documentação utilizada em vias administrativas, bem como o indeferimento do benefício (fls. 64/82). Como é cediço, para fins de comprovação do tempo de labor rural, o início de prova material deve ser contemporâneo à época dos fatos a provar. Nesse sentido é o teor da súmula 34 da Turma Nacional de Uniformização de Jurisprudência dos Juizados Especiais Federais. Também nessa linha: RESP nº 434.015/CE; - AgRg nos EDCL no Ag nº 561.483/SP; - AgRg no RESP nº 712.825/SP; AR nº 1808/SP; PU nº 2004.85.01.003420-0/SE -Turma de Uniformização (Julgamento de 26 de junho de 2006). Assim, considerando as provas colhidas aos autos, é possível o reconhecimento da condição de lavrador da parte autora pelos períodos de 28/02/1964 (referente depoimento testemunhal acostado aos autos, e a idade de 14 anos completada, fls. 13 e 173/174) a 13/02/1973 (anterior o seu primeiro recolhimento, conforme CTPS, fls. 24). Cumpre consignar que deve ser considerado o trabalho realizado pelo autor após ter completado 14 anos de idade, ou seja, a contar de 28/02/1964, fl. 13. Além disso, a prova testemunhal trazida corrobora os indícios de prova material acima mencionada, no sentido de que o requerente exerceu trabalho no campo, até o ano de 1973 (fls. 14,15 e 24). Os depoimentos, congruentes, harmônicos e, ainda, de acordo com as alegações iniciais, somados às provas documentais apresentadas, comprovam o desempenho das atividades da parte autora no meio rural antes do primeiro registro na CTPS. Assim, o período de labor campesino, temos evidente a atividade rural compreendida no período entre 01/01/1964 a 13/02/1973 (anterior o seu primeiro recolhimento, conforme CTPS, fls. 24), que perfazem 08 anos, 11 meses e 16 dias. Por outro canto, tenho que o tempo de trabalho rural anterior ao advento da Lei 8.213/91 pode ser considerado para concessão do benefício previdenciário, exceto para efeitos de carência, nos termos do artigo 55, § 2º, Lei 8.213/91. A este propósito editou-se a Súmula 24, JEFs: ?O tempo de serviço do segurado trabalhador rural anterior ao advento da Lei nº 8.213/91, sem o recolhimento das contribuições previdenciárias, pode ser considerado para a concessão dos benefícios do Regime Geral de Previdência Social (RGPS), exceto para efeitos de carência, conforme a regra do art. 55, § 2º, da Lei nº 8.213/91?. Desse dispositivo legal, depreende-se que a atividade rural desempenhada em data anterior a novembro de 1991 pode ser considerada para averbação do tempo de serviço, sem necessidade de recolhimento de contribuições previdenciárias, exceto para fins de carência. A partir do advento da Lei 8.213/91, cabe ao segurado especial o recolhimento de contribuições previdenciárias facultativas, se pretender o cômputo do tempo de serviço rural para fins de obtenção de outros benefícios que não os arrolados no inciso I do artigo 39. (TF3. Agravo legal em Apelação Cível Nº 0012325-80.2008.4.03.9999/ SP. 8ª T. Rel. Marcia Hoffmann. j. 06.02.2012). Assim, para efeito de carência do benefício da aposentadoria por tempo de contribuição, não é possível computar-se o tempo de labor rural desprovido de recolhimento aos cofres previdenciários. Decorre disso, aferir-se se o tempo laborado na atividade urbana satisfará a referida carência. Salienta-se que a verificação do direito do segurado ao recebimento de aposentadoria por tempo de serviço ou de contribuição deve partir das seguintes balizas: 1. A aposentadoria por tempo de serviço (integral ou proporcional) somente é devida se o segurado não necessitar de período de atividade posterior a 16.12.1998, sendo aplicável o art. 52 da Lei 8.213/1991. Em havendo contagem de tempo posterior a 16.12.1998, somente será possível a concessão de aposentadoria por tempo de contribuição. Nos termos do art. 4º da EC nº 20/1998, ?o tempo de serviço considerado pela legislação vigente para efeito de aposentadoria, cumprido até que a lei discipline a matéria, será contado como tempo de contribuição?. 2. Cumprida o requisito específico de 35 anos de contribuição, se homem, e 30 anos, se mulher, o segurado faz jus à aposentadoria por tempo de serviço/contribuição. Se poderia se aposentar por tempo de serviço em 16.12.1998, deve-se conceder a aposentadoria mais vantajosa, nos termos do art. 122 da Lei 8.213/1991. Inaplicável a regra de transição da EC 20/1998, art. 9º, caput, relativa à aposentadoria integral, por ser mais gravosa que a regra geral (CF/1988, art. 201, §7º, I). 3. O segurado filiado ao RGPS antes da publicação da Emenda 20/1998 faz jus à aposentadoria por tempo de serviço/contribuição proporcional. Se os requisitos foram preenchidos antes da publicação da emenda, aplica-se a redação original do art. 202, II, da CF/1988 (30 anos de tempo de serviço para homem e 25 para mulher). Caso contrário, são requisitos cumulativos: I) idade mínima de 53 (homem) e 48 (mulher); II) Soma de 30 anos (homem) e 25 (mulher) com o período adicional de contribuição de 40% do tempo que faltava, na data de publicação da Emenda, para alcançar o tempo mínimo acima referido (EC 20/1998, art. 9º, §1º, I, e cálculo da RMI na forma do inciso II). 4. Não preenchendo o segurado os requisitos necessários para obter aposentadoria por tempo de serviço/contribuição até 28.11.1999, o cálculo da renda mensal do seu benefício deverá seguir as diretrizes da Lei nº 9.876/1999 que instituiu a figura do fator previdenciário. As questões trazidas à análise deste Juízo limitam-se à discussão em torno do direito da parte autora ao cômputo de tempo de serviço/contribuição para fins de concessão de benefício previdenciário de tempo de serviço/contribuição. Os períodos de trabalho elencados na inicial estão registrados em CTPS. No presente caso, aliás, nem sequer poderá ser invocado o relativo valor probante da CTPS (Súmula 12 do TST), porquanto não foi produzida qualquer prova indicativa da ausência de veracidade das anotações nela constantes. De fato, se referido instrumento não faz prova absoluta das relações empregatícias que nele constam, ao menos gera uma presunção relativa a respeito, cabendo à parte que as questionar o ônus de ilidir tal presunção. Ao contrário, no caso, as datas são coincidentes, conforme se abstrai do CNIS juntado às fls. 115. Nesse diapasão, constata-se que a parte autora verteu contribuições nos períodos de 14/02/1973 a 25/04/1973; 05/07/1973 a 03/10/1974; 09/10/1974 a 11/02/1980; 01/07/1980 a 31/12/1985; 07/01/1986 a 01/12/1986; 02/12/1986 a 31/05/189; 08/08/1994 a 01/04/1997 (conforme CNIS apresentado às fls. 115 e CTPS às fls. 23/58), que perfazem 18 anos 01 mês e 22 dias de tempo de contribuição, os quais restaram incontroversos. No que tange as contribuições como contribuinte individual, o requerente alegou na exordial os períodos de 01/12/1999 a 01/02/2011 (data do ajuizamento da ação); contudo tenho que razão não lhe assiste, uma vez que não juntou aos autos todas as contribuições que recolheu como contribuinte individual. Desse modo, deve ser reconhecido apenas o período compreendido entre 01/12/1999 a 01/05/2010, demonstrado no CNIS às fls. 115. Assim, o requerente contribuiu 10 anos 04 meses e 31 dias como contribuinte individual. A parte autora requer que seja reconhecido também o período compreendido entre 01/06/1989 a 07/08/1994 e 01/09/1997 a 17/01/1998 como laborados em condições especiais, períodos esses que a parte autora laborou como cargo de guarda noturno e vigilante noturno mediante informação da CTPS fls. 41 e CNIS fls. 115. Temos que a profissão de vigilante é considerada profissão especial por analogia a função de guarda, conforme Súmula nº. 26 do JEF: A atividade de vigilante enquadra-se como especial, equiparando-se à de guarda, elencada no item 2.5.7. do Anexo III do Decreto n. 53.831/64. Assim: PREVIDENCIÁRIO. REVISÃO DE COEFICIENTE DE APOSENTADORIA PROPORCIONAL POR TEMPO DE SERVIÇO. RECONHECIMENTO DE PERÍODOS DE LABOR ESPECIAL, COM CONVERSÃO PARA TEMPO COMUM. REMESSA OFICIAL E APELAÇÃO AUTÁRQUICA IMPROVIDAS. - Comprovada a realização de trabalho nocivo, com conversão para tempo comum, nos intervalos de labor de 07.04.81 a 06.11.90, 08.03.91 a 16.09.91, 07.04.92 a 22.10.93 e 18.02.94 a 28.04.95. - Existência de formulários dando conta da realização, pelo falecido autor, da atividade de vigia. - Embora os Decretos 53.831/64 e 83.080/79 não tenham previsto a profissão de “vigilante” como insalubre, o rol das atividades constantes nos referidos decretos é meramente exemplificativo, de modo que, a jurisprudência tem entendido que os vigilantes, por exercerem a ocupação de guarda, desempenham trabalho de natureza especial. Ressalte-se que o fato de não ter ficado comprovado o desempenho da atividade de vigilante munido de arma de fogo não impede o reconhecimento do tempo especial, uma vez que o Decreto 53.831/64, código 2.5.7, não impõe tal exigência para aqueles que tenham a ocupação de “guarda”, a qual é a mesma exercida pelos vigilantes. Entendimento jurisprudencial. - Correto o decisum que entendeu cabível a revisão pretendida. - Remessa oficial e apelação autárquica improvidas. (Tribunal Regional Federal da 3ª Região. APELREEX ? APELAÇÃO / REEXAME NECESSÁRIO 952197. Processo: 0000114-87.2003.4.03.6183 00. UF: SP. Órgão Julgador: Oitava Turma. Data do Julgamento: 29/07/2013. Relator: Desembargadora Federal Therezinha Cazerta). Assim, convertendo os períodos supramencionados de especial para comum, no fator 1.4, o requerente comprovou tempo de serviço de 07 anos 09 meses e 16 dias. Somando os períodos supramencionados temos: 08 anos, 11 meses e 16 de tempo laborado em lides rurais. 18 anos, 01 mês e 22 dias de tempo de contribuição, sem a contagem do labor em condições especiais e como contribuinte individual. 10 anos 04 meses e 31 dias de tempo de contribuição como contribuinte individual. 07 anos 09 meses e 16 dias de tempo de contribuição em atividade especial: Guarda e Vigilante Noturno. Totalizando têm-se 36 anos 04 meses e 09 dias de tempo de contribuição, bem como a carência exigida pela legislação previdenciária e, somando com o período laborado em lides rurais (08 anos, 11 meses e 16 dias), temos 45 anos, 03 meses e 25 dias. Posto isso, julgo PROCEDENTE o pedido formulado na presente ação, para: a. declarar que o autor SEBASTIÃO LIMA DA SILVA exerceu trabalho rural no período de 28/02/1964 a 13/02/1973, como também, exerceu atividade especial nos períodos de 01/06/1989 a 07/08/1994 e 01/09/1997 a 17/01/1998; b. condenar o requerido INSS a averbar os períodos mencionados na letra anterior, nos termos da fundamentação, e; c. implantar em favor do requerente o benefício previdenciário aposentadoria por tempo de contribuição, a partir do indeferimento administrativo (fls. 79/82, 13/12/2010), calculando conforme as regras gerais previstas no artigo 29 da Lei n°. 8.213/91. Os valores em atraso deverão ser corrigidos monetariamente a partir de cada vencimento, acrescidos, ainda, de juros de mora que incidirá, uma única vez, com base nos índices oficiais de remuneração básica aplicados à caderneta de poupança, nos termos do art. 1º F, da Lei 9.494/97, desde a citação (artigo 219 do Código de Processo Civil). Por conseguinte, JULGO RESOLVIDO o processo, com apreciação de mérito, nos termos do artigo 269, inciso I, do Código de Processo Civil. Arcará o requerido com o pagamento das despesas processuais e honorários advocatícios, estes fixados em 10% do valor atualizado da condenação, não incidentes sobre as prestações vincendas, em atenção às alíneas do § 3º, do artigo 20, do CPC e ao enunciado Sumular nº 111, do C. STJ. Sem recolhimento de custas, pois a ré goza de isenção (art. 4º da Lei 9.289/1996). Não é possível determinar se o valor da controvérsia recursal é inferior a sessenta salários mínimos (art. 475, § 2º do CPC); assim, decorrido o prazo legal para a apresentação de eventuais recursos voluntários, remetam-se os autos a superior apreciação do Egrégio Tribunal Regional Federal da 3ª Região, para o reexame necessário. Nesse sentido: STJ - EREsp 934642/PR, Rel. Min. Ari Pargendler, julgado em 30-06-2009. P.R.I.C. Monte Alto, 11 de setembro de 2013. GILSON MIGUEL GOMES DA SILVA Juiz de Direito ", {
        'entities': []
    }),

    ("FAZENDA PÚBLICA DO ESTADO DE SÃO PAULO - Vistos, Trata-se da ação de obrigação de fazer proposta por NANCI CLEO FERREIRA LOPES contra FAZENDA PÚBLICA DO ESTADO DE SÃO PAULO, onde se traz prova inequívoca da morte da autora (fls. 273) e se requer a extinção do feito (fls.276, 279 e 282/284). A tutela foi deferida à fls. 73/77 em relação ao serviço de HOME CARE, em sede de agravo e à fls. 129, em relação aos medicamentos ALOIS 10mg e OXIBUTININA 5,0mg e às FRALDAS GERIÁTRICAS. É o relatório. DECIDO. Com efeito, caracterizada está a hipótese de extinção do processo, sem resolução do mérito, nos termos do art. 267, inciso IX, do Código de Processo Civil, vez que o direito aqui discutido é intransmissível. Face ao exposto e por tudo mais que consta dos autos, DECLARO EXTINTA, sem resolução do mérito, a ação em epígrafe. Diante do principio da causalidade, considerando os fundamentos expostos na fundamentação desta sentença, condeno a parte ré ao reembolso das custas processuais, ficando delas isenta em razão de sua natureza jurídica e ao pagamento de honorários advocatícios, os quais fixo, por equidade, em um mil reais, considerando o grau de zelo e o tempo despendido pelo profissional, além da natureza repetitiva e pouco complexa da causa, corriqueira para entidade que propôs a demanda, instituída também para os fins de prestar assistência a doentes portadores de várias patologias e não somente daquela inserida em seu nome, salientando-se que no caso em questão não se está adstrito às percentagens mínima e máxima previstas no parágrafo terceiro do mesmo dispositivo legal (neste sentido: STJ 2ª Turma, REsp 260.188-MG, rel. Min. Eliana Calmon, j. 23.10.01, não conheceram, v.u., DJU 18.02.02, p. 302). Oficie-se ao DRS XV para cessação na prestação de serviço de HOME CARE e no fornecimento das FRALDAS GERIÁTRICAS, já que os fármacos Memantina 10mg (Alois) e Oxibutinina 5,0mg foram suspensos à fls. 261/262. Transitada esta em julgado, arquivem-se. P.R.I. e C.", {
        'entities': []
    }),

    ("Esporte Clube Banespa de Campo e Nautica - Vistos.Converto o julgamento em diligência, haja vista que por ocasião do saneamento (fls. 157/158) foi determinada a realização de prova pericial.Dessa maneira, determino a realização da perícia; no entanto, diante da desistência da autora quanto ao pedido de recuperação ambiental da área (fls. 226/227), a perícia deverá se ater somente em relação à existência de benfeitorias na área reintegranda e à propriedade. Para isso, apresento os quesitos do Juízo:A) a quem pertence a área?B) há, ou haviam, benfeitorias na área?C) caso positivo o item acima, deverá o Sr. Perito precisar se por ocasião da imissão da autora na posse do imóvel (fls. 43) as benfeitorias já existiam e se havia autorização para a requerida mantê-las e usufruí-las.D) outras considerações que o Sr. Perito julgar pertinentes para o desfecho da demanda. Nomeio perito o Sr LUPÉRCIO CHAGAS NETO, independente de compromisso.Fixo os honorários periciais em R$2.000,00, os quais deverão ser suportados pela parte requerida, uma vez que a requereu às fls. 140. Prazo para recolhimento: 15 dias.Com o depósito, intime-se o Sr. Perito para realizar o trabalho, haja vista que as partes já apresentaram seu quesitos e indicaram assistentes técnicos às fls. 161/163 e 165/166.Laudo pericial em 30 dias.", {
        'entities': [(951, 986, 'VALOR')]
    }),

    ("Alvaro’s Car Comércio de Veículos Ltda - - ALVARO LUIS FILHO - - Vera Lucia dos Santos Luis - Vistos.Determinei, através do sistema BacenJud, o bloqueio de eventual numerário existente em instituições financeiras em nome dos executados. Tendo-se em vista que o valor bloqueado “on line” (R$ 2,92) é irrisório em relação ao débito exequendo, com fulcro no art. 836 do Código de Processo Civil, procedi, em 29/03/2016, ao desbloqueio da conta do executado, conforme recibo que segue. Diante da não localização de ativos financeiros, indique a exequente outros bens para constrição. Prazo: 15 dias.No silêncio, fica suspensa a execução, por analogia ao artigo 921, III, do Código de Processo Civil, aguardando-se provocação no arquivo. Int.", {
        'entities': [(289, 296, 'VALOR')]
    }),

    ("CONDOMINIO EDIFÍCIO SALLES JUNIOR - Vistos etc. 1 - DETERMINO a qualquer Oficial de Justiça desta jurisdição que, em cumprimento ao presente, e utilizando, se necessário, as prerrogativas do artigo 172, § 2º, do CPC, proceda à 1. CITAÇÃO do executado CONDOMÍNIO EDIFÍCIO SALLES JUNIOR para, no prazo de 03 (três) dias, pagar a dívida no valor de R$ 38.026,14 que deverá ser atualizada até a data do efetivo pagamento, acrescida dos honorários advocatícios da parte exequente arbitrados em 10% (dez por cento) sobre o valor atualizado do débito, conforme pedido inicial, cuja cópia segue anexa e faz parte integrante deste. Caso o(a,s) executado(a,s) efetue o pagamento no prazo acima assinalado, os honorários advocatícios serão reduzidos pela metade (art.652-A, parágrafo único, do Código de Processo Civil). 2. No prazo para embargos, reconhecendo o crédito do(a) exequente e comprovando o depósito de 30% (trinta por cento) do valor em execução, inclusive custas e honorários de advogado, o(a,s) executado(a,s) poderá(ão) requerer autorização do Juízo para pagar(em) o restante do débito em até 6 (seis) parcelas mensais, corrigidas pela Tabela Prática do Tribunal de Justiça e acrescidas de juros de 1% (um por cento) ao mês (Art. 745-A do Código de Processo Civil). O não pagamento de qualquer das prestações implicará, de pleno direito, o vencimento das subseqüentes e o prosseguimento do processo, com o imediato início dos atos executivos, imposta ao executado multa de 10% (dez por cento) sobre o valor das prestações não pagas e vedação à oposição de embargos (art. 745-A, § 2º, do Código de Processo Civil). 3. Não efetuado o pagamento, nem o parcelamento, munido da segunda via do mandado, o oficial de justiça procederá, de imediato, à PENHORA e AVALIAÇÃO de tantos bens quantos bastem para a satisfação da dívida, lavrando-se o competente auto, intimando-se o executado de tais atos na mesma oportunidade e efetivando-se o depósito na forma da lei. Caso não sejam localizados bens, o(a,s) executado(a,s) deve(m) ser intimado(a,s) a indicá-los em 05 (cinco) dias, sob pena de multa de até 20% do valor da causa, se constatada omissão (arts. 600 e 601 do Código de Processo Civil). 4. PRAZO PARA EMBARGOS: 15 (quinze) dias, contados da data da juntada aos autos do mandado de citação (artigo 738 do Código de Processo Civil). 2 - Ao autor: em conformidade com o Comunicado CSM 2195/14, providencie o recolhimento da taxa de impressão da(s) contrafé(s) - (R$0,55 por folha). CIENTIFIQUE(M)-SE a(o)(s) interessados(o)(s) dos termos da ação em epígrafe, cuja cópia da petição inicial segue em anexo. Servirá o presente, por cópia digitada, como mandado. Cumpra-se na forma e sob as penas da Lei. Intime-se.", {
        'entities': [(335, 356, 'VALOR'), (2468, 2474, 'VALOR')]
    }),

    ("Embargado: Carlos Gomes do Nascimento - Magistrado(a) Maria do Carmo Honorio - Rejeitaram os embargos. V. U. (Para eventual interposição de recurso extraordinário, comprovar o recolhimento de R$ 145,36 na Guia de Recolhimento da União - GRU, do tipo ‘Cobrança’ - Ficha de Compensação, a ser emitida no sítio eletrônico do Supremo Tribunal Federal; e o valor referente a porte de remessa e retorno em guia FEDTJ, código 140-6, no Banco do Brasil S.A. ou internet, conforme tabela “D” da Resolução nº 500 do STF, de 16 de janeiro de 2013 e Provimento nº 831/2004 do CSM)", {
        'entities': [(177, 202, 'VALOR')]
    }),

    ("Milton Luiz Tezzi - Certifico e dou fé que os valores a serem recolhidos, para eventual recurso da r. Sentença de fls. 38 são até a presente data: 1) Custas Processuais no valor de R$ 96,85 (Gare - Cód. 230-6); 2) Porte de Remessa e Retorno, no valor de R$ 29,50 (FEDT - Cód. 110-4); 3) Despesas Processuais, no valor de R$ 52,18 (FEDT - Cód. 120-1), referente a (01) Carta com Registro + Aviso de Recebimento + Mão Própria (R$ 17,50 cada); (01) Carta de Remessa Local (R$ 7,50 cada); 02 ato(s) Mandado (R$ 13,59 = 01 Ato). Nada Mais.", {
        'entities': [(170, 190, 'VALOR'), (243, 264, 'VALOR'), (310, 330, 'VALOR'), (426, 434, 'VALOR'), (471, 479, 'VALOR'), (505, 513, 'VALOR')]
    }),

    ("META PAINEIS LTDA - - Gian Arturo Augusto de Miccolis - Vistos. Defiro o pedido de pesquisa de endereço através de Infojud e Bacenjud mediante o recolhimento de custas de pesquisa (R$ 12,20 por sistema e por CPF/CNPJ). Int", {
        'entities': [(182, 190, 'VALOR')]
    }),

    ("Vistos.Fls. 179: foi determinado o bloqueio de valores pertencentes ao(s) executado(s) pelo sistema BacenJud.Intime-se o(a) exequente para que se manifeste sobre o Detalhamento de Ordem Judicial de Bloqueio de Valores enviado pelo BACEN, que adiante segue, dando conta de que o bloqueio de valores pertencentes ao(s) executado(s) resultou positivo, no importe de R$ 1.034,81, cuja transferência para conta judicial já foi solicitada, devendo o valor transferido permanecer à ordem e à disposição deste Juízo.Intime(m)-se o(a)(s) executado(a)(s), na(s) pessoa(s) de seu(s) advogado(s), ou pessoalmente, na falta de representação, recolhendo-se nesse caso a diligência necessária, a impugnar no prazo legal a penhora ora efetivada.Decorrido o prazo, sem impugnação pela(s) parte(s) executada(s), expeça-se o mandado de levantamento pertinente em favor da credora.Int", {
        'entities': [(350, 375, 'VALOR')]
    }),

    ("Vistos. 1. Providencie o Cartório a exclusão da tarja de segredo de justiça, por não se enquadrar nas hipóteses previstas em lei. 2. Concedo os benefícios da Assistência Judiciária Gratuita à parte autora, anote-se. 3. Ante os termos da inicial e respectiva prova documental, presentes os requisitos legais, DEFIRO A LIMINAR PLEITEADA para determinar que o requerido exiba em Juízo, no prazo de defesa, cópia do contrato firmado entre as partes de n° 000000269530770, no valor de R$ 29,72 (vinte e nove reais e setenta e dois centavos) , conforme descrito na inicial. 4. Cite(m)-se, com as advertências da lei (arts. 285 e 319, do Código de Processo Civil). 5. Ficando o(s) réu(s) advertido(s) do prazo de 05 (cinco) dias para apresentar(em) a defesa, sob pena de serem presumidos como verdadeiros os fatos articulados na inicial, nos termos do artigo 285 do Código de Processo Civil. Cumpra-se na forma e sob as penas da lei. 6. Expeça-se carta de citação e intimação. Intimem-se e cumpra-se.", {
        'entities': [(469, 489, 'VALOR')]
    }),

    ("Controlsystem Automação Industrial Ltda Epp e outros - Fica a exequente devidamente intimada a proceder ao recolhimento de mais uma guia de R$ 24,40 para pesquisa de informações/bens em nome dos executados tendo em vista equívoco na intimação de fls 23 dos autos quando naquela ocasião constou o valor de R$ 48,80, frisando-se que são 03 réus no total para mais 02 sistemas a serem pesquisados.", {
        'entities': [(129, 149, 'VALOR'), (295, 314, 'VALOR')]
    }),

    ("Porto Seguro Cia de Seguros Gerais - Vistos. Realizado o procedimento do sistema BACENJUD, verifica- se, conforme expedientes que seguem, que ocorreu bloqueio em ativos financeiros de titularidade do devedor (R$ 7.344,33), conforme autorizado pelo artigo 665-A, do Código de Processo Civil. Tal valor satisfaz integralmente o crédito. Com a juntada da guia de depósito judicial, fica convertido em penhora o bloqueio realizado. Após a juntada da guia, intime-se a parte devedora para que, querendo, ofereça impugnação no prazo de quinze dias, nos termos do artigo 475-J, § 1º, do Código de Processo Civil. Intime-se.", {
        'entities': [(210, 221, 'VALOR')]
    }),

    ("H. D. R. - Vistos. Cite-se o devedor para que, em 3 dias, efetue o pagamento do débito de R$ 634,64 (devidamente atualizado e acrescido das pensões que se vencerem ao longo da demanda) ou comprove que já o fez ou ainda justifique a impossibilidade de efetuá-lo, sob pena de prisão. Servirá o presente, por cópia digitada, como mandado. Cumpra-se na forma e sob as penas da Lei. Intime-se. (fls. 10)", {
        'entities': [(81, 100, 'VALOR')]
    }),

    ("C P Ribeiro Agropecuaria Epp - Certifico e dou fé que, nos termos do COMUNICADO CG Nº 1307/2007, ENCAMINHO estes autos à publicação para: que o (a) autor (a) /exequente recolha, em 5 cinco dias, de acordo com o COMUNICADO CSM 170/2011, a taxa no valor de R$ 12,20", {
        'entities': [(244, 264, 'VALOR')]
    }),

    ("Município de Votuporanga - Luiz Carlos de Oliveira - Vistos.Trata-se de cumprimento de sentença proposto por Luiz Carlos de Oliveira em face do Município de Votuporanga, objetivando o pagamento de R$800,00 referente a honorários sucumbenciais, nos autos do processo principal de nº 1002423- 39.2015.8.26.0664.O Município, intimado na forma do art. 535 do Código de Processo Civil, não se manifestou (fls. 29 e 30). Portanto, homologo os cálculos de fls. 4.Providencie o advogado o peticionamento eletrônico, nos termos do Comunicado 394/2015 de 25/06/2015, através do portal e-saj petição intermediária, cuja funcionalidade específica (RPV ou precatório) estará habilitada, tanto para processos físicos como digitais, podendo seguir as instruções do link: www.tjsp.jus.br/sistemas/mensagem/ comunicado2.Aspx, para posterior requisição de pagamento.Após, arquive-se este incidente.Intime-se.", {
        'entities': [(185, 206, 'VALOR')]
    }),

    ("Fls. 98. Manifeste-se no prazo de 05 dias. “CERTIFICO eu, Oficial de Justiça, que em cumprimento ao mandado nº 564.2014/004670-0 dirigi-me ao endereço: Estrada Alberto Silva, Tatetos, e aí sendo deixei de citar os requeridos, uma vez que não encontrei o numero 288, apesar de percorrer a rua em toda a sua extensão. Ato continuo, dirigi- me a Estrada dos Camponeses, 800, Bairro dos Fincos, e lá, fui informado pelo Sr. Rodolfo Korb, de que o requerido Carlos Ramos da Silva é um ex-caseiro seu, e que ele desconhece o seu paradeiro. Ato continuo, dirigi-me a Rua Roque Silva, Santa Terezinha, e la sendo, não encontrei o numero 288, cuja numeração vai até o numero 60. Diante do exposto, devolvo o presente mandado a Cartório, para os devidos fins. Número de atos: 3 + 9l - Tatetos - 26,5 Km - tatetos a Fincos - 9 Km - Fincos a Santa Terezinha - 12 Km - R$61,02 - A recolher: R$13,59. Na inércia, intime-se pela via postal, para promover o regular andamento do feito, no prazo de 48 horas, sob pena de extinção e arquivamento(art. 598 c.c. art. 267, III, ambos do Código de Processo Civil).", {
        'entities': [(857, 864, 'VALOR'), (879, 886, 'VALOR')]
    }),

    ("Banco Safra S/A. - Vistos.Ciência à parte credora acerca do depósito realizado nos autos em 26/06/2017 pela parte devedora no valor de R$ 987,00 .Expeça-se Alvará com prazo de 30 dias, ao(s) credor(es) para o levantamento, observando-se os poderes estabelecidos na procuração, devendo a parte credora providenciar sua impressão através do site www.tjsp.jus.br. Após, deverá a parte credora manifestar-se acerca da satisfação no recebimento de seu crédito, ressaltando que o seu silêncio será interpretado como satisfação do crédito recebido e a execução extinta.Int.", {
        'entities': [(124, 145, 'VALOR')]
    }),

    ("III - Diante do exposto e por tudo que dos autos consta, CONCEDO A ORDEM para que seja efetivada a matrícula do ( a ) ( s ) impetrante ( s ) em escola de educação infantil próxima à ( s ) sua ( s ) residência ( s ), sob pena de pagamento da multa diária no valor de R$ 5,00, tornando definitiva a liminar concedida. Incabíveis honorários advocatícios em mandado de segurança, em atenção à Súmula 512 do C. Supremo Tribunal Federal. Arbitro os honorários em caso de Defensor Dativo em R$ 343,76 ( Cód. 501 ), expedindo-se oportunamente a respectiva certidão, com atualização do referido valor, se o caso. ", {
        'entities': [(255, 274, 'VALOR'), (485, 494, 'VALOR')]
    }),

    ("Epp - Michele Oliveira de Paula - Vistos.Dispensado o relatório.Fundamento e decido.Trata-se de ação de execução.Com efeito, a pessoa jurídica, como regra, não pode figurar no polo ativo perante o Juizado Especial, com exceção das microempresas, conforme consta, expressamente, do art. 8° da LJE.Veja-se:”Art. 8º Não poderão ser partes, no processo instituído por esta Lei, o incapaz, o preso, as pessoas jurídicas de direito público, as empresas públicas da União, a massa falida e o insolvente civil. § 1o Somente serão admitidas a propor ação perante o Juizado Especial: I - as pessoas físicas capazes, excluídos os cessionários de direito de pessoas jurídicas;II - as microempresas, assim definidas pela Lei nº 9.841, de 5 de outubro de 1999; III - as pessoas jurídicas qualificadas como Organização da Sociedade Civil de Interesse Público, nos termos da Lei nº 9.790, de 23 de março de 1999; IV - as sociedades de crédito ao microempreendedor, nos termos do art.1º da Lei nº 10.194, de 14 de fevereiro de 2001”. De outra parte, não se ignora o cancelamento do Enunciado nº 49 do FONAJE, que dispunha o seguinte: “as empresas de pequeno porte não poderão ser autoras nos Juizados Especiais”, mas o fato é que a norma trazida pela Lei Complementar nº 123/06 afigura-se inconstitucional. Assim, deve ser reconhecida, incidentalmente, a inconstitucionalidade do dispositivo, qual seja: o artigo 74 da Lei Complementar nº 123, de 14 de dezembro de 2006, em face da norma contida no art. 5º, inc. XXXV, da Constituição da República Federativa do Brasil (princípio da inafastabilidade).Buscou o legislador contemplar, apenas, às pessoas jurídicas de pequena monta, haja vista que, se fosse possível que sociedades empresárias maiores postulassem perante o Juizado, haveria uma enormidade cobrança de empresas, prejudicando a celeridade dos processos de pequenos valores, em que figuram, como parte ativa, pessoas físicas.Em resumo, caso seja admitida tal ampliação dos legitimados para o rito sumaríssimo, haverá um congestionamento considerável dos Juizados, com o desprestígio da instituição e o desvirtuamento da norma de procedimento.Portanto, empresas de pequeno porte não podem propor ação no Juizado Especial Cível, posto que não estão previstas no inciso II, do artigo 8º. Ante o exposto, JULGO EXTINTO o processo, sem julgamento de mérito, por falta de pressuposto processual, na forma do art. 485, IV, do Código de Processo Civil, c. c. o art. 51, II, da Lei nº 9.099/95.Sem condenação em verbas de sucumbência (art. 55 da LJE).P.R.I. [O 1º Ofício do JEC informa que: O prazo legal para interposição de recurso será contado em DIAS CORRIDOS, conforme Enunciado 74 do FOJESP: todos os prazos, no Sistema dos Juizados Especiais, serão contados de forma contínua, excluindo o dia do começo e incluindo o dia do vencimento. O valor do preparo é de R$ 235,50]", {
        'entities': [(2829, 2862, 'VALOR')]
    }),

    ("Vistos. Tendo em vista que o bloqueio de valores se efetivou em relação às contas indicadas (R$5.239,95), determinei nesta data a transferência do numerário suficiente para conta judicial, conforme cópia do protocolo que segue. Noticiada a transferência, intime-se o executado para manifestar-se, querendo, no prazo de 10 dias (art. 668 do CPC). Int.", {
        'entities': [(94, 104, 'VALOR')]
    }),

    ("Guarujá Veículos Construções Ltda - *Fica o (a) DD .Advogado (A) que mantém os presentes autos em seu poder com o respectivo prazo expirado. Devidamente notificado por meio desta publicação, a devolver os autos a Cartório no prazo de vinte e quatro horas, sob pena de expedição de mandado de busca e apreensão. Além de sujeitar-se ás normas disciplinadoras relativa a prazos excedidos (art. 89,2º ,inciso IV do Estatuto da Ordem dos Advogados do Brasil, e Art. 102 das normas de serviço a Corregedoria Geral da Justiça do Estado de São Paulo)”", {
        'entities': []
    }),

    ("Volkswagen do Brasil Indústria de Veículos Automotores Ltda - Ante o exposto, julgo o pedido PROCEDENTE, nos termos do artigo 269, I, do Código de Processo Civil para: I - CONDENAR a requerida Volkswagen do Brasil Indústria de Veículos Automotores Ltda., qualificada nos autos, ao pagamento do importe de R$ 1.071,02 (mil e setenta e um reais e dois centavos) a titulo de repetição de indébito, à requerente, corrigidos monetariamente de acordo com a tabela do Tribunal de Justiça do Estado de São Paulo desde o desembolso e com juros moratórios de 1% ao mês a partir da citação, até o efetivo pagamento. II CONDENAR em devolver o importe de 20% (vinte por cento) sobre o valor total do bem, constante da tabela vigente, na data dos fatos, a título de abatimento pela depreciação do bem em razão do evento. III CONDENAR a requerida a indenizar moralmente a parte autora, no valor de R$ 13.911,48 (treze mil e novecentos e onze reais e quarenta e oito centavos), sendo este valor corrigido monetariamente a partir desta data, até o efetivo pagamento, conforme tabela prática do TJSP, acrescida de juros moratórios de 1% ao mês a partir da citação. Deixo de carrear as verbas da sucumbência em razão do disposto no artigo 55 da Lei n.º 9.099/95. Na hipótese de recurso, o preparo será efetuado, independentemente de intimação, nas quarenta e oito horas seguintes à interposição do recurso e deverá corresponder à soma das parcelas previstas nos incisos I e II do art. 4º da Lei nº 11.608/03, sendo no mínimo 5 UFESP’S para cada parcela, em cumprimento ao artigo 54, parágrafo único, da Lei nº 9.099/95, sob pena de deserção (Enunciado 11 do Colégio Recursal da 48ª Circunscrição Judiciária Guaratinguetá). Na hipótese de não proceder ao recolhimento integral do preparo recursal no prazo do artigo 42 da Lei nº 9.099/95, o recurso será considerado deserto, sendo inaplicável o artigo 511 do Código de Processo Civil. Entretanto, em caso de indeferimento do benefício da gratuidade no momento do exame de admissibilidade do recurso, conceder-se-á o prazo de 48 (quarenta e oito) horas para o recolhimento do preparo, a contar da ciência do indeferimento (Enunciado 12 do Colégio Recursal da 48ª Circunscrição Judiciária Guaratinguetá). Com o trânsito em julgado, intime-se a parte vencida a efetuar o pagamento no prazo de 15 (quinze) dias, sob pena de o montante da condenação ser acrescido de multa no percentual de 10% (dez por cento) (LJE, 52, III c.c. CPC, 475-J, caput). Publique-se. Registre-se. Intimem-se", {
        'entities': [(282, 317, 'VALOR'), (872, 896, 'VALOR')]
    }),

    ("Vistos. Defiro o pedido do credor pela averbação na matrícula do imóvel enquanto não possibilitada a penhora. SErvindo estA como mandado DE AVERBAÇÃO, determino ao Oficial do Registro de Imóveis desta cidade e Comarca de presidente Venceslau que proceda à AVERBAÇÃO do processamento desta ação (em fase de execução/cumprimento de sentença R$ 9.265,84 - valor atualizado para setembro/2014) na matrícula do imóvel (nº 3.273 - copiada a fls. 261262) deixado pela genitora do executado FABIO DA SILVA PACITO, CPF 164.642.208-29, a fim de assim se conferir efeitos de publicidade e de conhecimento a terceiros eventualmente interessados. Esta decisão/mandado deverá ser impressa pelo interessado e levada a cumprimento ao Oficial do Registro de Imóveis desta cidade e Comarca de Presidente Venceslau, comprovando nos autos em dez dias o protocolo do expediente, ficando a seu cargo o pagamento de emolumentos e taxas eventualmente exigidos pelo Tabelião. Fixados os honorários da fase executória em decisão proferida a fls. 178. Em trinta dias Requeira o/a exequente o mais que entender de direito, em termos de prosseguimento do feito. Na inércia, cumpra-se o determinado a fls. 178, parte final. Intime-se.", {
        'entities': [(340, 351, 'VALOR')]
    })
    
]

In [34]:
# # Start with blank English model
# nlp = spacy.blank("pt")
# # Create blank entity recognizer and add it to the pipeline
# ner = nlp.create_pipe("ner")
# nlp.add_pipe(ner)
# # Add a new label
# ner.add_label("VALOR")

# # Start the training
# nlp.begin_training()

# # Train for 10 iterations
# for itn in range(10):
#     random.shuffle(TRAIN_DATA)
#     losses={}
#     # Divide examples into batches
#     for batch in spacy.util.minibatch(TRAIN_DATA, size=2):
#         texts = [text for text, annotation in batch]
#         annotations = [annotation for text, annotation in batch]
#         # Update the model
#         nlp.update(texts, annotations, losses=losses)
#         print(losses)

In [35]:
model = None
output_dir=Path("/media/rafael/D/2020 RAFAEL/Faculdade/ICs/IC Direito/codes/pre-processing/model_test/")
n_iter=10

if model is not None:
    nlp = spacy.load(model)  # load existing spaCy model
    print("Loaded model '%s'" % model)
else:
    nlp = spacy.load('pt_core_news_sm')  # create blank Language class
    print("Created blank 'pt-br' model")
    
# create the built-in pipeline components and add them to the pipeline
    # nlp.create_pipe works for built-ins that are registered with spaCy
if 'ner' not in nlp.pipe_names:
    ner = nlp.create_pipe('ner')
    nlp.add_pipe(ner, last=True)
# otherwise, get it so we can add labels
else:
    ner = nlp.get_pipe('ner')
    
# add labels
for _, annotations in TRAIN_DATA:
    for ent in annotations.get('entities'):
        ner.add_label(ent[2])

    # get names of other pipes to disable them during training
other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']
with nlp.disable_pipes(*other_pipes):  # only train NER
    optimizer = nlp.begin_training()
    for itn in range(n_iter):
        random.shuffle(TRAIN_DATA)
        losses = {}
        for text, annotations in tqdm(TRAIN_DATA):
            nlp.update(
                [text],  # batch of texts
                [annotations],  # batch of annotations
                drop=0.5,  # dropout - make it harder to memorise data
                sgd=optimizer,  # callable to update weights
                losses=losses)
        print(losses)
# for text, _ in TRAIN_DATA:
#     doc = nlp(text)
#     print('Entities', [(ent.text, ent.label_) for ent in doc.ents])
#     print('Tokens', [(t.text, t.ent_type_, t.ent_iob) for t in doc])
    
if output_dir is not None:
    output_dir = Path(output_dir)
    if not output_dir.exists():
        output_dir.mkdir()
    nlp.to_disk(output_dir)
    print("Saved model to", output_dir)

KeyboardInterrupt: 

In [ ]:
# test the saved model
print("Loading from", output_dir)
nlp2 = spacy.load(output_dir)
# nlp = spacy.load('pt_core_news_sm')
for conteudo in tqdm(conteudo_all):
    doc = nlp2(conteudo)
    for ent in doc.ents:
        print(ent.text, ent.start_char, ent.end_char, ent.label_)
#     print('Entities', [(ent.text, ent.label_) for ent in doc.ents])
#     print('Tokens', [(t.text, t.ent_type_, t.ent_iob) for t in doc])

### FIM TESTE TRAIN

In [ ]:
# reus_classificados.count(0), 
# reus_classificados, autores_classificados
# filtrados,
filtrados_autores
autores_classificados.count(0), 
autores_classificados
# primeiros_nomes_autor
# classify_download('Hortencia')
# filtrados_reus
#     for frase in pri_nome.ents:
#         print(frase.label_, frase.text)
#     print(primeiros_nomes[i])
#     print(list_index)

In [ ]:
spacy.explain('MISC')
# sp_data.loc[~sp_data['Reu'].isnull(),('Reu')]
sp_data.loc[~sp_data['Autor'].isnull(),('Autor')]
# sp_data['Reu'][5], sp_data['Conteúdo'][0] 
sp_data['Autor'][7], sp_data['Conteúdo'][7] 
# pd.Series(count_separator_post_filter) > 4
# # len(ind_range), data, np.array(count_separator_post_filter)

# l = []
# tt = back_sp_data['Conteúdo']

# for i in range (10):
    
#     for j,a in enumerate(tt.str.find(' - ')):
        
#         if (a != -1):
#             l.append(tt.loc[tt.str.find(' - ') != -1].str.slice(stop=a)[j])
#             tt.loc[j] = tt.str.slice(start= a + 3)[j]
# l, tt

In [ ]:
# USING FOR TAKE ALL ASSUNTOS OF DOCUMENTS
# ass = pd.Series(assuntos_all)
# ass_possible = pd.Series(partes_all)

# # a = ass.str.findall(regex_assunto1)
# teste_1 = ass
# teste_2 = ass_possible
# for i,assunto in enumerate(assuntos):

#     for j, possible_ass in enumerate(ass):
#         if(ass[j].lower() == assunto.lower()):
            
#             teste_1.loc[j] = assunto
            
#         if(ass_possible[j].lower() == assunto.lower()):
            
#             teste_2.loc[j] = assunto

##################################################################################
##################################################################################
##################################################################################
# new_sp_data.loc[:,('Assunto')][data_less_4[p.str.len() > 1].index] = assunto        

# def multiprocessing_compare_assunto(assuntos):
    
#     for i,assunto in enumerate(assuntos):
#         positions = data_less_4['Conteúdo'].str.find(assunto)
#         positions = positions[positions > -1]
        
#         if(len(positions.index) > 0):
#             for i,position in enumerate(positions):
# #                 print(array[positions.index], positions.index[i], position)

#                 if(position < array[positions.index[i]]):
#                     array[positions.index[i]] = position
#                     list_list.append([positions.index[i],assunto, position])
# #                     back_sp_data.loc[positions.index[i],('Assunto')] = assunto
#     return list_list
# #                     print(list_list)
# #                     print( back_sp_data.loc[positions.index[i],('Assunto')], assunto, positions.index[i])

# if __name__ == '__main__':
#     assuntos_1 = assuntos[0 : int(len(assuntos)/4)]
#     assuntos_2 = assuntos[int(len(assuntos)/4) : 2 * int(len(assuntos)/4)]
#     assuntos_3 = assuntos[2 * int(len(assuntos)/4) :3 * int(len(assuntos)/4)]
#     assuntos_4 = assuntos[3 * int(len(assuntos)/4):]
#     starttime = time.time()
#     pool = multiprocessing.Pool(4)
#     tete = pool.map(multiprocessing_compare_assunto, [assuntos_1, assuntos_2, assuntos_3, assuntos_4])
#     pool.close()
#     print('That took {} seconds'.format(time.time() - starttime))

##############################################################################
##############################################################################
##############################################################################
# p = data_less_4['Conteúdo'].str.findall('Fica a defesa')
# a = data_less_4['Conteúdo'].str.findall('tese')
# b = data_less_4['Conteúdo'].str.find('tese')
# d = data_less_4['Conteúdo'].str.find('teste')